# **Universal Data Loader**

In [32]:
# ============================================
# UNIVERSAL DATA PREPARER  (Auto CSV / Image)
# ============================================
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def prepare(path: str,
            target_col: str = "y",
            img_size=(224,224),
            flatten_images_for_ml: bool=True,
            handle_missing_strategy="mean",
            encode_categorical=True,
            normalize_numeric=True,
            test_size=0.2,
            val_size=0.1,
            random_state=42):
    """
    Universal data loader.
    But for tabular CSVs like bank-full.csv, it matches EXACTLY the preprocessing
    used during FNN model training:
        - y cleaned (yes/no → 1/0)
        - one-hot encoding with drop_first=True
        - StandardScaler on numeric columns
    """

    dtype = detect_dataset_type(path)
    result = {"dataset_type": dtype}

    # ============================
    # ---- TABULAR DATA ----------
    # ============================
    if dtype in {"csv", "excel", "json"}:

        df = load_tabular(path)

        # --- CLEAN TARGET COLUMN LIKE TRAINING ---
        if target_col not in df.columns:
            raise ValueError(f"Target column '{target_col}' not found. Available: {df.columns}")

        df[target_col] = df[target_col].astype(str).str.strip().str.lower()
        df[target_col] = df[target_col].replace({
            "yes": 1,
            "no": 0,
            "unknown": 0,
            "nan": 0,
        }).astype(int)

        # Ensure no NaN in target
        df[target_col].fillna(0, inplace=True)

        # --- SEPARATE FEATURES / TARGET ---
        y = df[target_col].values
        X_raw = df.drop(columns=[target_col])

        # --- ONE-HOT ENCODING (DROP FIRST) ---
        X_encoded = pd.get_dummies(X_raw, drop_first=True)

        # Store feature names (important for pruning input shape!)
        result["feature_names"] = list(X_encoded.columns)

        # --- TRAIN-VAL-TEST SPLIT ---
        X_train, X_temp, y_train, y_temp = train_test_split(
            X_encoded, y, test_size=test_size, random_state=random_state, stratify=y
        )
        val_ratio = val_size / (1 - test_size)

        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=val_ratio, random_state=random_state, stratify=y_temp
        )

        # --- STANDARD SCALING (NUMERIC ONLY) ---
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

        result.update({
            "X_train": X_train,
            "X_val": X_val,
            "X_test": X_test,
            "y_train": y_train,
            "y_val": y_val,
            "y_test": y_test,
            "scaler": scaler,
        })

        return result

    # =========================================================
    # ---- IMAGES (untouched; same as your original loader) ---
    # =========================================================
    elif dtype in {"image_folder", "image_file"}:

        if dtype == "image_file":
            img = load_image_file(path, img_size=img_size)
            X = np.array([img])
            y = np.array([0])
            classes = [Path(path).stem]
        else:
            imgs = load_images_from_folder(path, img_size=img_size)
            if not imgs:
                raise RuntimeError("No images found or no image library available.")
            X = imgs["images"]
            y = imgs["labels"]
            classes = imgs["classes"]

        if flatten_images_for_ml:
            X = X.reshape((len(X), -1))

        X_train, X_val, X_test, y_train, y_val, y_test = split_data(
            X, y, test_size=test_size, val_size=val_size, random_state=random_state
        )

        result.update({
            "X_train": X_train, "X_val": X_val, "X_test": X_test,
            "y_train": y_train, "y_val": y_val, "y_test": y_test,
            "classes": classes
        })
        return result

    else:
        raise ValueError(f"Unsupported dataset type: {dtype}")


In [34]:
# -------------------------------------------------------
# AUTO DETECT DATASET TYPE (CSV → TABULAR, FOLDER → IMAGES)
# -------------------------------------------------------
import os

def detect_dataset_type(path):
    """
    Detect whether dataset is TABULAR (CSV) or IMAGE folder.
    """
    if os.path.isfile(path):
        if path.endswith(".csv"):
            return "tabular"
        else:
            raise ValueError(f"Unsupported file format: {path}")

    if os.path.isdir(path):
        # folder → assume images
        exts = {".jpg", ".jpeg", ".png", ".bmp"}
        for root, dirs, files in os.walk(path):
            for f in files:
                if os.path.splitext(f)[1].lower() in exts:
                    return "image"
        raise ValueError("No image files found in folder.")

    raise ValueError("Dataset path is neither a file nor folder.")


# RUN for FNN

In [38]:
# ============================================
# unified_pruning_pipeline.py  (UPDATED)
# ============================================
import os
import json
import math
import time
from typing import Tuple, Dict, List, Optional

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Layer, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# ====================================================
# DATASET LOADING (unchanged)
# ====================================================
def _is_image_folder(path: str) -> bool:
    if not os.path.isdir(path):
        return False
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
    for root, _, files in os.walk(path):
        for f in files:
            if os.path.splitext(f)[1].lower() in exts:
                return True
    return False

def _detect_csv_delimiter(path: str) -> str:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    candidates = [",", ";", "\t", "|"]
    counts = {c: sample.count(c) for c in candidates}
    return max(counts, key=counts.get)

def load_tabular_csv(path: str, target_col: Optional[str]=None, test_size=0.2, val_size=0.1, random_state=42):
    sep = _detect_csv_delimiter(path)
    df = pd.read_csv(path, sep=sep, engine="python")
    df.columns = [c.strip() for c in df.columns]

    if target_col is None:
        for cand in ["y", "target", "label", "class"]:
            if cand in df.columns:
                target_col = cand
                break
    if target_col is None:
        raise ValueError("target_col not provided and could not be inferred.")

    y_series = df[target_col].astype(str).str.strip().str.lower()

    if set(y_series.unique()) <= {"yes","no","y","n","true","false","1","0"}:
        y_series = y_series.replace({"yes":"1","no":"0","y":"1","n":"0","true":"1","false":"0"})

    try:
        y = y_series.astype(int).values
    except:
        y = pd.factorize(y_series)[0]

    X_df = df.drop(columns=[target_col])
    cat_cols = X_df.select_dtypes(include=["object","category"]).columns.tolist()
    if len(cat_cols) > 0:
        X_df = pd.get_dummies(X_df, columns=cat_cols, drop_first=True)

    X_df = X_df.apply(pd.to_numeric, errors="coerce").fillna(0.0)
    feature_names = list(X_df.columns)
    X = X_df.values.astype(float)

    strat = y if len(np.unique(y)) > 1 else None
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size+val_size), random_state=random_state, stratify=strat
    )

    val_ratio = val_size/(test_size+val_size)
    strat_temp = y_temp if len(np.unique(y_temp))>1 else None
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=1-val_ratio, random_state=random_state, stratify=strat_temp
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return {
        "X_train": X_train, "y_train": y_train,
        "X_val": X_val, "y_val": y_val,
        "feature_names": feature_names,
        "scaler": scaler,
    }

def load_image_folder(path: str, image_size=(224,224), batch_size=64, val_split=0.1):
    ds_train = tf.keras.preprocessing.image_dataset_from_directory(
        path, image_size=image_size, batch_size=batch_size,
        label_mode="int", validation_split=val_split,
        subset="training", seed=123
    )
    ds_val = tf.keras.preprocessing.image_dataset_from_directory(
        path, image_size=image_size, batch_size=batch_size,
        label_mode="int", validation_split=val_split,
        subset="validation", seed=123
    )
    return ds_train, ds_val

def auto_load_dataset(path, target_col, image_size, batch_size):
    if os.path.isdir(path) and _is_image_folder(path):
        t, v = load_image_folder(path, image_size=image_size, batch_size=batch_size)
        return True, t, v, {"input_shape": (image_size[0], image_size[1], 3)}

    if os.path.isfile(path) and path.endswith(".csv"):
        tab = load_tabular_csv(path, target_col=target_col)
        meta = {
            "feature_names": tab["feature_names"],
            "input_shape": (len(tab["feature_names"]),)
        }
        return False, (tab["X_train"], tab["y_train"]), (tab["X_val"], tab["y_val"]), meta

    raise ValueError("Dataset must be CSV or image folder.")


# ====================================================
# PRUNEABLE LAYERS
# ====================================================
def get_pruneable_layers(model):
    out = []
    for l in model.layers:
        if isinstance(l, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
            out.append({"name": l.name, "layer": l})
    return out


# ====================================================
# WEIGHT IMPORTANCE
# ====================================================
def per_unit_weight_importance(layer):
    try:
        kernel = layer.get_weights()[0]
    except:
        return None
    w = np.array(kernel)
    if w.ndim == 2:
        return np.sum(np.abs(w), axis=0)
    elif w.ndim == 4:
        return np.sum(np.abs(w), axis=(0,1,2))
    return np.sum(np.abs(w.reshape(-1, w.shape[-1])), axis=0)


# ====================================================
# ACTIVATION IMPORTANCE
# ====================================================
def build_activation_model(model, pruneable):
    return Model(inputs=model.inputs, outputs=[p["layer"].output for p in pruneable])

def collect_activation_magnitudes(model, pruneable, X, batch_size, max_batches):
    act_model = build_activation_model(model, pruneable)
    n = len(X)
    acc = [None]*len(pruneable)
    count = [0]*len(pruneable)
    b = 0

    for i in range(0, n, batch_size):
        if b >= max_batches:
            break
        xb = tf.cast(X[i:i+batch_size], tf.float32)
        acts = act_model(xb, training=False)
        if not isinstance(acts, list):
            acts = [acts]

        for idx, a in enumerate(acts):
            a = a.numpy()
            if a.ndim == 4:
                val = np.mean(np.abs(a), axis=(0,1,2))
            elif a.ndim == 2:
                val = np.mean(np.abs(a), axis=0)
            else:
                flat = a.reshape(a.shape[0], -1)
                val = np.mean(np.abs(flat), axis=0)

            acc[idx] = val if acc[idx] is None else acc[idx] + val
            count[idx] += 1

        b += 1

    return [(acc[i]/count[i] if acc[i] is not None else None) for i in range(len(acc))]


# ====================================================
# normalize + combine
# ====================================================
def normalize_vector(x):
    x = x.astype(np.float32)
    x -= x.min()
    m = x.max()
    return x/m if m > 0 else np.ones_like(x)

def compute_combined_importance(w, a, alpha, beta):
    if w is None and a is None:
        return None
    if w is None: w = np.ones_like(a)
    if a is None: a = np.ones_like(w)
    return alpha*normalize_vector(w) + beta*normalize_vector(a)

def select_topk_mask(score, keep_ratio):
    n = score.shape[0]
    k = max(1, int(n*keep_ratio))
    th = np.partition(score, -k)[-k]
    return (score >= th).astype(np.float32)


# ====================================================
# APPLY MASKS
# ====================================================
def apply_masks_to_model_weights(model, masks):
    for layer in model.layers:
        if layer.name not in masks:
            continue
        mask = masks[layer.name]
        w = layer.get_weights()
        if not w:
            continue
        k = np.array(w[0])
        b = np.array(w[1]) if len(w)>1 else None

        if k.ndim == 2:
            k = k * mask.reshape(1,-1)
            if b is not None: b = b * mask
        elif k.ndim == 4:
            k = k * mask.reshape(1,1,1,-1)
            if b is not None: b = b * mask

        if b is None: layer.set_weights([k])
        else: layer.set_weights([k,b])


# ====================================================
# AUTO LOSS (IMPORTANT FIX)
# ====================================================
def choose_loss(model):
    outputs = model.output_shape

    if isinstance(outputs, list):   # not common
        outputs = outputs[0]

    if outputs[-1] == 1:
        print("Detected BINARY classifier → using binary_crossentropy")
        return "binary_crossentropy"

    print("Detected MULTI-CLASS classifier → using sparse_categorical_crossentropy")
    return "sparse_categorical_crossentropy"


# ====================================================
# MAIN PIPELINE
# ====================================================
def prune_pipeline(
    model_path,
    dataset_path,
    target_col=None,
    keep_ratio=0.7,
    alpha=0.5,
    beta=0.5,
    calib_batches=20,
    calib_batch_size=64,
    finetune_epochs=3,
    finetune_batch_size=64,
    save_dir="pruned_out",
    image_size=(224,224),
    batch_size=64
):

    os.makedirs(save_dir, exist_ok=True)

    base_model = load_model(model_path)
    base_model.summary()

    is_tf_ds, train_ds, val_ds, meta = auto_load_dataset(dataset_path, target_col, image_size, batch_size)

    if is_tf_ds:
        sample_batch = next(iter(val_ds.take(1)))[0]
    else:
        X_train, y_train = train_ds
        X_val, y_val = val_ds
        X_train = np.asarray(X_train)
        X_val = np.asarray(X_val)
        sample_batch = X_val[:batch_size]

    pruneable = get_pruneable_layers(base_model)

    weight_imps = [per_unit_weight_importance(p["layer"]) for p in pruneable]

    if is_tf_ds:
        Xcal = []
        c = 0
        for xb, _ in train_ds:
            Xcal.append(xb.numpy())
            c += 1
            if c >= calib_batches:
                break
        Xcal = np.concatenate(Xcal, axis=0)
        Xcal = Xcal.reshape((-1,)+meta["input_shape"])
    else:
        Xcal = X_train[:calib_batches*calib_batch_size]

    act_imps = collect_activation_magnitudes(
        base_model, pruneable,
        Xcal,
        batch_size=calib_batch_size,
        max_batches=calib_batches
    )

    masks = {}
    summary = {}
    for info, w_imp, a_imp in zip(pruneable, weight_imps, act_imps):
        score = compute_combined_importance(w_imp, a_imp, alpha, beta)
        if score is None:
            continue
        mask = select_topk_mask(score, keep_ratio)
        masks[info["name"]] = mask
        summary[info["name"]] = {
            "kept": int(mask.sum()),
            "total": int(len(mask))
        }

    json.dump(summary, open(os.path.join(save_dir,"importance.json"),"w"), indent=2)

    pruned = tf.keras.models.clone_model(base_model)
    pruned.set_weights(base_model.get_weights())
    apply_masks_to_model_weights(pruned, masks)

    loss_fn = choose_loss(pruned)
    pruned.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=loss_fn, metrics=["accuracy"])

    if is_tf_ds:
        before = pruned.evaluate(val_ds, verbose=0)
    else:
        before = pruned.evaluate(X_val, y_val, verbose=0)

    if finetune_epochs>0:
        if is_tf_ds:
            pruned.fit(train_ds, validation_data=val_ds, epochs=finetune_epochs, verbose=2)
            after = pruned.evaluate(val_ds, verbose=0)
        else:
            pruned.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                batch_size=finetune_batch_size,
                epochs=finetune_epochs,
                verbose=2
            )
            after = pruned.evaluate(X_val, y_val, verbose=0)
    else:
        after = None

    pruned.save(os.path.join(save_dir,"pruned_model.h5"))
    json.dump({k:v.tolist() for k,v in masks.items()}, open(os.path.join(save_dir,"masks.json"),"w"), indent=2)

    print("DONE. Saved to:", save_dir)
    return pruned, {
        "before": before,
        "after": after,
        "masks": masks,
        "summary": summary
    }


# ====================================================
# EXAMPLE
# ====================================================
pruned_model, artifacts = prune_pipeline(
    model_path=r"D:\college\sem-8\fnn_bank_marketing_model.h5",
    dataset_path=r"D:\college\sem-8\bank-full.csv",
    target_col="y",
    keep_ratio=0.7,
    alpha=0.5, beta=0.5,
    calib_batches=20,
    finetune_epochs=3,
    save_dir="pruned_out"
)



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,875 (62.02 KB)

 Trainable params: 15,873 (62.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

c:\Users\Ishan\Documents\sample_p1\env\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(64, 42))
  warnings.warn(msg)


Detected BINARY classifier → using binary_crossentropy
Epoch 1/3
495/495 - 4s - 8ms/step - accuracy: 0.9117 - loss: 0.1953 - val_accuracy: 0.9144 - val_loss: 0.1944
Epoch 2/3
495/495 - 2s - 4ms/step - accuracy: 0.9189 - loss: 0.1777 - val_accuracy: 0.9179 - val_loss: 0.1906
Epoch 3/3
495/495 - 2s - 4ms/step - accuracy: 0.9211 - loss: 0.1736 - val_accuracy: 0.9184 - val_loss: 0.1897


DONE. Saved to: pruned_out


In [46]:
import numpy as np
import time
import tensorflow as tf

# ---------------------------------------------------------
# FLOPS: For Dense (FNN)
# ---------------------------------------------------------
def dense_flops(input_units, output_units, use_bias=True):
    flops = input_units * output_units
    if use_bias:
        flops += output_units
    return flops


# ---------------------------------------------------------
# FLOPS: For Conv2D (CNN)
# ---------------------------------------------------------
def conv2d_flops(layer, input_shape):
    filters = layer.filters
    kernel_h, kernel_w = layer.kernel_size
    in_channels = input_shape[-1]
    stride_h, stride_w = layer.strides

    # Output feature map size
    if layer.padding == "same":
        out_h = input_shape[0] // stride_h
        out_w = input_shape[1] // stride_w
    else:  # valid
        out_h = (input_shape[0] - kernel_h + 1) // stride_h
        out_w = (input_shape[1] - kernel_w + 1) // stride_w

    flops = out_h * out_w * filters * (kernel_h * kernel_w * in_channels)
    if layer.use_bias:
        flops += out_h * out_w * filters
    return flops, (out_h, out_w, filters)



# ---------------------------------------------------------
# FLOPS ESTIMATOR for ANY MODEL (FNN + CNN)
# ---------------------------------------------------------
def calculate_model_flops(model, input_shape, verbose=True):
    total_flops = 0
    current_shape = input_shape

    if verbose:
        print("\nFLOPS Breakdown:")
        print("-" * 60)

    for layer in model.layers:
        layer_flops = 0

        # --------------------------
        # Conv2D (CNN)
        # --------------------------
        if isinstance(layer, tf.keras.layers.Conv2D):
            layer_flops, current_shape = conv2d_flops(layer, current_shape)
            total_flops += layer_flops

            if verbose:
                print(f"{layer.name:20s}  (Conv2D)   FLOPS = {layer_flops:,}")

        # --------------------------
        # Dense (FNN)
        # --------------------------
        elif isinstance(layer, tf.keras.layers.Dense):
            if len(current_shape) > 1:
                input_units = np.prod(current_shape)
            else:
                input_units = current_shape[0]

            output_units = layer.units
            layer_flops = dense_flops(input_units, output_units, layer.use_bias)
            total_flops += layer_flops
            current_shape = (output_units,)

            if verbose:
                print(f"{layer.name:20s}  (Dense)    FLOPS = {layer_flops:,}")

        # --------------------------
        # Flatten
        # --------------------------
        elif isinstance(layer, tf.keras.layers.Flatten):
            current_shape = (np.prod(current_shape),)

            if verbose:
                print(f"{layer.name:20s}  (Flatten)")

        # --------------------------
        # MaxPooling2D
        # --------------------------
        elif isinstance(layer, tf.keras.layers.MaxPooling2D):
            pool = layer.pool_size[0]
            h, w, c = current_shape
            current_shape = (h // pool, w // pool, c)

            if verbose:
                print(f"{layer.name:20s}  (MaxPool2D)")

        # --------------------------
        # Minimal FLOPs layers
        # --------------------------
        elif isinstance(layer, (tf.keras.layers.BatchNormalization, tf.keras.layers.Dropout)):
            if verbose:
                print(f"{layer.name:20s}  ({layer.__class__.__name__}) minimal FLOPS")

    if verbose:
        print("-" * 60)
        print(f"Total FLOPS = {total_flops:,}")
        print(f"Total GFLOPS = {total_flops/1e9:.6f}")

    return total_flops



# ---------------------------------------------------------
# INFERENCE TIME
# ---------------------------------------------------------
def measure_inference_time(model, input_shape, runs=50):
    dummy_input = np.random.rand(1, *input_shape).astype(np.float32)

    # Warm-up
    for _ in range(5):
        model.predict(dummy_input, verbose=0)

    start = time.time()
    for _ in range(runs):
        model.predict(dummy_input, verbose=0)
    end = time.time()

    avg_time = (end - start) / runs
    return avg_time


In [48]:
orig = tf.keras.models.load_model(r"D:\college\sem-8\fnn_bank_marketing_model.h5")
pruned = tf.keras.models.load_model(r"D:\college\sem-8\final\pruned_out\pruned_model.h5")

input_shape = (orig.input_shape[1],)

flops_orig = calculate_model_flops(orig, input_shape)
flops_pruned = calculate_model_flops(pruned, input_shape)

time_orig = measure_inference_time(orig, input_shape)
time_pruned = measure_inference_time(pruned, input_shape)

print("Orig FLOPS:", flops_orig)
print("Pruned FLOPS:", flops_pruned)
print("Orig Time:", time_orig)
print("Pruned Time:", time_pruned)



FLOPS Breakdown:
------------------------------------------------------------
dense                 (Dense)    FLOPS = 5,504
dense_1               (Dense)    FLOPS = 8,256
dense_2               (Dense)    FLOPS = 2,080
dense_3               (Dense)    FLOPS = 33
------------------------------------------------------------
Total FLOPS = 15,873
Total GFLOPS = 0.000016

FLOPS Breakdown:
------------------------------------------------------------
dense                 (Dense)    FLOPS = 5,504
dense_1               (Dense)    FLOPS = 8,256
dense_2               (Dense)    FLOPS = 2,080
dense_3               (Dense)    FLOPS = 33
------------------------------------------------------------
Total FLOPS = 15,873
Total GFLOPS = 0.000016
Orig FLOPS: 15873
Pruned FLOPS: 15873
Orig Time: 0.13581624031066894
Pruned Time: 0.13804460048675538


# **FNN as user input working**

In [8]:
"""
FNN-only pruning pipeline (auto-detect model type).
- Detects model type from final layer (binary / multiclass / regression).
- Adapts CSV loader accordingly.
- Keeps final layer intact (never pruned).
- Reports FLOPs, GFLOPs, MFLOPs and inference time.
"""

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# -----------------------
# Helper: detect model type
# -----------------------
def detect_model_type(model):
    out = model.layers[-1]
    units = getattr(out, "units", None)
    act = getattr(out, "activation", None)
    act_name = act.__name__ if act is not None else None

    if units == 1 and act_name in ("sigmoid",):
        return "binary"
    if units == 1 and act_name in ("linear", "relu", "tanh"):
        return "regression"
    if units is not None and units > 1 and act_name == "softmax":
        return "multiclass"
    # fallback tries
    if units == 1:
        return "binary"
    if units and units > 1:
        return "multiclass"
    return "unknown"

def get_loss_and_metrics_for_type(model_type):
    if model_type == "binary":
        return ("binary_crossentropy", ["accuracy"])
    if model_type == "multiclass":
        # use sparse labels to avoid forcing one-hot encoding
        return ("sparse_categorical_crossentropy", ["sparse_categorical_accuracy"])
    if model_type == "regression":
        return ("mse", ["mse"])
    raise ValueError("Unknown model type")

# =====================================================================
# 1) UNIVERSAL CSV LOADER  (adapts target based on model_type)
# =====================================================================
def load_any_csv_dataset(csv_path, target=None, test_size=0.2, val_size=0.1, model_type="binary"):
    """
    Loads CSV (auto-detect delimiter). Adapts target processing to model_type:
      - binary/regression: returns y as float32 shaped (n,1)
      - multiclass: returns integer labels shaped (n,1) for sparse loss
    """
    print("[INFO] Loading CSV with automatic delimiter detection...")
    df = pd.read_csv(csv_path, sep=None, engine="python")
    print("[INFO] Columns detected:", list(df.columns))

    if target is None:
        target = df.columns[-1]
        print(f"[INFO] Auto-detected target column = {target}")

    if target not in df.columns:
        raise ValueError(f"Target column '{target}' not found in dataset!")

    y = df[target]
    X = df.drop(columns=[target])

    # Preprocess target based on model_type
    if model_type == "binary":
        # convert yes/no/true/false -> 1/0
        if y.dtype == object:
            y = y.astype(str).str.strip().str.lower()
            y = y.replace({"yes": 1, "no": 0, "true": 1, "false": 0})
        # ensure numeric
        y = y.astype(np.float32).values.reshape(-1, 1)

    elif model_type == "multiclass":
        # convert strings to categorical integer codes if needed
        if y.dtype == object:
            y = y.astype("category").cat.codes
        # ensure integer labels (sparse labels)
        y = y.astype(np.int32).values.reshape(-1, 1)

    elif model_type == "regression":
        # numeric continuous
        y = y.astype(np.float32).values.reshape(-1, 1)

    else:
        raise ValueError("Unsupported model_type for loader")

    # One-hot encode only categorical predictors (keep numeric as-is)
    cat_cols = X.select_dtypes(include=["object"]).columns
    if len(cat_cols) > 0:
        print("[INFO] One-hot encoding predictors:", list(cat_cols))
        X = pd.get_dummies(X, drop_first=True)
    else:
        print("[INFO] No categorical predictor columns to encode.")

    # convert to float32 (features)
    X = X.astype(np.float32)

    # Split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size + val_size), random_state=42, stratify=y if model_type!="regression" else None
    )
    # second split into val/test
    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1 - val_ratio), random_state=42,
        stratify=y_temp if model_type!="regression" else None
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


# =====================================================================
# 2) GET DENSE LAYERS
# =====================================================================
def get_dense_layers(model):
    return [layer for layer in model.layers if isinstance(layer, Dense)]


# =====================================================================
# 3) ACTIVATION + GRADIENT STATS  (SHAPE-SAFE)
# =====================================================================
def compute_activation_grad_stats(model, dense_layers, X, y, model_type,
                                  batch_size=64, max_batches=30):
    """
    Returns stats dict: {layer.name: (A_mean_per_neuron, G_mean_per_neuron, Var_per_neuron)}
    Uses appropriate loss for model_type when computing gradients.
    """
    # Convert X,y to numpy if DataFrame
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
        y_np = y.to_numpy()
    else:
        y_np = y

    ds = tf.data.Dataset.from_tensor_slices((X_np, y_np)).batch(batch_size)
    acc_A = {l.name: [] for l in dense_layers}
    acc_G = {l.name: [] for l in dense_layers}
    acc_V = {l.name: [] for l in dense_layers}

    # choose loss for gradient computation
    if model_type == "binary":
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "multiclass":
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "regression":
        loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    else:
        raise ValueError("Unknown model_type for stats")

    batch_count = 0
    for xb, yb in ds:
        if batch_count >= max_batches:
            break
        batch_count += 1

        # ensure shapes consistent
        x = tf.cast(xb, tf.float32)
        # For sparse multiclass loss, yb should be shape (batch,) ints
        if model_type == "multiclass":
            yb_proc = tf.cast(tf.squeeze(yb, axis=-1), tf.int32)
        else:
            # binary/regression: keep shape (batch,1) as float32
            yb_proc = tf.cast(yb, tf.float32)

        layer_outputs = {}
        with tf.GradientTape(persistent=True) as tape:
            for layer in model.layers:
                x = layer(x)
                if layer in dense_layers:
                    tape.watch(x)
                    layer_outputs[layer.name] = x
            preds = x
            # compute per-sample loss (shape [batch,] or [batch, ...])
            per_sample_loss = loss_fn(yb_proc, preds)
            # compute scalar loss for gradient direction
            loss = tf.reduce_mean(per_sample_loss)

        for layer in dense_layers:
            name = layer.name
            if name not in layer_outputs:
                continue
            a = layer_outputs[name]            # shape (batch, units)
            A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
            V = tf.math.reduce_variance(a, axis=0).numpy()

            g = tape.gradient(loss, a)
            if g is None:
                G = np.zeros_like(A)
            else:
                G = tf.reduce_mean(tf.abs(g), axis=0).numpy()

            acc_A[name].append(A)
            acc_G[name].append(G)
            acc_V[name].append(V)

        del tape

    # Aggregate
    stats = {}
    for layer in dense_layers:
        name = layer.name
        if len(acc_A[name]) == 0:
            stats[name] = (np.array([]), np.array([]), np.array([]))
            continue
        A = np.mean(np.stack(acc_A[name], axis=0), axis=0)
        G = np.mean(np.stack(acc_G[name], axis=0), axis=0)
        V = np.mean(np.stack(acc_V[name], axis=0), axis=0)
        stats[name] = (A, G, V)

    return stats


# =====================================================================
# 4) IMPORTANCE MASKS (protect last Dense layer)
# =====================================================================
def compute_importance_mask(
        stats,
        keep_ratio=0.7,
        alpha=0.5,
        beta=0.3,
        gamma=0.2,
        last_dense_name=None,
        min_units=4  # <-- increase minimum neurons per layer
    ):
    
    masks = {}
    for name, (A, G, V) in stats.items():
        if A.size == 0:
            masks[name] = np.array([], dtype=np.float32)
            continue

        def normalize(x):
            x2 = x - np.min(x)
            mx = np.max(x2)
            return x2 / (mx + 1e-12)

        # compute score
        score = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)

        # ---- NEVER prune final Dense layer ----
        if last_dense_name is not None and name == last_dense_name:
            masks[name] = np.ones_like(score, dtype=np.float32)
            continue

        # ---- compute K ----
        k = int(len(score) * keep_ratio)
        k = max(k, min_units)        # enforce minimum
        k = min(k, len(score))       # safety

        # ---- If all scores are zero → randomly pick k neurons ----
        if np.all(score == 0):
            sel = np.random.choice(len(score), size=k, replace=False)
            mask = np.zeros_like(score)
            mask[sel] = 1
            masks[name] = mask.astype(np.float32)
            continue

        # normal path
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)

        # final safety: ensure >= min_units survive
        if np.sum(mask) < min_units:
            topk = np.argsort(score)[-min_units:]
            mask = np.zeros_like(score)
            mask[topk] = 1

        masks[name] = mask.astype(np.float32)

    return masks



# =====================================================================
# 5) DNAGate Layer (Zeroing)
# =====================================================================
class DNAGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        self._mask_np = np.array(mask, dtype=np.float32) if mask is not None else None
        self.mask = None

    def build(self, input_shape):
        if self._mask_np is None:
            self.mask = None
        else:
            if input_shape[-1] != self._mask_np.shape[0]:
                raise ValueError(f"DNAGate mask length {self._mask_np.shape[0]} != layer units {input_shape[-1]}")
            self.mask = tf.constant(self._mask_np, dtype=self.dtype)
        super().build(input_shape)

    def call(self, x):
        if self.mask is None:
            return x
        return x * self.mask


def build_masked_model(orig_model, masks):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = DNAGate(mask=masks[layer.name], name=layer.name + "_dna")(x)
    new_model = Model(inp, x)
    # copy weights where possible
    for l in orig_model.layers:
        try:
            new_model.get_layer(l.name).set_weights(l.get_weights())
        except Exception:
            pass
    return new_model


# =====================================================================
# 6) FLOPS + INFERENCE TIME
# =====================================================================
def dense_flops(in_size, out_size, bias=True):
    # multiply-adds counted as 2 ops per MAC
    return int(in_size * out_size * 2 + (out_size if bias else 0))

def model_flops(model):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            total += dense_flops(in_size, layer.units, layer.use_bias)
            in_size = layer.units
    return total

def effective_flops(model, masks):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            if layer.name in masks and masks[layer.name].size > 0:
                units = int(np.sum(masks[layer.name]))
            else:
                units = layer.units
            total += dense_flops(in_size, units, layer.use_bias)
            in_size = units
    return total

def flops_to_gflops(f):
    return f / 1e9

def flops_to_mflops(f):
    return f / 1e6

def measure_inference_time(model, X, runs=200):
    """Return average inference time in milliseconds per sample.
       Accepts X as pandas DataFrame or numpy array."""
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if len(X_np) == 0:
        return float("nan")
    # pick random indices
    idx = np.random.randint(0, len(X_np), size=min(runs, len(X_np)))
    samples = X_np[idx]
    # warm-up single predict
    model.predict(samples[:1], verbose=0)
    t0 = time.time()
    model.predict(samples, verbose=0)
    t1 = time.time()
    total_ms = (t1 - t0) * 1000.0
    return total_ms / len(samples)


# =====================================================================
# 7) STRUCTURAL PRUNING (REMOVE NEURONS) — protect final Dense layer
# =====================================================================
def structurally_prune_fnn(orig_model, masks, last_dense_name=None):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    prev_sel = None

    for layer in orig_model.layers:
        if isinstance(layer, Dense):
            W, b = layer.get_weights()

            # If previous layer was pruned → prune rows
            if prev_sel is not None:
                W = W[prev_sel, :]

            # ---- SAFETY: determine neurons to keep ----
            if last_dense_name is not None and layer.name == last_dense_name:
                # never prune final layer
                sel = np.arange(layer.units)

            else:
                mask = masks.get(layer.name, None)
                if mask is not None and mask.size > 0:
                    sel = np.where(mask == 1)[0]

                    # 💥 SAFETY FIX: ensure >=1 neuron survives
                    if len(sel) == 0:
                        sel = np.array([np.argmax(mask)])  
                else:
                    sel = None

            # ---- Apply pruning ----
            if sel is not None:
                W_new = W[:, sel]
                b_new = b[sel]
                new_units = len(sel)
            else:
                W_new = W
                b_new = b
                new_units = W.shape[1]

            new_layer = Dense(new_units, activation=layer.activation, name=layer.name)
            x = new_layer(x)
            new_layer.set_weights([W_new, b_new])

            prev_sel = sel

        else:
            x = layer(x)

    return Model(inp, x)


# =====================================================================
# 8) MAIN PIPELINE (auto-detect model type)
# =====================================================================
def fnn_pruning_pipeline(model_path, dataset_path,
                         keep_ratio=0.7,
                         alpha=0.5, beta=0.3, gamma=0.2,
                         calib_batches=30, batch_size=64, ft_epochs=3):
    # 1) load model and detect type
    print("Loading model:", model_path)
    model = load_model(model_path)
    model_type = detect_model_type(model)
    print(f"[INFO] Detected model type: {model_type}")

    loss_name, metrics = get_loss_and_metrics_for_type(model_type)
    print(f"[INFO] Using loss='{loss_name}', metrics={metrics}")

    # 2) load dataset adapted to model_type
    print("Loading dataset:", dataset_path)
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_any_csv_dataset(dataset_path, model_type=model_type)

    # 3) find dense layers and the last Dense layer name
    dense_layers = get_dense_layers(model)
    last_dense_name = dense_layers[-1].name if len(dense_layers) > 0 else None
    print("Dense layers:", [l.name for l in dense_layers], "| last_dense:", last_dense_name)

    # 4) compute stats (activation + gradient)
    print("Computing activation & gradient stats...")
    stats = compute_activation_grad_stats(model, dense_layers, X_train, y_train, model_type,
                                          batch_size=batch_size, max_batches=calib_batches)

    # 5) compute masks (protect final layer)
    masks = compute_importance_mask(stats, keep_ratio=keep_ratio, alpha=alpha, beta=beta, gamma=gamma, last_dense_name=last_dense_name)
    for k,v in masks.items():
        if v.size > 0:
            print(f"{k} -> kept units: {int(np.sum(v))}/{len(v)}")

    # 6) build masked model (zeroing) and compile with correct loss/metrics
    print("Building masked model (DNAGate zeroing)...")
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    # compute baseline & masked flops/accuracy
    baseline_acc = None
    masked_acc_before = None
    if model_type == "regression":
        baseline_eval = model.evaluate(X_val, y_val, verbose=0)
        baseline_acc = baseline_eval[0] if len(baseline_eval)>0 else None
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[0]
    else:
        baseline_acc = model.evaluate(X_val, y_val, verbose=0)[1]  # accuracy or sparse_accuracy
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[1]

    baseline_flops = model_flops(model)
    masked_flops = effective_flops(model, masks)

    print(f"Baseline Acc: {baseline_acc} | FLOPS: {baseline_flops}")
    print(f"Masked Acc (before FT): {masked_acc_before} | Effective FLOPS: {masked_flops}")

    # 7) fine-tune masked model
    print("Fine-tuning masked model...")
    masked_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # evaluate masked after FT
    if model_type == "regression":
        masked_eval = masked_model.evaluate(X_val, y_val, verbose=0)
        masked_acc_after = masked_eval[0] if len(masked_eval)>0 else None
    else:
        masked_acc_after = masked_model.evaluate(X_val, y_val, verbose=0)[1]

    # 8) structural pruning (permanent neuron removal)
    print("Constructing structurally-pruned model (remove neurons)...")
    pruned_model = structurally_prune_fnn(model, masks, last_dense_name=last_dense_name)
    pruned_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    print("Fine-tuning pruned model...")
    pruned_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # final evals
    if model_type == "regression":
        pruned_eval = pruned_model.evaluate(X_val, y_val, verbose=0)
        pruned_acc = pruned_eval[0] if len(pruned_eval)>0 else None
    else:
        pruned_acc = pruned_model.evaluate(X_val, y_val, verbose=0)[1]

    pruned_flops = model_flops(pruned_model)

    # compute GFLOPS / MFLOPS / inference times
    baseline_gflops = flops_to_gflops(baseline_flops)
    masked_gflops = flops_to_gflops(masked_flops)
    pruned_gflops = flops_to_gflops(pruned_flops)

    baseline_mflops = flops_to_mflops(baseline_flops)
    masked_mflops = flops_to_mflops(masked_flops)
    pruned_mflops = flops_to_mflops(pruned_flops)

    baseline_time_ms = measure_inference_time(model, X_val)
    masked_time_ms = measure_inference_time(masked_model, X_val)
    pruned_time_ms = measure_inference_time(pruned_model, X_val)

    print("\n=== FINAL REPORT ===")
    print(f"Model type: {model_type}")
    print(f"Baseline:   Acc/metric={baseline_acc} | FLOPS={baseline_flops} | MFLOPS={baseline_mflops:.6f} | GFLOPS={baseline_gflops:.10f} | Time={baseline_time_ms:.4f} ms/sample")
    print(f"Masked:     Acc/metric={masked_acc_after} | EFLOPS={masked_flops} | MFLOPS={masked_mflops:.6f} | GFLOPS={masked_gflops:.10f} | Time={masked_time_ms:.4f} ms/sample")
    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")
    if baseline_flops > 0:
        print(f"FLOPS reduction (pruned): {(baseline_flops - pruned_flops)/baseline_flops:.2%}")
    print("==============================================\n")

    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")

    # ---- NEW METRIC REDUCTIONS ADDED BELOW ----
    if baseline_flops > 0:
        flop_reduction = (baseline_flops - pruned_flops) / baseline_flops
        gflop_reduction = (baseline_gflops - pruned_gflops) / baseline_gflops if baseline_gflops > 0 else 0
    else:
        flop_reduction = 0
        gflop_reduction = 0

    acc_reduction = baseline_acc - pruned_acc if baseline_acc is not None else None

    print(f"FLOPS reduction (pruned): {flop_reduction:.2%}")
    print(f"GFLOPS reduction (pruned): {gflop_reduction:.2%}")
    print(f"Accuracy reduction: {acc_reduction}")
    # ---- END NEW LINES ----

    print("==============================================\n")


    # return X_val for further external checks if desired
    return model, masked_model, pruned_model, masks, X_val


# =====================================================================
# 9) RUN (example)
# =====================================================================
if __name__ == "__main__":
    # change these to your files
    model_path = r"D:\college\sem-8\models\dementia_fnn_model.h5"
    dataset_path = r"D:\college\sem-8\dataset\clean_dementia_dataset.csv"

    original_model, masked_model, pruned_model, masks, X_val = fnn_pruning_pipeline(
        model_path=model_path,
        dataset_path=dataset_path,
        keep_ratio=0.8,
        alpha=0.5, beta=0.3, gamma=0.2,
        calib_batches=30, batch_size=64, ft_epochs=15
    )


Loading model: D:\college\sem-8\models\dementia_fnn_model.h5
[INFO] Detected model type: binary
[INFO] Using loss='binary_crossentropy', metrics=['accuracy']
Loading dataset: D:\college\sem-8\dataset\clean_dementia_dataset.csv
[INFO] Loading CSV with automatic delimiter detection...
[INFO] Columns detected: ['Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF', 'M/F_M', 'Group']
[INFO] Auto-detected target column = Group
[INFO] No categorical predictor columns to encode.
Dense layers: ['dense_14', 'dense_15', 'dense_16', 'dense_17', 'dense_18', 'dense_19'] | last_dense: dense_19
Computing activation & gradient stats...
dense_14 -> kept units: 102/128
dense_15 -> kept units: 51/64
dense_16 -> kept units: 51/64
dense_17 -> kept units: 51/64
dense_18 -> kept units: 25/32
dense_19 -> kept units: 1/1
Building masked model (DNAGate zeroing)...
Baseline Acc: 0.59375 | FLOPS: 39585
Masked Acc (before FT): 0.59375 | Effective FLOPS: 25525
Fine-tuning masked model...
Epoch 1/15


4/4 - 2s - 552ms/step - accuracy: 0.5837 - loss: 6.7050 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 2/15
4/4 - 0s - 85ms/step - accuracy: 0.6244 - loss: 5.9836 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 3/15
4/4 - 0s - 28ms/step - accuracy: 0.5656 - loss: 6.8380 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 4/15
4/4 - 0s - 26ms/step - accuracy: 0.5973 - loss: 6.4779 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 5/15
4/4 - 0s - 23ms/step - accuracy: 0.5973 - loss: 6.4910 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 6/15
4/4 - 0s - 22ms/step - accuracy: 0.5928 - loss: 6.5631 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 7/15
4/4 - 0s - 28ms/step - accuracy: 0.5973 - loss: 6.4910 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 8/15
4/4 - 0s - 26ms/step - accuracy: 0.5973 - loss: 6.4910 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 9/15
4/4 - 0s - 27ms/step - accuracy: 0.5928 - loss: 6.5631 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 10/15
4/4 - 0s - 26ms/step - acc

# **FNN pruned model save**

In [2]:
"""
FNN-only pruning pipeline (auto-detect model type).
- Detects model type from final layer (binary / multiclass / regression).
- Adapts CSV loader accordingly.
- Keeps final layer intact (never pruned).
- Reports FLOPs, GFLOPs, MFLOPs and inference time.
"""

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# -----------------------
# Helper: detect model type
# -----------------------
def detect_model_type(model):
    out = model.layers[-1]
    units = getattr(out, "units", None)
    act = getattr(out, "activation", None)
    act_name = act.__name__ if act is not None else None

    if units == 1 and act_name == "sigmoid":
        return "binary"
    if units == 1 and act_name in ("linear", "relu", "tanh"):
        return "regression"
    if units is not None and units > 1 and act_name == "softmax":
        return "multiclass"

    # fallback
    if units == 1:
        return "binary"
    if units and units > 1:
        return "multiclass"

    return "unknown"


def get_loss_and_metrics_for_type(model_type):
    if model_type == "binary":
        return ("binary_crossentropy", ["accuracy"])
    if model_type == "multiclass":
        return ("sparse_categorical_crossentropy", ["sparse_categorical_accuracy"])
    if model_type == "regression":
        return ("mse", ["mse"])
    raise ValueError("Unknown model type")


# =====================================================================
# 1) UNIVERSAL CSV LOADER
# =====================================================================
def load_any_csv_dataset(csv_path, target=None, test_size=0.2, val_size=0.1, model_type="binary"):
    print("[INFO] Loading CSV with automatic delimiter detection...")
    df = pd.read_csv(csv_path, sep=None, engine="python")
    print("[INFO] Columns detected:", list(df.columns))

    if target is None:
        target = df.columns[-1]
        print(f"[INFO] Auto-detected target column = {target}")

    y = df[target]
    X = df.drop(columns=[target])

    # Process target
    if model_type == "binary":
        if y.dtype == object:
            y = y.astype(str).str.strip().str.lower().replace(
                {"yes": 1, "no": 0, "true": 1, "false": 0}
            )
        y = y.astype(np.float32).values.reshape(-1, 1)

    elif model_type == "multiclass":
        if y.dtype == object:
            y = y.astype("category").cat.codes
        y = y.astype(np.int32).values.reshape(-1, 1)

    elif model_type == "regression":
        y = y.astype(np.float32).values.reshape(-1, 1)

    cat_cols = X.select_dtypes(include=["object"]).columns
    if len(cat_cols) > 0:
        print("[INFO] One-hot encoding:", list(cat_cols))
        X = pd.get_dummies(X, drop_first=True)

    X = X.astype(np.float32)

    # Split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size + val_size), random_state=42,
        stratify=y if model_type != "regression" else None
    )
    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1 - val_ratio), random_state=42,
        stratify=y_temp if model_type != "regression" else None
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


# =====================================================================
# 2) GET DENSE LAYERS
# =====================================================================
def get_dense_layers(model):
    return [layer for layer in model.layers if isinstance(layer, Dense)]


# =====================================================================
# 3) ACTIVATION + GRADIENT STATS
# =====================================================================
def compute_activation_grad_stats(model, dense_layers, X, y, model_type,
                                  batch_size=64, max_batches=30):

    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()
    if isinstance(y, (pd.DataFrame, pd.Series)):
        y = y.to_numpy()

    ds = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

    acc_A = {l.name: [] for l in dense_layers}
    acc_G = {l.name: [] for l in dense_layers}
    acc_V = {l.name: [] for l in dense_layers}

    if model_type == "binary":
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction="none")
    elif model_type == "multiclass":
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction="none")
    else:
        loss_fn = tf.keras.losses.MeanSquaredError(reduction="none")

    batch_count = 0

    for xb, yb in ds:
        if batch_count >= max_batches:
            break
        batch_count += 1

        x = tf.cast(xb, tf.float32)

        if model_type == "multiclass":
            yb_proc = tf.cast(tf.squeeze(yb, axis=-1), tf.int32)
        else:
            yb_proc = tf.cast(yb, tf.float32)

        layer_outputs = {}

        with tf.GradientTape(persistent=True) as tape:
            for layer in model.layers:
                x = layer(x)
                if layer in dense_layers:
                    tape.watch(x)
                    layer_outputs[layer.name] = x

            preds = x
            per_sample_loss = loss_fn(yb_proc, preds)
            loss = tf.reduce_mean(per_sample_loss)

        for layer in dense_layers:
            name = layer.name
            a = layer_outputs[name]
            A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
            V = tf.math.reduce_variance(a, axis=0).numpy()

            g = tape.gradient(loss, a)
            G = np.zeros_like(A) if g is None else tf.reduce_mean(tf.abs(g), axis=0).numpy()

            acc_A[name].append(A)
            acc_G[name].append(G)
            acc_V[name].append(V)

        del tape

    stats = {}
    for layer in dense_layers:
        name = layer.name
        if len(acc_A[name]) == 0:
            stats[name] = (np.array([]), np.array([]), np.array([]))
        else:
            stats[name] = (
                np.mean(np.stack(acc_A[name]), axis=0),
                np.mean(np.stack(acc_G[name]), axis=0),
                np.mean(np.stack(acc_V[name]), axis=0)
            )

    return stats


# =====================================================================
# 4) IMPORTANCE MASK
# =====================================================================
def compute_importance_mask(stats, keep_ratio=0.7,
                            alpha=0.5, beta=0.3, gamma=0.2,
                            last_dense_name=None, min_units=4):

    masks = {}

    for name, (A, G, V) in stats.items():

        if A.size == 0:
            masks[name] = np.array([], dtype=np.float32)
            continue

        def normalize(x):
            x2 = x - np.min(x)
            mx = np.max(x2)
            return x2 / (mx + 1e-12)

        score = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)

        if last_dense_name is not None and name == last_dense_name:
            masks[name] = np.ones_like(score, dtype=np.float32)
            continue

        k = max(int(len(score) * keep_ratio), min_units)
        k = min(k, len(score))

        if np.all(score == 0):
            sel = np.random.choice(len(score), k, replace=False)
            mask = np.zeros_like(score)
            mask[sel] = 1
            masks[name] = mask
            continue

        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)

        if np.sum(mask) < min_units:
            topk = np.argsort(score)[-min_units:]
            mask = np.zeros_like(score)
            mask[topk] = 1

        masks[name] = mask

    return masks


# =====================================================================
# 5) DNAGate Layer
# =====================================================================
class DNAGate(tf.keras.layers.Layer):
    def __init__(self, gate, **kwargs):
        super().__init__(**kwargs)
        self.gate_init = np.array(gate, dtype=np.float32)

    def build(self, input_shape):
        self.gate = self.add_weight(
            name="gate",
            shape=self.gate_init.shape,
            initializer=tf.constant_initializer(self.gate_init),
            trainable=False
        )

    def call(self, inputs):
        return inputs * self.gate

    def get_config(self):
        config = super().get_config()
        config.update({"gate": self.gate.numpy().tolist()})
        return config

    @classmethod
    def from_config(cls, config):
        gate = config.pop("gate")
        return cls(gate=gate, **config)


def build_masked_model(orig_model, masks):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp

    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = DNAGate(gate=masks[layer.name], name=layer.name + "_dna")(x)

    new_model = Model(inp, x)

    for l in orig_model.layers:
        try:
            new_model.get_layer(l.name).set_weights(l.get_weights())
        except:
            pass

    return new_model


# =====================================================================
# 6) FLOPS + INFERENCE TIME
# =====================================================================
def dense_flops(in_size, out_size, bias=True):
    return int(in_size * out_size * 2 + (out_size if bias else 0))

def model_flops(model):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            total += dense_flops(in_size, layer.units, layer.use_bias)
            in_size = layer.units
    return total

def effective_flops(model, masks):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            units = np.sum(masks[layer.name]) if layer.name in masks else layer.units
            total += dense_flops(in_size, int(units), layer.use_bias)
            in_size = int(units)
    return total

def flops_to_gflops(f): return f / 1e9
def flops_to_mflops(f): return f / 1e6


def measure_inference_time(model, X, runs=200):

    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()

    if len(X) == 0:
        return float("nan")

    idx = np.random.randint(0, len(X), size=min(runs, len(X)))
    samples = X[idx]

    model.predict(samples[:1], verbose=0)
    t0 = time.time()
    model.predict(samples, verbose=0)
    t1 = time.time()

    return (t1 - t0) * 1000 / len(samples)


# =====================================================================
# 7) STRUCTURAL PRUNING
# =====================================================================
def structurally_prune_fnn(orig_model, masks, last_dense_name=None):

    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    prev_sel = None

    for layer in orig_model.layers:

        if isinstance(layer, Dense):
            W, b = layer.get_weights()

            if prev_sel is not None:
                W = W[prev_sel, :]

            if last_dense_name and layer.name == last_dense_name:
                sel = np.arange(layer.units)
            else:
                mask = masks.get(layer.name, None)
                if mask is None or mask.size == 0:
                    sel = None
                else:
                    sel = np.where(mask == 1)[0]
                    if len(sel) == 0:
                        sel = np.array([np.argmax(mask)])

            if sel is not None:
                W_new = W[:, sel]
                b_new = b[sel]
                units_new = len(sel)
            else:
                W_new, b_new = W, b
                units_new = W.shape[1]

            new_layer = Dense(units_new, activation=layer.activation, name=layer.name)
            x = new_layer(x)
            new_layer.set_weights([W_new, b_new])

            prev_sel = sel

        else:
            x = layer(x)

    return Model(inp, x)


# =====================================================================
# 8) MAIN PIPELINE
# =====================================================================
def fnn_pruning_pipeline(model_path, dataset_path,
                         keep_ratio=0.7,
                         alpha=0.5, beta=0.3, gamma=0.2,
                         calib_batches=30, batch_size=64, ft_epochs=3):

    print("Loading model:", model_path)
    model = load_model(model_path)

    model_type = detect_model_type(model)
    print(f"[INFO] Detected model type: {model_type}")

    loss_name, metrics = get_loss_and_metrics_for_type(model_type)

    print("Loading dataset:", dataset_path)
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_any_csv_dataset(
        dataset_path, model_type=model_type
    )

    dense_layers = get_dense_layers(model)
    last_dense_name = dense_layers[-1].name if dense_layers else None
    print("Dense layers:", [l.name for l in dense_layers])

    print("Computing activation & gradient stats...")
    stats = compute_activation_grad_stats(
        model, dense_layers, X_train, y_train, model_type,
        batch_size=batch_size, max_batches=calib_batches
    )

    masks = compute_importance_mask(
        stats, keep_ratio, alpha, beta, gamma, last_dense_name
    )

    for k, v in masks.items():
        if v.size > 0:
            print(f"{k} → kept {int(np.sum(v))}/{len(v)}")

    # ============================================================
    # MASKED MODEL
    # ============================================================
    print("Building masked model...")
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer="adam", loss=loss_name, metrics=metrics)

    print("Fine-tuning masked model...")
    masked_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                     epochs=ft_epochs, batch_size=batch_size, verbose=2)

    # ============================================================
    # STRUCTURAL PRUNING
    # ============================================================
    print("Building permanent pruned model...")
    pruned_model = structurally_prune_fnn(model, masks, last_dense_name)
    pruned_model.compile(optimizer="adam", loss=loss_name, metrics=metrics)

    print("Fine-tuning pruned model...")
    pruned_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                     epochs=ft_epochs, batch_size=batch_size, verbose=2)

    # ============================================================
    # SAVE MODELS
    # ============================================================
    import os

    base = os.path.basename(model_path)
    root = os.path.splitext(base)[0]
    folder = os.path.dirname(model_path)

    masked_path = os.path.join(folder, root + "_masked.h5")
    pruned_path = os.path.join(folder, root + "_pruned.h5")

    print(f"[INFO] Saving masked model to: {masked_path}")
    masked_model.save(masked_path)

    print(f"[INFO] Saving pruned model to: {pruned_path}")
    pruned_model.save(pruned_path)

    return model, masked_model, pruned_model, masks, X_val


# =====================================================================
# 9) RUN
# =====================================================================
if __name__ == "__main__":
    model_path = r"D:\college\sem-8\models\dementia_fnn_model.h5"
    dataset_path = r"D:\college\sem-8\dataset\clean_dementia_dataset.csv"

    original_model, masked_model, pruned_model, masks, X_val = fnn_pruning_pipeline(
        model_path=model_path,
        dataset_path=dataset_path,
        keep_ratio=0.7,
        alpha=0.5, beta=0.3, gamma=0.2,
        calib_batches=30, batch_size=64, ft_epochs=3
    )


Loading model: D:\college\sem-8\models\dementia_fnn_model.h5


[INFO] Detected model type: binary
Loading dataset: D:\college\sem-8\dataset\clean_dementia_dataset.csv
[INFO] Loading CSV with automatic delimiter detection...
[INFO] Columns detected: ['Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF', 'M/F_M', 'Group']
[INFO] Auto-detected target column = Group
Dense layers: ['dense_14', 'dense_15', 'dense_16', 'dense_17', 'dense_18', 'dense_19']
Computing activation & gradient stats...
dense_14 → kept 89/128
dense_15 → kept 44/64
dense_16 → kept 44/64
dense_17 → kept 44/64
dense_18 → kept 22/32
dense_19 → kept 1/1
Building masked model...



Fine-tuning masked model...
Epoch 1/3
4/4 - 2s - 438ms/step - accuracy: 0.5973 - loss: 6.4902 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 2/3
4/4 - 0s - 21ms/step - accuracy: 0.5928 - loss: 6.5623 - val_accuracy: 0.5938 - val_loss: 6.5480
Epoch 3/3
4/4 - 0s - 26ms/step - accuracy: 0.5928 - loss: 6.5631 - val_accuracy: 0.5938 - val_loss: 6.5480
Building permanent pruned model...
Fine-tuning pruned model...
Epoch 1/3
4/4 - 1s - 332ms/step - accuracy: 0.5928 - loss: 747.1626 - val_accuracy: 0.5938 - val_loss: 289.3863
Epoch 2/3
4/4 - 0s - 22ms/step - accuracy: 0.5566 - loss: 301.4309 - val_accuracy: 0.4062 - val_loss: 210.6923
Epoch 3/3
4/4 - 0s - 21ms/step - accuracy: 0.4434 - loss: 223.3150 - val_accuracy: 0.4062 - val_loss: 233.4635
[INFO] Saving masked model to: D:\college\sem-8\models\dementia_fnn_model_masked.h5


[INFO] Saving pruned model to: D:\college\sem-8\models\dementia_fnn_model_pruned.h5


# **Main FNN Prune Model Saved with Flops and Gflops**

In [12]:
"""
FNN-only pruning pipeline (auto-detect model type).
- Detects model type from final layer (binary / multiclass / regression).
- Adapts CSV loader accordingly.
- Keeps final layer intact (never pruned).
- Reports FLOPs, GFLOPs, MFLOPS and inference time.
"""

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
import os

# -----------------------
# Helper: detect model type
# -----------------------
def detect_model_type(model):
    out = model.layers[-1]
    units = getattr(out, "units", None)
    act = getattr(out, "activation", None)
    act_name = act.__name__ if act is not None else None

    if units == 1 and act_name in ("sigmoid",):
        return "binary"
    if units == 1 and act_name in ("linear", "relu", "tanh"):
        return "regression"
    if units is not None and units > 1 and act_name == "softmax":
        return "multiclass"
    # fallback tries
    if units == 1:
        return "binary"
    if units and units > 1:
        return "multiclass"
    return "unknown"

def get_loss_and_metrics_for_type(model_type):
    if model_type == "binary":
        return ("binary_crossentropy", ["accuracy"])
    if model_type == "multiclass":
        # use sparse labels to avoid forcing one-hot encoding
        return ("sparse_categorical_crossentropy", ["sparse_categorical_accuracy"])
    if model_type == "regression":
        return ("mse", ["mse"])
    raise ValueError("Unknown model type")

# =====================================================================
# 1) UNIVERSAL CSV LOADER  (adapts target based on model_type)
# =====================================================================
def load_any_csv_dataset(csv_path, target=None, test_size=0.2, val_size=0.1, model_type="binary"):
    """
    Loads CSV (auto-detect delimiter). Adapts target processing to model_type:
      - binary/regression: returns y as float32 shaped (n,1)
      - multiclass: returns integer labels shaped (n,1) for sparse loss
    """
    print("[INFO] Loading CSV with automatic delimiter detection...")
    df = pd.read_csv(csv_path, sep=None, engine="python")
    print("[INFO] Columns detected:", list(df.columns))

    if target is None:
        target = df.columns[-1]
        print(f"[INFO] Auto-detected target column = {target}")

    if target not in df.columns:
        raise ValueError(f"Target column '{target}' not found in dataset!")

    y = df[target]
    X = df.drop(columns=[target])

    # Preprocess target based on model_type
    if model_type == "binary":
        # convert yes/no/true/false -> 1/0
        if y.dtype == object:
            y = y.astype(str).str.strip().str.lower()
            y = y.replace({"yes": 1, "no": 0, "true": 1, "false": 0})
        # ensure numeric
        y = y.astype(np.float32).values.reshape(-1, 1)

    elif model_type == "multiclass":
        # convert strings to categorical integer codes if needed
        if y.dtype == object:
            y = y.astype("category").cat.codes
        # ensure integer labels (sparse labels)
        y = y.astype(np.int32).values.reshape(-1, 1)

    elif model_type == "regression":
        # numeric continuous
        y = y.astype(np.float32).values.reshape(-1, 1)

    else:
        raise ValueError("Unsupported model_type for loader")

    # One-hot encode only categorical predictors (keep numeric as-is)
    cat_cols = X.select_dtypes(include=["object"]).columns
    if len(cat_cols) > 0:
        print("[INFO] One-hot encoding predictors:", list(cat_cols))
        X = pd.get_dummies(X, drop_first=True)
    else:
        print("[INFO] No categorical predictor columns to encode.")

    # convert to float32 (features)
    X = X.astype(np.float32)

    # Split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size + val_size), random_state=42, stratify=y if model_type!="regression" else None
    )
    # second split into val/test
    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1 - val_ratio), random_state=42,
        stratify=y_temp if model_type!="regression" else None
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

# =====================================================================
# 2) GET DENSE LAYERS
# =====================================================================
def get_dense_layers(model):
    return [layer for layer in model.layers if isinstance(layer, Dense)]

# =====================================================================
# 3) ACTIVATION + GRADIENT STATS  (SHAPE-SAFE)
# =====================================================================
def compute_activation_grad_stats(model, dense_layers, X, y, model_type,
                                  batch_size=64, max_batches=30):
    """
    Returns stats dict: {layer.name: (A_mean_per_neuron, G_mean_per_neuron, Var_per_neuron)}
    Uses appropriate loss for model_type when computing gradients.
    """
    # Convert X,y to numpy if DataFrame
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
        y_np = y.to_numpy()
    else:
        y_np = y

    ds = tf.data.Dataset.from_tensor_slices((X_np, y_np)).batch(batch_size)
    acc_A = {l.name: [] for l in dense_layers}
    acc_G = {l.name: [] for l in dense_layers}
    acc_V = {l.name: [] for l in dense_layers}

    # choose loss for gradient computation
    if model_type == "binary":
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "multiclass":
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "regression":
        loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    else:
        raise ValueError("Unknown model_type for stats")

    batch_count = 0
    for xb, yb in ds:
        if batch_count >= max_batches:
            break
        batch_count += 1

        # ensure shapes consistent
        x = tf.cast(xb, tf.float32)
        # For sparse multiclass loss, yb should be shape (batch,) ints
        if model_type == "multiclass":
            yb_proc = tf.cast(tf.squeeze(yb, axis=-1), tf.int32)
        else:
            # binary/regression: keep shape (batch,1) as float32
            yb_proc = tf.cast(yb, tf.float32)

        layer_outputs = {}
        with tf.GradientTape(persistent=True) as tape:
            for layer in model.layers:
                x = layer(x)
                if layer in dense_layers:
                    tape.watch(x)
                    layer_outputs[layer.name] = x
            preds = x
            # compute per-sample loss (shape [batch,] or [batch, ...])
            per_sample_loss = loss_fn(yb_proc, preds)
            # compute scalar loss for gradient direction
            loss = tf.reduce_mean(per_sample_loss)

        for layer in dense_layers:
            name = layer.name
            if name not in layer_outputs:
                continue
            a = layer_outputs[name]            # shape (batch, units)
            A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
            V = tf.math.reduce_variance(a, axis=0).numpy()

            g = tape.gradient(loss, a)
            if g is None:
                G = np.zeros_like(A)
            else:
                G = tf.reduce_mean(tf.abs(g), axis=0).numpy()

            acc_A[name].append(A)
            acc_G[name].append(G)
            acc_V[name].append(V)

        del tape

    # Aggregate
    stats = {}
    for layer in dense_layers:
        name = layer.name
        if len(acc_A[name]) == 0:
            stats[name] = (np.array([]), np.array([]), np.array([]))
            continue
        A = np.mean(np.stack(acc_A[name], axis=0), axis=0)
        G = np.mean(np.stack(acc_G[name], axis=0), axis=0)
        V = np.mean(np.stack(acc_V[name], axis=0), axis=0)
        stats[name] = (A, G, V)

    return stats

# =====================================================================
# 4) IMPORTANCE MASKS (protect last Dense layer)
# =====================================================================
def compute_importance_mask(
        stats,
        keep_ratio=0.7,
        alpha=0.5,
        beta=0.3,
        gamma=0.2,
        last_dense_name=None,
        min_units=4  # <-- increase minimum neurons per layer
    ):
    
    masks = {}
    for name, (A, G, V) in stats.items():
        if A.size == 0:
            masks[name] = np.array([], dtype=np.float32)
            continue

        def normalize(x):
            x2 = x - np.min(x)
            mx = np.max(x2)
            return x2 / (mx + 1e-12)

        # compute score
        score = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)

        # ---- NEVER prune final Dense layer ----
        if last_dense_name is not None and name == last_dense_name:
            masks[name] = np.ones_like(score, dtype=np.float32)
            continue

        # ---- compute K ----
        k = int(len(score) * keep_ratio)
        k = max(k, min_units)        # enforce minimum
        k = min(k, len(score))       # safety

        # ---- If all scores are zero → randomly pick k neurons ----
        if np.all(score == 0):
            sel = np.random.choice(len(score), size=k, replace=False)
            mask = np.zeros_like(score)
            mask[sel] = 1
            masks[name] = mask.astype(np.float32)
            continue

        # normal path
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)

        # final safety: ensure >= min_units survive
        if np.sum(mask) < min_units:
            topk = np.argsort(score)[-min_units:]
            mask = np.zeros_like(score)
            mask[topk] = 1

        masks[name] = mask.astype(np.float32)

    return masks

# =====================================================================
# 5) DNAGate Layer (Zeroing)
# =====================================================================
# =====================================================================
# 5) DNAGate Layer (Zeroing) — NOW FULLY SERIALIZABLE
# =====================================================================
class DNAGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        # store original numpy mask (for config saving)
        self._mask_np = None
        if mask is not None:
            self._mask_np = np.array(mask, dtype=np.float32)

        # keras-serializable field (converted to list)
        self.mask_list = self._mask_np.tolist() if self._mask_np is not None else None
        self.mask = None

    def build(self, input_shape):
        if self.mask_list is None:
            self.mask = None
        else:
            # rebuild tensor mask
            mask_np = np.array(self.mask_list, dtype=np.float32)
            if input_shape[-1] != mask_np.shape[0]:
                raise ValueError(
                    f"DNAGate mask length {mask_np.shape[0]} != layer units {input_shape[-1]}"
                )
            self.mask = tf.constant(mask_np, dtype=self.dtype)
        super().build(input_shape)

    def call(self, x):
        if self.mask is None:
            return x
        return x * self.mask

    # ⭐⭐ IMPORTANT — MAKES THE LAYER SAVEABLE ⭐⭐
    def get_config(self):
        config = super().get_config()
        config.update({
            "mask": self.mask_list,   # store mask as list
        })
        return config

    @classmethod
    def from_config(cls, config):
        # mask list gets restored automatically
        return cls(**config)

def build_masked_model(orig_model, masks):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = DNAGate(mask=masks[layer.name], name=layer.name + "_dna")(x)
    new_model = Model(inp, x)
    # copy weights where possible
    for l in orig_model.layers:
        try:
            new_model.get_layer(l.name).set_weights(l.get_weights())
        except Exception:
            pass
    return new_model

# =====================================================================
# 6) FLOPS + INFERENCE TIME
# =====================================================================
def dense_flops(in_size, out_size, bias=True):
    # multiply-adds counted as 2 ops per MAC
    return int(in_size * out_size * 2 + (out_size if bias else 0))

def model_flops(model):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            total += dense_flops(in_size, layer.units, layer.use_bias)
            in_size = layer.units
    return total

def effective_flops(model, masks):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            if layer.name in masks and masks[layer.name].size > 0:
                units = int(np.sum(masks[layer.name]))
            else:
                units = layer.units
            total += dense_flops(in_size, units, layer.use_bias)
            in_size = units
    return total

def flops_to_gflops(f):
    return f / 1e9

def flops_to_mflops(f):
    return f / 1e6

def measure_inference_time(model, X, runs=200):
    """Return average inference time in milliseconds per sample.
       Accepts X as pandas DataFrame or numpy array."""
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if len(X_np) == 0:
        return float("nan")
    # pick random indices
    idx = np.random.randint(0, len(X_np), size=min(runs, len(X_np)))
    samples = X_np[idx]
    # warm-up single predict
    model.predict(samples[:1], verbose=0)
    t0 = time.time()
    model.predict(samples, verbose=0)
    t1 = time.time()
    total_ms = (t1 - t0) * 1000.0
    return total_ms / len(samples)

# =====================================================================
# 7) STRUCTURAL PRUNING (REMOVE NEURONS) — protect final Dense layer
# =====================================================================
def structurally_prune_fnn(orig_model, masks, last_dense_name=None):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    prev_sel = None

    for layer in orig_model.layers:
        if isinstance(layer, Dense):
            W, b = layer.get_weights()

            # If previous layer was pruned → prune rows
            if prev_sel is not None:
                W = W[prev_sel, :]

            # ---- SAFETY: determine neurons to keep ----
            if last_dense_name is not None and layer.name == last_dense_name:
                # never prune final layer
                sel = np.arange(layer.units)

            else:
                mask = masks.get(layer.name, None)
                if mask is not None and mask.size > 0:
                    sel = np.where(mask == 1)[0]

                    # 💥 SAFETY FIX: ensure >=1 neuron survives
                    if len(sel) == 0:
                        sel = np.array([np.argmax(mask)])  
                else:
                    sel = None

            # ---- Apply pruning ----
            if sel is not None:
                W_new = W[:, sel]
                b_new = b[sel]
                new_units = len(sel)
            else:
                W_new = W
                b_new = b
                new_units = W.shape[1]

            new_layer = Dense(new_units, activation=layer.activation, name=layer.name)
            x = new_layer(x)
            new_layer.set_weights([W_new, b_new])

            prev_sel = sel

        else:
            x = layer(x)

    return Model(inp, x)

# =====================================================================
# 8) MAIN PIPELINE (auto-detect model type)
# =====================================================================
def fnn_pruning_pipeline(model_path, dataset_path,
                         keep_ratio=0.7,
                         alpha=0.5, beta=0.3, gamma=0.2,
                         calib_batches=30, batch_size=64, ft_epochs=3):
    # 1) load model and detect type
    print("Loading model:", model_path)
    model = load_model(model_path)
    model_type = detect_model_type(model)
    print(f"[INFO] Detected model type: {model_type}")

    loss_name, metrics = get_loss_and_metrics_for_type(model_type)
    print(f"[INFO] Using loss='{loss_name}', metrics={metrics}")

    # 2) load dataset adapted to model_type
    print("Loading dataset:", dataset_path)
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_any_csv_dataset(dataset_path, model_type=model_type)

    # 3) find dense layers and the last Dense layer name
    dense_layers = get_dense_layers(model)
    last_dense_name = dense_layers[-1].name if len(dense_layers) > 0 else None
    print("Dense layers:", [l.name for l in dense_layers], "| last_dense:", last_dense_name)

    # 4) compute stats (activation + gradient)
    print("Computing activation & gradient stats...")
    stats = compute_activation_grad_stats(model, dense_layers, X_train, y_train, model_type,
                                          batch_size=batch_size, max_batches=calib_batches)

    # 5) compute masks (protect final layer)
    masks = compute_importance_mask(stats, keep_ratio=keep_ratio, alpha=alpha, beta=beta, gamma=gamma, last_dense_name=last_dense_name)
    for k,v in masks.items():
        if v.size > 0:
            print(f"{k} -> kept units: {int(np.sum(v))}/{len(v)}")

    # 6) build masked model (DNAGate zeroing) and compile with correct loss/metrics
    print("Building masked model (DNAGate zeroing)...")
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    # compute baseline & masked flops/accuracy
    baseline_acc = None
    masked_acc_before = None
    if model_type == "regression":
        baseline_eval = model.evaluate(X_val, y_val, verbose=0)
        baseline_acc = baseline_eval[0] if len(baseline_eval)>0 else None
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[0]
    else:
        baseline_acc = model.evaluate(X_val, y_val, verbose=0)[1]  # accuracy or sparse_accuracy
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[1]

    baseline_flops = model_flops(model)
    masked_flops = effective_flops(model, masks)

    print(f"Baseline Acc: {baseline_acc} | FLOPS: {baseline_flops}")
    print(f"Masked Acc (before FT): {masked_acc_before} | Effective FLOPS: {masked_flops}")

    # 7) fine-tune masked model
    print("Fine-tuning masked model...")
    masked_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # evaluate masked after FT
    if model_type == "regression":
        masked_eval = masked_model.evaluate(X_val, y_val, verbose=0)
        masked_acc_after = masked_eval[0] if len(masked_eval)>0 else None
    else:
        masked_acc_after = masked_model.evaluate(X_val, y_val, verbose=0)[1]

    # 8) structural pruning (permanent neuron removal)
    print("Constructing structurally-pruned model (remove neurons)...")
    pruned_model = structurally_prune_fnn(model, masks, last_dense_name=last_dense_name)
    pruned_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    print("Fine-tuning pruned model...")
    pruned_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # final evals
    if model_type == "regression":
        pruned_eval = pruned_model.evaluate(X_val, y_val, verbose=0)
        pruned_acc = pruned_eval[0] if len(pruned_eval)>0 else None
    else:
        pruned_acc = pruned_model.evaluate(X_val, y_val, verbose=0)[1]

    pruned_flops = model_flops(pruned_model)

    # compute GFLOPS / MFLOPS / inference times
    baseline_gflops = flops_to_gflops(baseline_flops)
    masked_gflops = flops_to_gflops(masked_flops)
    pruned_gflops = flops_to_gflops(pruned_flops)

    baseline_mflops = flops_to_mflops(baseline_flops)
    masked_mflops = flops_to_mflops(masked_flops)
    pruned_mflops = flops_to_mflops(pruned_flops)

    baseline_time_ms = measure_inference_time(model, X_val)
    masked_time_ms = measure_inference_time(masked_model, X_val)
    pruned_time_ms = measure_inference_time(pruned_model, X_val)

    print("\n=== FINAL REPORT ===")
    print(f"Model type: {model_type}")
    print(f"Baseline:   Acc/metric={baseline_acc} | FLOPS={baseline_flops} | MFLOPS={baseline_mflops:.6f} | GFLOPS={baseline_gflops:.10f} | Time={baseline_time_ms:.4f} ms/sample")
    print(f"Masked:     Acc/metric={masked_acc_after} | EFLOPS={masked_flops} | MFLOPS={masked_mflops:.6f} | GFLOPS={masked_gflops:.10f} | Time={masked_time_ms:.4f} ms/sample")
    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")
    if baseline_flops > 0:
        print(f"FLOPS reduction (pruned): {(baseline_flops - pruned_flops)/baseline_flops:.2%}")
    print("==============================================\n")

    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")

    # ---- NEW METRIC REDUCTIONS ADDED BELOW ----
    if baseline_flops > 0:
        flop_reduction = (baseline_flops - pruned_flops) / baseline_flops
        gflop_reduction = (baseline_gflops - pruned_gflops) / baseline_gflops if baseline_gflops > 0 else 0
    else:
        flop_reduction = 0
        gflop_reduction = 0

    acc_reduction = baseline_acc - pruned_acc if baseline_acc is not None else None

    print(f"FLOPS reduction (pruned): {flop_reduction:.2%}")
    print(f"GFLOPS reduction (pruned): {gflop_reduction:.2%}")
    print(f"Accuracy reduction: {acc_reduction}")
    # ---- END NEW LINES ----

    print("==============================================\n")

    # save masked/pruned models next to original
    base = os.path.basename(model_path)
    root = os.path.splitext(base)[0]
    folder = os.path.dirname(model_path) or "."

    masked_path = os.path.join(folder, root + "_masked.h5")
    pruned_path = os.path.join(folder, root + "_pruned.h5")

    try:
        print(f"[INFO] Saving masked model to: {masked_path}")
        masked_model.save(masked_path)
    except Exception as e:
        print(f"[WARN] Failed to save masked model: {e}")

    try:
        print(f"[INFO] Saving pruned model to: {pruned_path}")
        pruned_model.save(pruned_path)
    except Exception as e:
        print(f"[WARN] Failed to save pruned model: {e}")

    # return X_val for further external checks if desired
    return model, masked_model, pruned_model, masks, X_val

# =====================================================================
# 9) RUN (example)
# =====================================================================
if __name__ == "__main__":
    # change these to your files
    model_path = r"D:\college\sem-8\models\dementia_fnn_model.h5"
    dataset_path = r"D:\college\sem-8\dataset\clean_dementia_dataset.csv"

    original_model, masked_model, pruned_model, masks, X_val = fnn_pruning_pipeline(
        model_path=model_path,
        dataset_path=dataset_path,
        keep_ratio=0.8,
        alpha=0.5, beta=0.3, gamma=0.2,
        calib_batches=30, batch_size=64, ft_epochs=15
    )


Loading model: D:\college\sem-8\models\dementia_fnn_model.h5
[INFO] Detected model type: binary
[INFO] Using loss='binary_crossentropy', metrics=['accuracy']
Loading dataset: D:\college\sem-8\dataset\clean_dementia_dataset.csv
[INFO] Loading CSV with automatic delimiter detection...
[INFO] Columns detected: ['Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF', 'M/F_M', 'Group']
[INFO] Auto-detected target column = Group
[INFO] No categorical predictor columns to encode.
Dense layers: ['dense_14', 'dense_15', 'dense_16', 'dense_17', 'dense_18', 'dense_19'] | last_dense: dense_19
Computing activation & gradient stats...
dense_14 -> kept units: 102/128
dense_15 -> kept units: 51/64
dense_16 -> kept units: 51/64
dense_17 -> kept units: 51/64
dense_18 -> kept units: 25/32
dense_19 -> kept units: 1/1
Building masked model (DNAGate zeroing)...
Baseline Acc: 0.59375 | FLOPS: 39585
Masked Acc (before FT): 0.59375 | Effective FLOPS: 25525
Fine-tuning masked model...
Epoch 1/15
4/4 - 2s - 


=== FINAL REPORT ===
Model type: binary
Baseline:   Acc/metric=0.59375 | FLOPS=39585 | MFLOPS=0.039585 | GFLOPS=0.0000395850 | Time=3.1087 ms/sample
Masked:     Acc/metric=0.59375 | EFLOPS=25525 | MFLOPS=0.025525 | GFLOPS=0.0000255250 | Time=3.4252 ms/sample
Pruned:     Acc/metric=0.59375 | FLOPS=25525 | MFLOPS=0.025525 | GFLOPS=0.0000255250 | Time=3.1285 ms/sample
FLOPS reduction (pruned): 35.52%

Pruned:     Acc/metric=0.59375 | FLOPS=25525 | MFLOPS=0.025525 | GFLOPS=0.0000255250 | Time=3.1285 ms/sample
FLOPS reduction (pruned): 35.52%
GFLOPS reduction (pruned): 35.52%
Accuracy reduction: 0.0

[INFO] Saving masked model to: D:\college\sem-8\models\dementia_fnn_model_masked.h5
[INFO] Saving pruned model to: D:\college\sem-8\models\dementia_fnn_model_pruned.h5


# **FNN for alzihmerA**

In [32]:
"""
FNN-only pruning pipeline (auto-detect model type).
- Detects model type from final layer (binary / multiclass / regression).
- Adapts CSV loader accordingly.
- Keeps final layer intact (never pruned).
- Reports FLOPs, GFLOPs, MFLOPS and inference time.
- Reports absolute + percent reductions for FLOPS, GFLOPS and Accuracy.
"""
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# -----------------------
# Helper: detect model type
# -----------------------
def detect_model_type(model):
    out = model.layers[-1]
    units = getattr(out, "units", None)
    act = getattr(out, "activation", None)
    act_name = act.__name__ if act is not None else None

    if units == 1 and act_name in ("sigmoid",):
        return "binary"
    if units == 1 and act_name in ("linear", "relu", "tanh"):
        return "regression"
    if units is not None and units > 1 and act_name == "softmax":
        return "multiclass"
    # fallback tries
    if units == 1:
        return "binary"
    if units and units > 1:
        return "multiclass"
    return "unknown"

def get_loss_and_metrics_for_type(model_type):
    if model_type == "binary":
        return ("binary_crossentropy", ["accuracy"])
    if model_type == "multiclass":
        # use sparse labels to avoid forcing one-hot encoding
        return ("sparse_categorical_crossentropy", ["sparse_categorical_accuracy"])
    if model_type == "regression":
        return ("mse", ["mse"])
    raise ValueError("Unknown model type")

# =====================================================================
# 1) UNIVERSAL CSV LOADER  (robust, factorizes non-numeric targets)
# =====================================================================
def load_any_csv_dataset(csv_path, target=None, test_size=0.2, val_size=0.1, model_type="binary"):
    """
    Loads CSV (auto-detect delimiter). Adapts target processing to model_type:
      - binary/regression: returns y as float32 shaped (n,1)
      - multiclass: returns integer labels shaped (n,1) for sparse loss
    Also handles non-numeric targets by factorization, handles categorical predictors.
    """
    print("[INFO] Loading CSV with automatic delimiter detection...")
    df = pd.read_csv(csv_path, sep=None, engine="python")
    print("[INFO] Columns detected:", list(df.columns))

    if target is None:
        target = df.columns[-1]
        print(f"[INFO] Auto-detected target column = {target}")

    if target not in df.columns:
        raise ValueError(f"Target column '{target}' not found in dataset!")

    y_raw = df[target]
    X = df.drop(columns=[target])

    # ---- Robust target processing ----
    # Try numeric conversion first (works for regression or numeric-coded labels)
    y_num = pd.to_numeric(y_raw, errors="coerce")
    if not y_num.isna().any():
        y_proc = y_num
        print("[INFO] Target converted to numeric successfully.")
    else:
        # contains non-numeric values -> fallback to categorical factorization
        print("[WARN] Target column contains non-numeric values; factorizing to integer codes.")
        codes, uniques = pd.factorize(y_raw.astype(str))
        y_proc = pd.Series(codes)
        print(f"[INFO] Factorized target into {len(uniques)} unique codes.")

    # Now adapt shape & dtype by model_type
    if model_type == "binary":
        unique_vals = np.unique(y_proc)
        if len(unique_vals) > 2:
            print(f"[WARN] Detected {len(unique_vals)} unique target values for model_type='binary'. Using numeric/factorized values as-is.")
        y = y_proc.astype(np.float32).values.reshape(-1, 1)

    elif model_type == "multiclass":
        # ensure integer labels (sparse labels)
        if np.issubdtype(y_proc.dtype, np.floating):
            if np.all(np.mod(y_proc.fillna(0).values, 1) == 0):
                y = y_proc.astype(np.int32).values.reshape(-1, 1)
            else:
                codes, uniques = pd.factorize(y_raw.astype(str))
                y = codes.astype(np.int32).reshape(-1, 1)
                print("[WARN] Multiclass target had non-integer numeric values; factorized instead.")
        else:
            y = y_proc.astype(np.int32).values.reshape(-1, 1)

    elif model_type == "regression":
        if np.any(pd.isna(y_proc)):
            raise ValueError("Regression target contains non-numeric values that could not be converted.")
        y = y_proc.astype(np.float32).values.reshape(-1, 1)
    else:
        raise ValueError("Unsupported model_type for loader")
    # ---- end target processing ----

    # One-hot encode only categorical predictors (keep numeric as-is)
    cat_cols = X.select_dtypes(include=["object"]).columns
    if len(cat_cols) > 0:
        print("[INFO] One-hot encoding predictors:", list(cat_cols))
        X = pd.get_dummies(X, drop_first=True)
    else:
        print("[INFO] No categorical predictor columns to encode.")

    # convert to float32 (features)
    X = X.astype(np.float32)

    # ----- SAFER stratify handling -----
    use_stratify = None
    if model_type != "regression":
        try:
            y_flat = np.ravel(y).astype(np.int64)
            class_counts = np.bincount(y_flat)
            if np.all(class_counts >= 2):
                use_stratify = y_flat
            else:
                print("[WARN] Some classes have < 2 samples, disabling stratify for train/val/test split.")
                use_stratify = None
        except Exception:
            print("[WARN] Could not prepare integer labels for stratify; disabling stratify.")
            use_stratify = None

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y,
        test_size=(test_size + val_size),
        random_state=42,
        stratify=use_stratify
    )

    # second split into val/test
    use_stratify2 = None
    if model_type != "regression" and use_stratify is not None:
        try:
            y_temp_flat = np.ravel(y_temp).astype(np.int64)
            class_counts2 = np.bincount(y_temp_flat)
            if np.all(class_counts2 >= 2):
                use_stratify2 = y_temp_flat
            else:
                print("[WARN] Validation/Test split: classes <2 detected, disabling stratify.")
                use_stratify2 = None
        except Exception:
            use_stratify2 = None

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp,
        test_size=test_size/(test_size+val_size),
        random_state=42,
        stratify=use_stratify2
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

# =====================================================================
# 2) GET DENSE LAYERS
# =====================================================================
def get_dense_layers(model):
    return [layer for layer in model.layers if isinstance(layer, Dense)]

# =====================================================================
# 3) ACTIVATION + GRADIENT STATS  (SHAPE-SAFE)
# =====================================================================
def compute_activation_grad_stats(model, dense_layers, X, y, model_type,
                                  batch_size=64, max_batches=30):
    """
    Returns stats dict: {layer.name: (A_mean_per_neuron, G_mean_per_neuron, Var_per_neuron)}
    Uses appropriate loss for model_type when computing gradients.
    """
    # Convert X,y to numpy if DataFrame
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
        y_np = y.to_numpy()
    else:
        y_np = y

    ds = tf.data.Dataset.from_tensor_slices((X_np, y_np)).batch(batch_size)
    acc_A = {l.name: [] for l in dense_layers}
    acc_G = {l.name: [] for l in dense_layers}
    acc_V = {l.name: [] for l in dense_layers}

    # choose loss for gradient computation
    if model_type == "binary":
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "multiclass":
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "regression":
        loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    else:
        raise ValueError("Unknown model_type for stats")

    batch_count = 0
    for xb, yb in ds:
        if batch_count >= max_batches:
            break
        batch_count += 1

        # ensure shapes consistent
        x = tf.cast(xb, tf.float32)
        # For sparse multiclass loss, yb should be shape (batch,) ints
        if model_type == "multiclass":
            yb_proc = tf.cast(tf.squeeze(yb, axis=-1), tf.int32)
        else:
            # binary/regression: keep shape (batch,1) as float32
            yb_proc = tf.cast(yb, tf.float32)

        layer_outputs = {}
        with tf.GradientTape(persistent=True) as tape:
            for layer in model.layers:
                x = layer(x)
                if layer in dense_layers:
                    tape.watch(x)
                    layer_outputs[layer.name] = x
            preds = x
            # compute per-sample loss (shape [batch,] or [batch, ...])
            per_sample_loss = loss_fn(yb_proc, preds)
            # compute scalar loss for gradient direction
            loss = tf.reduce_mean(per_sample_loss)

        for layer in dense_layers:
            name = layer.name
            if name not in layer_outputs:
                continue
            a = layer_outputs[name]            # shape (batch, units)
            A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
            V = tf.math.reduce_variance(a, axis=0).numpy()

            g = tape.gradient(loss, a)
            if g is None:
                G = np.zeros_like(A)
            else:
                G = tf.reduce_mean(tf.abs(g), axis=0).numpy()

            acc_A[name].append(A)
            acc_G[name].append(G)
            acc_V[name].append(V)

        del tape

    # Aggregate
    stats = {}
    for layer in dense_layers:
        name = layer.name
        if len(acc_A[name]) == 0:
            stats[name] = (np.array([]), np.array([]), np.array([]))
            continue
        A = np.mean(np.stack(acc_A[name], axis=0), axis=0)
        G = np.mean(np.stack(acc_G[name], axis=0), axis=0)
        V = np.mean(np.stack(acc_V[name], axis=0), axis=0)
        stats[name] = (A, G, V)

    return stats

# =====================================================================
# 4) IMPORTANCE MASKS (protect last Dense layer)
# =====================================================================
def compute_importance_mask(stats, keep_ratio=0.7, alpha=0.5, beta=0.3, gamma=0.2, last_dense_name=None):
    """
    stats: dict layer.name -> (A,G,V)
    last_dense_name: name of final Dense layer which must be preserved
    """
    masks = {}
    for name, (A, G, V) in stats.items():
        if A.size == 0:
            masks[name] = np.array([], dtype=np.float32)
            continue

        def normalize(x):
            x2 = x - np.min(x)
            mx = np.max(x2)
            return x2 / (mx + 1e-12)

        score = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)

        # protect final Dense layer: keep all units
        if last_dense_name is not None and name == last_dense_name:
            masks[name] = np.ones_like(score, dtype=np.float32)
            continue

        k = max(1, int(len(score) * keep_ratio))
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)
        masks[name] = mask

    return masks

# =====================================================================
# 5) DNAGate Layer (Zeroing)
# =====================================================================
class DNAGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        self._mask_np = np.array(mask, dtype=np.float32) if mask is not None else None
        self.mask = None

    def build(self, input_shape):
        if self._mask_np is None:
            self.mask = None
        else:
            if input_shape[-1] != self._mask_np.shape[0]:
                raise ValueError(f"DNAGate mask length {self._mask_np.shape[0]} != layer units {input_shape[-1]}")
            self.mask = tf.constant(self._mask_np, dtype=self.dtype)
        super().build(input_shape)

    def call(self, x):
        if self.mask is None:
            return x
        return x * self.mask

def build_masked_model(orig_model, masks):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = DNAGate(mask=masks[layer.name], name=layer.name + "_dna")(x)
    new_model = Model(inp, x)
    # copy weights where possible
    for l in orig_model.layers:
        try:
            new_model.get_layer(l.name).set_weights(l.get_weights())
        except Exception:
            pass
    return new_model

# =====================================================================
# 6) FLOPS + INFERENCE TIME
# =====================================================================
def dense_flops(in_size, out_size, bias=True):
    # multiply-adds counted as 2 ops per MAC
    return int(in_size * out_size * 2 + (out_size if bias else 0))

def model_flops(model):
    total = 0
    # attempt to get input size robustly
    in_shape = model.input_shape
    if isinstance(in_shape, tuple):
        in_size = int(in_shape[1])
    else:
        in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            total += dense_flops(in_size, layer.units, layer.use_bias)
            in_size = layer.units
    return total

def effective_flops(model, masks):
    total = 0
    in_shape = model.input_shape
    if isinstance(in_shape, tuple):
        in_size = int(in_shape[1])
    else:
        in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            if layer.name in masks and masks[layer.name].size > 0:
                units = int(np.sum(masks[layer.name]))
            else:
                units = layer.units
            total += dense_flops(in_size, units, layer.use_bias)
            in_size = units
    return total

def flops_to_gflops(f):
    return f / 1e9

def flops_to_mflops(f):
    return f / 1e6

def measure_inference_time(model, X, runs=200):
    """Return average inference time in milliseconds per sample.
       Accepts X as pandas DataFrame or numpy array."""
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if len(X_np) == 0:
        return float("nan")
    idx = np.random.randint(0, len(X_np), size=min(runs, len(X_np)))
    samples = X_np[idx]
    # make sure the shape matches model input: model expects shape (batch, input_dim)
    try:
        model.predict(samples[:1], verbose=0)
    except Exception:
        # try flattening last axis or trimming/padding to required input dims externally
        raise
    t0 = time.time()
    model.predict(samples, verbose=0)
    t1 = time.time()
    total_ms = (t1 - t0) * 1000.0
    return total_ms / len(samples)

# =====================================================================
# 7) STRUCTURAL PRUNING (REMOVE NEURONS) — protect final Dense layer
# =====================================================================
def structurally_prune_fnn(orig_model, masks, last_dense_name=None):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    prev_sel = None

    for layer in orig_model.layers:
        if isinstance(layer, Dense):
            W, b = layer.get_weights()

            # If previous layer was pruned → prune rows
            if prev_sel is not None:
                W = W[prev_sel, :]

            # ---- SAFETY: determine neurons to keep ----
            if last_dense_name is not None and layer.name == last_dense_name:
                # never prune final layer
                sel = np.arange(layer.units)
            else:
                mask = masks.get(layer.name, None)
                if mask is not None and mask.size > 0:
                    sel = np.where(mask == 1)[0]
                    # ensure >=1 neuron survives
                    if len(sel) == 0:
                        sel = np.array([np.argmax(mask)])
                else:
                    sel = None

            # ---- Apply pruning ----
            if sel is not None:
                W_new = W[:, sel]
                b_new = b[sel]
                new_units = len(sel)
            else:
                W_new = W
                b_new = b
                new_units = W.shape[1]

            new_layer = Dense(new_units, activation=layer.activation, name=layer.name)
            x = new_layer(x)
            new_layer.set_weights([W_new, b_new])

            prev_sel = sel

        else:
            x = layer(x)

    return Model(inp, x)

# =====================================================================
# 8) MAIN PIPELINE (auto-detect model type)
# =====================================================================
def fnn_pruning_pipeline(model_path, dataset_path,
                         keep_ratio=0.7,
                         alpha=0.5, beta=0.3, gamma=0.2,
                         calib_batches=30, batch_size=64, ft_epochs=3,
                         save_pruned_path=None):
    # 1) load model and detect type
    print("Loading model:", model_path)
    model = load_model(model_path)
    model_type = detect_model_type(model)
    print(f"[INFO] Detected model type: {model_type}")

    loss_name, metrics = get_loss_and_metrics_for_type(model_type)
    print(f"[INFO] Using loss='{loss_name}', metrics={metrics}")

    # 2) load dataset adapted to model_type
    print("Loading dataset:", dataset_path)
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_any_csv_dataset(dataset_path, model_type=model_type)

    # --- ensure feature dimension matches model input by trimming or padding ---
    model_input_dim = int(model.input_shape[1])
    def ensure_feature_dim(X_df):
        # X_df can be numpy or DataFrame
        if isinstance(X_df, pd.DataFrame):
            arr = X_df.to_numpy().astype(np.float32)
        else:
            arr = np.asarray(X_df, dtype=np.float32)
        cur_dim = arr.shape[1]
        if cur_dim == model_input_dim:
            return arr
        if cur_dim > model_input_dim:
            print(f"[WARN] Truncating features from {cur_dim} -> {model_input_dim} to match model input.")
            return arr[:, :model_input_dim]
        # cur_dim < model_input_dim -> pad with zeros
        print(f"[WARN] Padding features from {cur_dim} -> {model_input_dim} to match model input.")
        pad = np.zeros((arr.shape[0], model_input_dim - cur_dim), dtype=np.float32)
        return np.concatenate([arr, pad], axis=1)

    X_train = ensure_feature_dim(X_train)
    X_val = ensure_feature_dim(X_val)
    X_test = ensure_feature_dim(X_test)

    # 3) find dense layers and the last Dense layer name
    dense_layers = get_dense_layers(model)
    last_dense_name = dense_layers[-1].name if len(dense_layers) > 0 else None
    print("Dense layers:", [l.name for l in dense_layers], "| last_dense:", last_dense_name)

    # 4) compute stats (activation + gradient)
    print("Computing activation & gradient stats...")
    stats = compute_activation_grad_stats(model, dense_layers, X_train, y_train, model_type,
                                          batch_size=batch_size, max_batches=calib_batches)

    # 5) compute masks (protect final layer)
    masks = compute_importance_mask(stats, keep_ratio=keep_ratio, alpha=alpha, beta=beta, gamma=gamma, last_dense_name=last_dense_name)
    for k, v in masks.items():
        if v.size > 0:
            print(f"{k} -> kept units: {int(np.sum(v))}/{len(v)}")

    # 6) build masked model (zeroing) and compile with correct loss/metrics
    print("Building masked model (DNAGate zeroing)...")
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    # compute baseline & masked flops/accuracy
    baseline_acc = None
    masked_acc_before = None
    if model_type == "regression":
        baseline_eval = model.evaluate(X_val, y_val, verbose=0)
        baseline_acc = baseline_eval[0] if len(baseline_eval) > 0 else None
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[0]
    else:
        baseline_eval = model.evaluate(X_val, y_val, verbose=0)
        masked_eval_before = masked_model.evaluate(X_val, y_val, verbose=0)
        # baseline_eval may produce [loss, metric...] so metric index 1
        baseline_acc = baseline_eval[1] if len(baseline_eval) > 1 else (baseline_eval[0] if len(baseline_eval)==1 else None)
        masked_acc_before = masked_eval_before[1] if len(masked_eval_before) > 1 else (masked_eval_before[0] if len(masked_eval_before)==1 else None)

    baseline_flops = model_flops(model)
    masked_flops = effective_flops(model, masks)

    print(f"Baseline Acc: {baseline_acc} | FLOPS: {baseline_flops}")
    print(f"Masked Acc (before FT): {masked_acc_before} | Effective FLOPS: {masked_flops}")

    # 7) fine-tune masked model
    print("Fine-tuning masked model...")
    masked_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # evaluate masked after FT
    if model_type == "regression":
        masked_eval = masked_model.evaluate(X_val, y_val, verbose=0)
        masked_acc_after = masked_eval[0] if len(masked_eval) > 0 else None
    else:
        masked_eval = masked_model.evaluate(X_val, y_val, verbose=0)
        masked_acc_after = masked_eval[1] if len(masked_eval) > 1 else (masked_eval[0] if len(masked_eval)==1 else None)

    # 8) structural pruning (permanent neuron removal)
    print("Constructing structurally-pruned model (remove neurons)...")
    pruned_model = structurally_prune_fnn(model, masks, last_dense_name=last_dense_name)
    pruned_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    print("Fine-tuning pruned model...")
    pruned_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # final evals
    if model_type == "regression":
        pruned_eval = pruned_model.evaluate(X_val, y_val, verbose=0)
        pruned_acc = pruned_eval[0] if len(pruned_eval) > 0 else None
    else:
        pruned_eval = pruned_model.evaluate(X_val, y_val, verbose=0)
        pruned_acc = pruned_eval[1] if len(pruned_eval) > 1 else (pruned_eval[0] if len(pruned_eval)==1 else None)

    pruned_flops = model_flops(pruned_model)

    # compute GFLOPS / MFLOPS / inference times
    baseline_gflops = flops_to_gflops(baseline_flops)
    masked_gflops = flops_to_gflops(masked_flops)
    pruned_gflops = flops_to_gflops(pruned_flops)

    baseline_mflops = flops_to_mflops(baseline_flops)
    masked_mflops = flops_to_mflops(masked_flops)
    pruned_mflops = flops_to_mflops(pruned_flops)

    baseline_time_ms = measure_inference_time(model, X_val)
    masked_time_ms = measure_inference_time(masked_model, X_val)
    pruned_time_ms = measure_inference_time(pruned_model, X_val)

    print("\n=== FINAL REPORT ===")
    print(f"Model type: {model_type}")
    print(f"Baseline:   Acc/metric={baseline_acc} | FLOPS={baseline_flops} | MFLOPS={baseline_mflops:.6f} | GFLOPS={baseline_gflops:.10f} | Time={baseline_time_ms:.4f} ms/sample")
    print(f"Masked:     Acc/metric={masked_acc_after} | EFLOPS={masked_flops} | MFLOPS={masked_mflops:.6f} | GFLOPS={masked_gflops:.10f} | Time={masked_time_ms:.4f} ms/sample")
    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")

    # ---- NEW METRIC REDUCTIONS ADDED BELOW ----
    # FLOPS reduction (absolute and percent)
    if baseline_flops > 0:
        flop_reduction_abs = baseline_flops - pruned_flops
        flop_reduction_pct = flop_reduction_abs / baseline_flops
    else:
        flop_reduction_abs = 0
        flop_reduction_pct = 0.0

    # GFLOPS reduction (absolute and percent)
    if baseline_gflops > 0:
        gflop_reduction_abs = baseline_gflops - pruned_gflops
        gflop_reduction_pct = gflop_reduction_abs / baseline_gflops
    else:
        gflop_reduction_abs = 0.0
        gflop_reduction_pct = 0.0

    # Accuracy reduction (absolute + percent relative to baseline if available)
    acc_abs_reduction = None
    acc_pct_reduction = None
    if baseline_acc is not None and pruned_acc is not None:
        acc_abs_reduction = baseline_acc - pruned_acc
        if baseline_acc != 0:
            acc_pct_reduction = acc_abs_reduction / baseline_acc
        else:
            acc_pct_reduction = None

    # Print nicely
    print("\n=== REDUCTION SUMMARY ===")
    print(f"FLOPS reduced (absolute) : {flop_reduction_abs} ops")
    print(f"FLOPS reduced (percent)  : {flop_reduction_pct:.2%}")
    print(f"GFLOPS reduced (absolute): {gflop_reduction_abs:.10f} GFLOPS")
    print(f"GFLOPS reduced (percent) : {gflop_reduction_pct:.2%}")
    if acc_abs_reduction is not None:
        print(f"Accuracy change (absolute): {acc_abs_reduction:.6f}")
        if acc_pct_reduction is not None:
            print(f"Accuracy change (percent of baseline): {acc_pct_reduction:.2%}")
        else:
            print("Accuracy percent change: N/A (baseline is zero or unavailable)")
    else:
        print("Accuracy change: N/A (baseline or pruned accuracy not available)")
    print("=========================\n")
    # ---- END NEW LINES ----

    # optional: save pruned model
    if save_pruned_path:
        pruned_model.save(save_pruned_path)
        print(f"[INFO] Saved pruned model to: {save_pruned_path}")

    return model, masked_model, pruned_model, masks, X_val

# =====================================================================
# 9) RUN (example)
# =====================================================================
if __name__ == "__main__":
    # change these to your files
    model_path = r"D:\college\sem-8\models\asthma_fnn_model (1).h5"
    dataset_path = r"D:\college\sem-8\dataset\asthma_disease_data (1).csv"

    original_model, masked_model, pruned_model, masks, X_val = fnn_pruning_pipeline(
        model_path=model_path,
        dataset_path=dataset_path,
        keep_ratio=0.9,
        alpha=0.5, beta=0.3, gamma=0.2,
        calib_batches=30, batch_size=64, ft_epochs=3,
        save_pruned_path=None
    )


Loading model: D:\college\sem-8\models\asthma_fnn_model (1).h5


[INFO] Detected model type: binary
[INFO] Using loss='binary_crossentropy', metrics=['accuracy']
Loading dataset: D:\college\sem-8\dataset\asthma_disease_data (1).csv
[INFO] Loading CSV with automatic delimiter detection...
[INFO] Columns detected: ['PatientID', 'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI', 'Smoking', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'PollutionExposure', 'PollenExposure', 'DustExposure', 'PetAllergy', 'FamilyHistoryAsthma', 'HistoryOfAllergies', 'Eczema', 'HayFever', 'GastroesophagealReflux', 'LungFunctionFEV1', 'LungFunctionFVC', 'Wheezing', 'ShortnessOfBreath', 'ChestTightness', 'Coughing', 'NighttimeSymptoms', 'ExerciseInduced', 'Unnamed: 27', 'DoctorInCharge', 'Diagnosis']
[INFO] Auto-detected target column = Diagnosis
[INFO] Target converted to numeric successfully.
[INFO] One-hot encoding predictors: ['DoctorInCharge']
Dense layers: ['dense_10', 'dense_11', 'dense_12', 'dense_13'] | last_dense: dense_13
Computing activation & gradient s

# **CNN Structure Pruning 3 Epochs**

In [5]:
"""
sequential_cnn_pruning_full.py
Full Sequential CNN pruning pipeline (single-file)

- Loads user Sequential CNN (.h5 / .keras / SavedModel)
- Sanitizes layer names in .h5 if they contain '/'
- Loads folder dataset (image_dataset_from_directory)
- Computes activation/gradient/variance importance
- Creates masks (keep_ratio)
- Builds masked model (gating layer)
- Fine-tunes masked model
- Structurally prunes model (conv filter removal, prune Dense outputs only)
- Computes FLOPS & timings, evaluates models
- Saves models and masks

Note: This is intended for Sequential CNN models (Conv2D -> ... -> Flatten -> Dense -> ... -> Dense output).
"""

import os
import json
import math
import time
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# ---------------------------
# USER CONFIG
# ---------------------------
MODEL_PATH = r"D:\college\sem-8\models\flower_cnn_model.h5"   # path to model (.h5/.keras or SavedModel dir)
DATASET_PATH = r"D:\college\sem-8\dataset\flower_photos"      # folder with subfolders per class
SAVE_DIR = r"pruning_output"                                 # where outputs are saved

KEEP_RATIO = 0.7            # fraction of channels/units to keep
ALPHA, BETA, GAMMA = 0.4, 0.3, 0.3  # importance weights
BATCH_SIZE = 64
CALIB_BATCHES = 30
FT_EPOCHS = 3
FT_BATCHES_TO_USE = 150
PLOT_RESULTS = True
VERBOSE = True

os.makedirs(SAVE_DIR, exist_ok=True)
tf.random.set_seed(42)
np.random.seed(42)

def log(*args):
    if VERBOSE:
        print("[INFO]", *args)

# ---------------------------
# Safe load model (handles '/' in h5 layer names)
# ---------------------------
def safe_load_model(model_path):
    """
    Try normal load_model; if fails (e.g., '/' in layer names in H5), sanitize model_config JSON and rebuild.
    Returns a Keras model (compiled=False).
    """
    log("Loading model:", model_path)
    # Try direct load first
    try:
        m = tf.keras.models.load_model(model_path, compile=False)
        log("Loaded model directly.")
        return m
    except Exception as e:
        log("Direct load failed:", e)

    # If HDF5, attempt to sanitize layer names in model_config
    try:
        with h5py.File(model_path, "r") as f:
            if "model_config" in f:
                raw = f["model_config"][()]
                if isinstance(raw, bytes):
                    raw = raw.decode("utf-8")
                cfg_json = json.loads(raw)
                changed = False
                for layer in cfg_json.get("config", {}).get("layers", []):
                    cfg = layer.get("config", {})
                    name = cfg.get("name")
                    if isinstance(name, str) and "/" in name:
                        new_name = name.replace("/", "_")
                        cfg["name"] = new_name
                        changed = True
                        log(f"[FIX] layer name: {name} -> {new_name}")
                if changed:
                    fixed_json = json.dumps(cfg_json)
                    model = tf.keras.models.model_from_json(fixed_json)
                    model.load_weights(model_path)
                    log("Loaded model from sanitized JSON + weights.")
                    return model
    except Exception as e2:
        log("H5 sanitization attempt failed:", e2)

    # final fallback: try load_model with safe_mode=False (older TF)
    try:
        m = tf.keras.models.load_model(model_path, compile=False, safe_mode=False)
        log("Loaded model with safe_mode=False.")
        return m
    except Exception as e3:
        log("All load attempts failed:", e3)
        raise RuntimeError("Failed to load model. Ensure path and format are correct.")

# ---------------------------
# Dataset loader (folder-based)
# ---------------------------
def load_image_folder_dataset(path, image_size, batch_size=BATCH_SIZE):
    log("Loading dataset folder:", path, "image_size:", image_size)
    train_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    rescaler = layers.Rescaling(1.0 / 255)
    train_ds = train_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds

# ---------------------------
# Activation & gradient stats
# ---------------------------
def compute_activation_grad_stats(model, layer_names, dataset, max_batches=CALIB_BATCHES):
    """
    For each layer in layer_names, compute:
      - A: mean(abs(activation)) per filter/unit
      - G: mean(abs(grad wrt activation)) per filter/unit
      - V: variance(activation) per filter/unit
    Returns: dict name -> (A, G, V)
    """
    log("Computing activation & gradient stats...")
    results = {n: [] for n in layer_names}
    grad_results = {n: [] for n in layer_names}
    var_results = {n: [] for n in layer_names}

    batch_count = 0
    for x_batch, y_batch in dataset:
        if batch_count >= max_batches:
            break
        batch_count += 1
        layer_acts = {}
        with tf.GradientTape(persistent=True) as tape:
            a = x_batch
            for layer in model.layers:
                a = layer(a)
                if layer.name in layer_names:
                    tape.watch(a)
                    layer_acts[layer.name] = a
            preds = a
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, preds)
            loss = tf.reduce_mean(loss)

        for name in layer_names:
            a = layer_acts[name]
            if len(a.shape) == 4:
                A = tf.reduce_mean(tf.abs(a), axis=(0,1,2)).numpy()
                V = tf.math.reduce_variance(a, axis=(0,1,2)).numpy()
            else:
                A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
                V = tf.math.reduce_variance(a, axis=0).numpy()

            grad = tape.gradient(loss, a)
            if grad is None:
                G = np.zeros_like(A)
            else:
                if len(grad.shape) == 4:
                    G = tf.reduce_mean(tf.abs(grad), axis=(0,1,2)).numpy()
                else:
                    G = tf.reduce_mean(tf.abs(grad), axis=0).numpy()

            results[name].append(A)
            var_results[name].append(V)
            grad_results[name].append(G)
        del tape

    stats = {}
    for name in layer_names:
        A = np.mean(results[name], axis=0)
        V = np.mean(var_results[name], axis=0)
        G = np.mean(grad_results[name], axis=0)
        stats[name] = (A, G, V)
        log(f"{name}: len={len(A)} meanA={A.mean():.6e} meanG={G.mean():.6e} meanV={V.mean():.6e}")
    return stats

# ---------------------------
# Importance scores & masks
# ---------------------------
def compute_importance_scores(stats, alpha=ALPHA, beta=BETA, gamma=GAMMA):
    def normalize(x):
        x = x - x.min()
        if x.max() > 0:
            x = x / x.max()
        return x
    scores = {}
    for name, (A, G, V) in stats.items():
        scores[name] = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)
    return scores

def make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO):
    masks = {}
    for name, score in score_map.items():
        k = max(1, int(len(score) * keep_ratio))
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)
        masks[name] = mask
        log(f"{name}: keep {int(mask.sum())}/{len(mask)}")
    return masks

# ---------------------------
# Mask gate layer & masked model
# ---------------------------
class CNNGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        self.mask = tf.constant(mask, dtype=tf.float32) if mask is not None else None
    def call(self, x):
        if self.mask is None:
            return x
        if len(x.shape) == 4:
            return x * tf.reshape(self.mask, (1,1,1,-1))
        else:
            return x * self.mask

def build_masked_model(orig_model, masks):
    log("Building masked model with gates...")
    inp = tf.keras.Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = CNNGate(mask=masks[layer.name], name=layer.name + "_gate")(x)
    masked = tf.keras.Model(inputs=inp, outputs=x, name=orig_model.name + "_masked")
    # copy weights (best-effort)
    for layer in orig_model.layers:
        try:
            masked_layer = masked.get_layer(layer.name)
            masked_layer.set_weights(layer.get_weights())
        except Exception:
            # layer might not exist by same name (e.g., InputLayer) -> ignore
            pass
    return masked

# ---------------------------
# Structural pruning (safe)
# ---------------------------
def prune_structural_sequential(orig_model, masks, input_shape):
    """
    Structural pruning for Sequential models.
    - Prune Conv2D output filters using masks[layer.name]
    - Prune Dense output units only (do not slice Dense input rows that come from Flatten/Conv)
    - Attempt to slice BatchNorm params to match conv outputs
    """
    log("Structural pruning (safe) start...")
    new_layers = []
    prev_was_conv_like = False  # indicates that Flatten/Conv preceded Dense inputs

    for layer in orig_model.layers:
        # Conv2D: slice output channels
        if isinstance(layer, layers.Conv2D):
            W, b = layer.get_weights()
            orig_out = W.shape[-1]
            mask = masks.get(layer.name, np.ones(orig_out, dtype=np.float32))
            keep_idx = np.where(mask == 1)[0]
            if keep_idx.size == 0:
                keep_idx = np.array([int(np.argmax(mask))], dtype=np.int32)
            W_new = W[:, :, :, keep_idx]
            b_new = b[keep_idx]
            new_conv = layers.Conv2D(
                filters=len(keep_idx),
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=layer.activation,
                use_bias=layer.use_bias,
                name=layer.name + "_pruned"
            )
            new_layers.append((new_conv, [W_new, b_new]))
            prev_was_conv_like = True
            continue

        # BatchNorm: slice params if prev was conv-like
        if isinstance(layer, layers.BatchNormalization):
            try:
                weights = layer.get_weights()
                if prev_was_conv_like and new_layers:
                    prev_layer_obj, prev_w = new_layers[-1]
                    if prev_w is not None:
                        out_ch = prev_w[0].shape[-1]  # kernel last dim
                        gamma, beta, mean, var = weights
                        gamma = gamma[:out_ch]
                        beta = beta[:out_ch]
                        mean = mean[:out_ch]
                        var = var[:out_ch]
                        new_bn = layers.BatchNormalization.from_config(layer.get_config())
                        new_layers.append((new_bn, [gamma, beta, mean, var]))
                        continue
            except Exception:
                pass
            # fallback keep BN as-is
            try:
                new_bn = layers.BatchNormalization.from_config(layer.get_config())
                new_layers.append((new_bn, layer.get_weights()))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            # prev_was_conv_like unchanged
            continue

        # MaxPool/Activation/Dropout/Flatten/GlobalAvgPool: clone or reuse
        if isinstance(layer, (layers.MaxPooling2D, layers.Activation, layers.ReLU, layers.Dropout, layers.Flatten, layers.GlobalAveragePooling2D)):
            try:
                cloned = layer.__class__.from_config(layer.get_config())
                w = layer.get_weights() if hasattr(layer, "get_weights") else None
                new_layers.append((cloned, w if w else None))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            if isinstance(layer, layers.Flatten) or isinstance(layer, layers.GlobalAveragePooling2D):
                prev_was_conv_like = True
            continue

        # Dense: prune outputs only (safe)
        if isinstance(layer, layers.Dense):
            W, b = layer.get_weights()  # shape (in_dim, out_dim)
            out_mask = masks.get(layer.name, np.ones(W.shape[1], dtype=np.float32))
            out_idx = np.where(out_mask == 1)[0]
            if out_idx.size == 0:
                out_idx = np.array([int(np.argmax(out_mask))], dtype=np.int32)
            W_new = W[:, out_idx]   # keep all input rows (safe)
            b_new = b[out_idx]
            new_dense = layers.Dense(units=W_new.shape[1], activation=layer.activation, name=layer.name + "_pruned")
            new_layers.append((new_dense, [W_new, b_new]))
            prev_was_conv_like = False
            continue

        # Fallback for other layers
        try:
            cloned = layer.__class__.from_config(layer.get_config())
            w = layer.get_weights() if hasattr(layer, "get_weights") else None
            new_layers.append((cloned, w if w else None))
        except Exception:
            new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
        prev_was_conv_like = False

    # Build new Sequential model with InputLayer
    seq = models.Sequential(name=orig_model.name + "_struct_pruned")
    seq.add(layers.InputLayer(input_shape=tuple(input_shape)))
    for lyr_obj, w in new_layers:
        seq.add(lyr_obj)
        if w is not None:
            try:
                seq.layers[-1].set_weights(w)
            except Exception as e:
                log("Warning: couldn't set weights for", seq.layers[-1].name, ":", e)
    log("Structural pruning complete. New model summary:")
    seq.summary()
    return seq

# ---------------------------
# FLOPS and timing helpers
# ---------------------------
def calculate_conv_flops(input_shape, kernel_shape, strides=(1,1), padding='same'):
    h_in, w_in, c_in = input_shape
    kh, kw, _, c_out = kernel_shape
    if padding == 'same':
        h_out = math.ceil(h_in / strides[0])
        w_out = math.ceil(w_in / strides[1])
    else:
        h_out = math.ceil((h_in - kh + 1) / strides[0])
        w_out = math.ceil((w_in - kw + 1) / strides[1])
    flops = h_out * w_out * (kh * kw * c_in) * c_out * 2
    return flops, (h_out, w_out, c_out)

def calculate_dense_flops(in_size, out_size):
    return in_size * out_size * 2

def calculate_model_flops(model, input_shape):
    total = 0
    current_shape = tuple(input_shape)
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            weights = layer.get_weights()
            if not weights:
                continue
            kernel_shape = weights[0].shape  # (kh, kw, in_c, out_c)
            layer_flops, current_shape = calculate_conv_flops(current_shape, kernel_shape, strides=layer.strides, padding=layer.padding)
            total += layer_flops
        elif isinstance(layer, layers.Flatten):
            current_shape = (int(np.prod(current_shape)),)
        elif isinstance(layer, layers.Dense):
            in_size = current_shape[0] if isinstance(current_shape, tuple) and len(current_shape)>0 else int(current_shape)
            layer_flops = calculate_dense_flops(in_size, layer.units)
            total += layer_flops
            current_shape = (layer.units,)
        elif isinstance(layer, layers.MaxPooling2D):
            h,w,c = current_shape
            pool = layer.pool_size[0] if hasattr(layer.pool_size, "__getitem__") else layer.pool_size
            current_shape = (h//pool, w//pool, c)
        else:
            # ignore other layers for shape changes
            pass
    return total

def measure_inference_time(model, sample_batch, steps=20):
    model.predict(sample_batch, verbose=0)  # warmup
    t0 = time.time()
    for _ in range(steps):
        model.predict(sample_batch, verbose=0)
    t1 = time.time()
    return (t1 - t0) / steps

# ---------------------------
# Save masks helper
# ---------------------------
def save_masks(masks, path):
    serial = {k: v.tolist() for k,v in masks.items()}
    with open(path, "w") as f:
        json.dump(serial, f, indent=2)
    log("Saved masks to", path)

# ---------------------------
# Plot helper
# ---------------------------
def plot_mask_histograms(masks, outdir=SAVE_DIR):
    if not PLOT_RESULTS:
        return
    for name, mask in masks.items():
        plt.figure(figsize=(5,2))
        plt.title(name)
        plt.hist(mask, bins=2)
        plt.xlabel("0=pruned, 1=kept")
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"mask_{name}.png"))
        plt.close()

# ---------------------------
# MAIN pipeline
# ---------------------------
def full_pipeline(model_path, dataset_path):
    # 1) load model (safe)
    model = safe_load_model(model_path)
    log("Model loaded. Summary:")
    model.summary()

    # compile original so evaluate works (use small lr default)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # 2) infer input shape
    input_shape = model.input_shape[1:]
    log("Inferred input shape:", input_shape)

    # 3) load dataset
    train_ds, val_ds = load_image_folder_dataset(dataset_path, image_size=input_shape[:2], batch_size=BATCH_SIZE)
    # calibration subset
    calib_ds = train_ds.take(CALIB_BATCHES)

    # sample for timing
    try:
        sample_x, _ = next(iter(val_ds))
    except Exception:
        sample_x, _ = next(iter(train_ds))
    sample_x_small = sample_x[:min(16, sample_x.shape[0])]

    # 4) choose layers to prune (conv + hidden dense only)
    dense_layers = [lyr for lyr in model.layers if isinstance(lyr, layers.Dense)]
    last_dense = dense_layers[-1] if dense_layers else None

    prune_layer_names = []
    for lyr in model.layers:
        if isinstance(lyr, layers.Conv2D):
            prune_layer_names.append(lyr.name)
        elif isinstance(lyr, layers.Dense) and lyr is not last_dense:
            prune_layer_names.append(lyr.name)
    log("Layers considered for pruning:", prune_layer_names)

    # 5) compute stats
    stats = compute_activation_grad_stats(model, prune_layer_names, calib_ds, max_batches=CALIB_BATCHES)

    # 6) importance & masks
    score_map = compute_importance_scores(stats)
    masks = make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO)
    save_masks(masks, os.path.join(SAVE_DIR, "masks.json"))
    plot_mask_histograms(masks)

    # 7) build masked model and compile
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    log("Masked model built.")

    # quick eval before FT
    try:
        loss0, acc0 = masked_model.evaluate(val_ds.take(5), verbose=0)
        log("Masked model pre-FT acc:", acc0)
    except Exception as e:
        log("Masked model pre-eval failed:", e)

    # 8) measure baseline flops & time
    baseline_flops = calculate_model_flops(model, input_shape)
    baseline_time = measure_inference_time(model, sample_x_small, steps=10)
    log(f"Baseline FLOPS: {baseline_flops:,}, baseline time (avg batch): {baseline_time:.4f}s")

    # 9) fine-tune masked model
    try:
        masked_model.fit(train_ds.take(FT_BATCHES_TO_USE), validation_data=val_ds.take(5), epochs=FT_EPOCHS, verbose=2)
    except Exception as e:
        log("Masked fine-tune failed/partial:", e)

    # 10) structural prune
    pruned_model = prune_structural_sequential(model, masks, input_shape)
    # compile pruned model for evaluation
    pruned_model.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # 11) evaluate models
    log("Evaluating Original model:")
    orig_loss, orig_acc = model.evaluate(val_ds, verbose=0)
    log("Original acc:", orig_acc)

    log("Evaluating Masked model (after FT):")
    try:
        mask_loss, mask_acc = masked_model.evaluate(val_ds, verbose=0)
        log("Masked acc:", mask_acc)
    except Exception as e:
        log("Masked evaluate failed:", e)
        mask_acc = None

    log("Evaluating Pruned model:")
    try:
        pruned_loss, pruned_acc = pruned_model.evaluate(val_ds, verbose=0)
        log("Pruned acc:", pruned_acc)
    except Exception as e:
        log("Pruned evaluate failed:", e)
        pruned_acc = None

    # 12) FLOPS & timing after prune
    pruned_flops = calculate_model_flops(pruned_model, input_shape)
    pruned_time = measure_inference_time(pruned_model, sample_x_small, steps=10)
    log(f"Pruned FLOPS: {pruned_flops:,}, pruned time: {pruned_time:.4f}s")

    # 13) summary & save
    reduction = 1.0 - (pruned_flops / baseline_flops) if baseline_flops > 0 else 0.0
    log("="*60)
    log("SUMMARY:")
    log(f"Baseline FLOPS: {baseline_flops:,}")
    log(f"Pruned FLOPS:   {pruned_flops:,}")
    log(f"FLOPS reduction: {reduction:.2%}")
    log(f"Original acc: {orig_acc}, Masked acc: {mask_acc}, Pruned acc: {pruned_acc}")
    log("="*60)

    # save artifacts
    try:
        model.save(os.path.join(SAVE_DIR, "model_baseline"))
        masked_model.save(os.path.join(SAVE_DIR, "model_masked"))
        pruned_model.save(os.path.join(SAVE_DIR, "model_pruned"))
        log("Saved models to", SAVE_DIR)
    except Exception as e:
        log("Save models failed:", e)

    return {
        "model": model,
        "masked_model": masked_model,
        "pruned_model": pruned_model,
        "masks": masks,
        "baseline_flops": baseline_flops,
        "pruned_flops": pruned_flops,
        "baseline_time": baseline_time,
        "pruned_time": pruned_time,
        "orig_acc": orig_acc,
        "mask_acc": mask_acc,
        "pruned_acc": pruned_acc
    }

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    out = full_pipeline(MODEL_PATH, DATASET_PATH)
    log("Pipeline finished. Outputs:", out.keys())


[INFO] Loading model: D:\college\sem-8\models\flower_cnn_model.h5
[INFO] Loaded model directly.
[INFO] Model loaded. Summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 18, 18, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 7, 7, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536,645 (2.05 MB)

 Trainable params: 536,645 (2.05 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5_pruned (Conv2D)        │ (None, 178, 178, 22)   │           616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 89, 89, 22)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6_pruned (Conv2D)        │ (None, 87, 87, 44)     │         8,756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 43, 43, 44)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7_pruned (Conv2D)        │ (None, 41, 41, 89)     │        35,333 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 20, 20, 89)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8_pruned (Conv2D)        │ (None, 18, 18, 89)     │        71,378 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 9, 9, 89)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9_pruned (Conv2D)        │ (None, 7, 7, 89)       │        71,378 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 3, 3, 89)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 801)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8_pruned (Dense)          │ (None, 89)             │        71,378 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 89)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9_pruned (Dense)          │ (None, 5)              │           450 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 259,289 (1012.85 KB)

 Trainable params: 259,289 (1012.85 KB)

 Non-trainable params: 0 (0.00 B)

[INFO] Evaluating Original model:
[INFO] Original acc: 0.4904632270336151
[INFO] Evaluating Masked model (after FT):
[INFO] Masked acc: 0.486376017332077
[INFO] Evaluating Pruned model:
[INFO] Pruned acc: 0.2220708429813385
[INFO] Pruned FLOPS: 341,338,238, pruned time: 0.1162s
[INFO] ============================================================
[INFO] SUMMARY:
[INFO] Baseline FLOPS: 691,945,472
[INFO] Pruned FLOPS:   341,338,238
[INFO] FLOPS reduction: 50.67%
[INFO] Original acc: 0.4904632270336151, Masked acc: 0.486376017332077, Pruned acc: 0.2220708429813385
[INFO] ============================================================
[INFO] Save models failed: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=pruning_output\model_baseline.
[INFO] Pipeline finished. Outputs: dict_keys(['

# **CNN structure Pruning with 5 Epochs**

In [45]:
"""
sequential_cnn_pruning_metrics.py
Extended Sequential CNN pruning pipeline with detailed metrics:
- GFLOPS (baseline & pruned)
- Speed-up ratio
- Accuracy drop %
- Parameter reduction %
- Per-layer remaining filters/neurons
- Saves models with .keras extension
"""

import os
import json
import math
import time
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# ---------------------------
# USER CONFIG
# ---------------------------
MODEL_PATH = r"D:\college\sem-8\models\garbage_cnn_model.h5"   # path to model (.h5/.keras or SavedModel dir)
DATASET_PATH = r"D:\college\sem-8\dataset\reduced_garbage_dataset"      # folder with subfolders per class
SAVE_DIR = r"pruning_output"                                 # where outputs are saved
MODEL_SAVE_EXT = ".keras"   # use ".keras" or ".h5"

KEEP_RATIO = 0.7            # fraction of channels/units to keep
ALPHA, BETA, GAMMA = 0.4, 0.3, 0.3  # importance weights
BATCH_SIZE = 64
CALIB_BATCHES = 30
FT_EPOCHS = 5
FT_BATCHES_TO_USE = 150
PLOT_RESULTS = True
VERBOSE = True

os.makedirs(SAVE_DIR, exist_ok=True)
tf.random.set_seed(42)
np.random.seed(42)

def log(*args):
    if VERBOSE:
        print("[INFO]", *args)

# ---------------------------
# Safe load model (handles '/' in h5 layer names)
# ---------------------------
def safe_load_model(model_path):
    log("Loading model:", model_path)
    try:
        m = tf.keras.models.load_model(model_path, compile=False)
        log("Loaded model directly.")
        return m
    except Exception as e:
        log("Direct load failed:", e)

    try:
        with h5py.File(model_path, "r") as f:
            if "model_config" in f:
                raw = f["model_config"][()]
                if isinstance(raw, bytes):
                    raw = raw.decode("utf-8")
                cfg_json = json.loads(raw)
                changed = False
                for layer in cfg_json.get("config", {}).get("layers", []):
                    cfg = layer.get("config", {})
                    name = cfg.get("name")
                    if isinstance(name, str) and "/" in name:
                        new_name = name.replace("/", "_")
                        cfg["name"] = new_name
                        changed = True
                        log(f"[FIX] layer name: {name} -> {new_name}")
                if changed:
                    fixed_json = json.dumps(cfg_json)
                    model = tf.keras.models.model_from_json(fixed_json)
                    model.load_weights(model_path)
                    log("Loaded model from sanitized JSON + weights.")
                    return model
    except Exception as e2:
        log("H5 sanitization attempt failed:", e2)

    try:
        m = tf.keras.models.load_model(model_path, compile=False, safe_mode=False)
        log("Loaded model with safe_mode=False.")
        return m
    except Exception as e3:
        log("All load attempts failed:", e3)
        raise RuntimeError("Failed to load model. Ensure path and format are correct.")

# ---------------------------
# Dataset loader (folder-based)
# ---------------------------
def load_image_folder_dataset(path, image_size, batch_size=BATCH_SIZE):
    log("Loading dataset folder:", path, "image_size:", image_size)
    train_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    rescaler = layers.Rescaling(1.0 / 255)
    train_ds = train_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds

# ---------------------------
# Activation & gradient stats
# ---------------------------
def compute_activation_grad_stats(model, layer_names, dataset, max_batches=CALIB_BATCHES):
    log("Computing activation & gradient stats...")
    results = {n: [] for n in layer_names}
    grad_results = {n: [] for n in layer_names}
    var_results = {n: [] for n in layer_names}
    batch_count = 0
    for x_batch, y_batch in dataset:
        if batch_count >= max_batches:
            break
        batch_count += 1
        layer_acts = {}
        with tf.GradientTape(persistent=True) as tape:
            a = x_batch
            for layer in model.layers:
                a = layer(a)
                if layer.name in layer_names:
                    tape.watch(a)
                    layer_acts[layer.name] = a
            preds = a
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, preds)
            loss = tf.reduce_mean(loss)
        for name in layer_names:
            a = layer_acts[name]
            if len(a.shape) == 4:
                A = tf.reduce_mean(tf.abs(a), axis=(0,1,2)).numpy()
                V = tf.math.reduce_variance(a, axis=(0,1,2)).numpy()
            else:
                A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
                V = tf.math.reduce_variance(a, axis=0).numpy()
            grad = tape.gradient(loss, a)
            if grad is None:
                G = np.zeros_like(A)
            else:
                if len(grad.shape) == 4:
                    G = tf.reduce_mean(tf.abs(grad), axis=(0,1,2)).numpy()
                else:
                    G = tf.reduce_mean(tf.abs(grad), axis=0).numpy()
            results[name].append(A)
            var_results[name].append(V)
            grad_results[name].append(G)
        del tape
    stats = {}
    for name in layer_names:
        A = np.mean(results[name], axis=0)
        V = np.mean(var_results[name], axis=0)
        G = np.mean(grad_results[name], axis=0)
        stats[name] = (A, G, V)
        log(f"{name}: len={len(A)} meanA={A.mean():.6e} meanG={G.mean():.6e} meanV={V.mean():.6e}")
    return stats

# ---------------------------
# Importance scores & masks
# ---------------------------
def compute_importance_scores(stats, alpha=ALPHA, beta=BETA, gamma=GAMMA):
    def normalize(x):
        x = x - x.min()
        if x.max() > 0:
            x = x / x.max()
        return x
    scores = {}
    for name, (A, G, V) in stats.items():
        scores[name] = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)
    return scores

def make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO):
    masks = {}
    for name, score in score_map.items():
        k = max(1, int(len(score) * keep_ratio))
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)
        masks[name] = mask
        log(f"{name}: keep {int(mask.sum())}/{len(mask)}")
    return masks

# ---------------------------
# Mask gate layer & masked model
# ---------------------------
class CNNGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        self.mask = tf.constant(mask, dtype=tf.float32) if mask is not None else None
    def call(self, x):
        if self.mask is None:
            return x
        if len(x.shape) == 4:
            return x * tf.reshape(self.mask, (1,1,1,-1))
        else:
            return x * self.mask

def build_masked_model(orig_model, masks):
    log("Building masked model with gates...")
    inp = tf.keras.Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = CNNGate(mask=masks[layer.name], name=layer.name + "_gate")(x)
    masked = tf.keras.Model(inputs=inp, outputs=x, name=orig_model.name + "_masked")
    for layer in orig_model.layers:
        try:
            masked_layer = masked.get_layer(layer.name)
            masked_layer.set_weights(layer.get_weights())
        except Exception:
            pass
    return masked

# ---------------------------
# Structural pruning (safe)
# ---------------------------
def prune_structural_sequential(orig_model, masks, input_shape):
    log("Structural pruning (safe) start...")
    new_layers = []
    prev_was_conv_like = False
    for layer in orig_model.layers:
        if isinstance(layer, layers.Conv2D):
            W, b = layer.get_weights()
            orig_out = W.shape[-1]
            mask = masks.get(layer.name, np.ones(orig_out, dtype=np.float32))
            keep_idx = np.where(mask == 1)[0]
            if keep_idx.size == 0:
                keep_idx = np.array([int(np.argmax(mask))], dtype=np.int32)
            W_new = W[:, :, :, keep_idx]
            b_new = b[keep_idx]
            new_conv = layers.Conv2D(
                filters=len(keep_idx),
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=layer.activation,
                use_bias=layer.use_bias,
                name=layer.name + "_pruned"
            )
            new_layers.append((new_conv, [W_new, b_new]))
            prev_was_conv_like = True
            continue
        if isinstance(layer, layers.BatchNormalization):
            try:
                weights = layer.get_weights()
                if prev_was_conv_like and new_layers:
                    prev_layer_obj, prev_w = new_layers[-1]
                    if prev_w is not None:
                        out_ch = prev_w[0].shape[-1]
                        gamma, beta, mean, var = weights
                        gamma = gamma[:out_ch]
                        beta = beta[:out_ch]
                        mean = mean[:out_ch]
                        var = var[:out_ch]
                        new_bn = layers.BatchNormalization.from_config(layer.get_config())
                        new_layers.append((new_bn, [gamma, beta, mean, var]))
                        continue
            except Exception:
                pass
            try:
                new_bn = layers.BatchNormalization.from_config(layer.get_config())
                new_layers.append((new_bn, layer.get_weights()))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            continue
        if isinstance(layer, (layers.MaxPooling2D, layers.Activation, layers.ReLU, layers.Dropout, layers.Flatten, layers.GlobalAveragePooling2D)):
            try:
                cloned = layer.__class__.from_config(layer.get_config())
                w = layer.get_weights() if hasattr(layer, "get_weights") else None
                new_layers.append((cloned, w if w else None))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            if isinstance(layer, layers.Flatten) or isinstance(layer, layers.GlobalAveragePooling2D):
                prev_was_conv_like = True
            continue
        if isinstance(layer, layers.Dense):
            W, b = layer.get_weights()
            out_mask = masks.get(layer.name, np.ones(W.shape[1], dtype=np.float32))
            out_idx = np.where(out_mask == 1)[0]
            if out_idx.size == 0:
                out_idx = np.array([int(np.argmax(out_mask))], dtype=np.int32)
            W_new = W[:, out_idx]
            b_new = b[out_idx]
            new_dense = layers.Dense(units=W_new.shape[1], activation=layer.activation, name=layer.name + "_pruned")
            new_layers.append((new_dense, [W_new, b_new]))
            prev_was_conv_like = False
            continue
        try:
            cloned = layer.__class__.from_config(layer.get_config())
            w = layer.get_weights() if hasattr(layer, "get_weights") else None
            new_layers.append((cloned, w if w else None))
        except Exception:
            new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
        prev_was_conv_like = False
    seq = models.Sequential(name=orig_model.name + "_struct_pruned")
    seq.add(layers.InputLayer(input_shape=tuple(input_shape)))
    for lyr_obj, w in new_layers:
        seq.add(lyr_obj)
        if w is not None:
            try:
                seq.layers[-1].set_weights(w)
            except Exception as e:
                log("Warning: couldn't set weights for", seq.layers[-1].name, ":", e)
    log("Structural pruning complete. New model summary:")
    seq.summary()
    return seq

# ---------------------------
# FLOPS and timing helpers
# ---------------------------
def calculate_conv_flops(input_shape, kernel_shape, strides=(1,1), padding='same'):
    h_in, w_in, c_in = input_shape
    kh, kw, _, c_out = kernel_shape
    if padding == 'same':
        h_out = math.ceil(h_in / strides[0])
        w_out = math.ceil(w_in / strides[1])
    else:
        h_out = math.ceil((h_in - kh + 1) / strides[0])
        w_out = math.ceil((w_in - kw + 1) / strides[1])
    flops = h_out * w_out * (kh * kw * c_in) * c_out * 2
    return flops, (h_out, w_out, c_out)

def calculate_dense_flops(in_size, out_size):
    return in_size * out_size * 2

def calculate_model_flops(model, input_shape):
    total = 0
    current_shape = tuple(input_shape)
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            weights = layer.get_weights()
            if not weights:
                continue
            kernel_shape = weights[0].shape
            layer_flops, current_shape = calculate_conv_flops(current_shape, kernel_shape, strides=layer.strides, padding=layer.padding)
            total += layer_flops
        elif isinstance(layer, layers.Flatten):
            current_shape = (int(np.prod(current_shape)),)
        elif isinstance(layer, layers.Dense):
            in_size = current_shape[0] if isinstance(current_shape, tuple) and len(current_shape)>0 else int(current_shape)
            layer_flops = calculate_dense_flops(in_size, layer.units)
            total += layer_flops
            current_shape = (layer.units,)
        elif isinstance(layer, layers.MaxPooling2D):
            h,w,c = current_shape
            pool = layer.pool_size[0] if hasattr(layer.pool_size, "__getitem__") else layer.pool_size
            current_shape = (h//pool, w//pool, c)
        else:
            pass
    return total

def measure_inference_time(model, sample_batch, steps=20):
    model.predict(sample_batch, verbose=0)
    t0 = time.time()
    for _ in range(steps):
        model.predict(sample_batch, verbose=0)
    t1 = time.time()
    return (t1 - t0) / steps

# ---------------------------
# Save helpers
# ---------------------------
def save_masks(masks, path):
    serial = {k: v.tolist() for k,v in masks.items()}
    with open(path, "w") as f:
        json.dump(serial, f, indent=2)
    log("Saved masks to", path)

def save_model_safe(model, path_no_ext):
    # choose extension from global config
    ext = MODEL_SAVE_EXT if MODEL_SAVE_EXT.startswith(".") else "." + MODEL_SAVE_EXT
    fullpath = path_no_ext + ext
    try:
        model.save(fullpath)
        log("Saved model to", fullpath)
    except Exception as e:
        log("Failed to save model to", fullpath, ":", e)

# ---------------------------
# Plot helper
# ---------------------------
def plot_mask_histograms(masks, outdir=SAVE_DIR):
    if not PLOT_RESULTS:
        return
    for name, mask in masks.items():
        plt.figure(figsize=(5,2))
        plt.title(name)
        plt.hist(mask, bins=2)
        plt.xlabel("0=pruned, 1=kept")
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"mask_{name}.png"))
        plt.close()

# ---------------------------
# Per-layer stat helper
# ---------------------------
def per_layer_remaining_stats(masks):
    stats = {}
    for name, mask in masks.items():
        total = len(mask)
        kept = int(mask.sum())
        stats[name] = {"kept": kept, "total": total, "ratio": kept/total}
    return stats

# ---------------------------
# MAIN pipeline
# ---------------------------
def full_pipeline(model_path, dataset_path):
    model = safe_load_model(model_path)
    log("Model loaded. Summary:")
    model.summary()

    # compile original for evaluation
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    input_shape = model.input_shape[1:]
    log("Inferred input shape:", input_shape)

    train_ds, val_ds = load_image_folder_dataset(dataset_path, image_size=input_shape[:2], batch_size=BATCH_SIZE)
    calib_ds = train_ds.take(CALIB_BATCHES)

    try:
        sample_x, _ = next(iter(val_ds))
    except Exception:
        sample_x, _ = next(iter(train_ds))
    sample_x_small = sample_x[:min(16, sample_x.shape[0])]

    dense_layers = [lyr for lyr in model.layers if isinstance(lyr, layers.Dense)]
    last_dense = dense_layers[-1] if dense_layers else None

    prune_layer_names = []
    for lyr in model.layers:
        if isinstance(lyr, layers.Conv2D):
            prune_layer_names.append(lyr.name)
        elif isinstance(lyr, layers.Dense) and lyr is not last_dense:
            prune_layer_names.append(lyr.name)
    log("Layers to prune:", prune_layer_names)

    stats = compute_activation_grad_stats(model, prune_layer_names, calib_ds, max_batches=CALIB_BATCHES)
    score_map = compute_importance_scores(stats)
    masks = make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO)
    save_masks(masks, os.path.join(SAVE_DIR, "masks.json"))
    plot_mask_histograms(masks)

    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    try:
        loss0, acc0 = masked_model.evaluate(val_ds.take(5), verbose=0)
        log("Masked model pre-FT acc:", acc0)
    except Exception as e:
        log("Masked model pre-eval failed:", e)

    baseline_flops = calculate_model_flops(model, input_shape)
    baseline_gflops = baseline_flops / 1e9
    baseline_time = measure_inference_time(model, sample_x_small, steps=10)
    log(f"Baseline FLOPS: {baseline_flops:,} ({baseline_gflops:.6f} GFLOPS), baseline time (avg batch): {baseline_time:.4f}s")

    try:
        masked_model.fit(train_ds.take(FT_BATCHES_TO_USE), validation_data=val_ds.take(5), epochs=FT_EPOCHS, verbose=2)
    except Exception as e:
        log("Masked fine-tune failed/partial:", e)

    pruned_model = prune_structural_sequential(model, masks, input_shape)
    pruned_model.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    log("Evaluating Original model:")
    orig_loss, orig_acc = model.evaluate(val_ds, verbose=0)
    log("Original acc:", orig_acc)

    log("Evaluating Masked model (after FT):")
    try:
        mask_loss, mask_acc = masked_model.evaluate(val_ds, verbose=0)
        log("Masked acc:", mask_acc)
    except Exception as e:
        log("Masked evaluate failed:", e)
        mask_acc = None

    log("Evaluating Pruned model:")
    try:
        pruned_loss, pruned_acc = pruned_model.evaluate(val_ds, verbose=0)
        log("Pruned acc:", pruned_acc)
    except Exception as e:
        log("Pruned evaluate failed:", e)
        pruned_acc = None

    pruned_flops = calculate_model_flops(pruned_model, input_shape)
    pruned_gflops = pruned_flops / 1e9
    pruned_time = measure_inference_time(pruned_model, sample_x_small, steps=10)
    log(f"Pruned FLOPS: {pruned_flops:,} ({pruned_gflops:.6f} GFLOPS), pruned time: {pruned_time:.4f}s")

    # Metrics: speedup, accuracy drop, parameter reduction
    speedup = (baseline_time / pruned_time) if pruned_time > 0 else float("inf")
    acc_drop_abs = (orig_acc - pruned_acc) if (orig_acc is not None and pruned_acc is not None) else None
    acc_drop_pct = (acc_drop_abs / orig_acc * 100) if (acc_drop_abs is not None and orig_acc != 0) else None

    orig_params = model.count_params()
    pruned_params = pruned_model.count_params()
    param_reduction_pct = (1 - pruned_params / orig_params) * 100 if orig_params > 0 else None

    # per-layer remaining stats
    layer_stats = per_layer_remaining_stats(masks)

    # Pretty print summary
    log("="*70)
    log("FINAL SUMMARY")
    log("="*70)
    log(f"Baseline GFLOPS: {baseline_gflops:.6f} GFLOPS")
    log(f"Pruned GFLOPS:   {pruned_gflops:.6f} GFLOPS")
    log(f"FLOPS reduction: {(1 - pruned_flops / baseline_flops) * 100:.2f}%")
    log(f"Baseline time (avg batch): {baseline_time:.4f}s")
    log(f"Pruned time (avg batch):   {pruned_time:.4f}s")
    log(f"Speed-up ratio (baseline/pruned): {speedup:.3f}x")
    if acc_drop_abs is not None:
        log(f"Original acc: {orig_acc:.4f}, Pruned acc: {pruned_acc:.4f}")
        log(f"Accuracy drop: {acc_drop_abs:.4f} ({acc_drop_pct:.2f}%)")
    else:
        log("Accuracy numbers unavailable for drop computation.")
    log(f"Original params: {orig_params:,}, Pruned params: {pruned_params:,}")
    log(f"Parameter reduction: {param_reduction_pct:.2f}%")
    log("Per-layer remaining (kept/total):")
    for name, s in layer_stats.items():
        log(f"  {name}: {s['kept']}/{s['total']} ({s['ratio']:.2%})")
    log("="*70)

    # Save outputs (models + masks + metadata)
    try:
        save_model_safe(model, os.path.join(SAVE_DIR, "model_baseline"))
        save_model_safe(masked_model, os.path.join(SAVE_DIR, "model_masked"))
        save_model_safe(pruned_model, os.path.join(SAVE_DIR, "model_pruned"))
    except Exception as e:
        log("Save models failed:", e)

    metadata = {
        "baseline_flops": int(baseline_flops),
        "pruned_flops": int(pruned_flops),
        "baseline_gflops": float(baseline_gflops),
        "pruned_gflops": float(pruned_gflops),
        "baseline_time": float(baseline_time),
        "pruned_time": float(pruned_time),
        "orig_acc": float(orig_acc) if orig_acc is not None else None,
        "pruned_acc": float(pruned_acc) if pruned_acc is not None else None,
        "mask_acc": float(mask_acc) if mask_acc is not None else None,
        "orig_params": int(orig_params),
        "pruned_params": int(pruned_params),
        "param_reduction_pct": float(param_reduction_pct) if param_reduction_pct is not None else None,
        "per_layer_stats": layer_stats
    }
    with open(os.path.join(SAVE_DIR, "prune_summary.json"), "w") as f:
        json.dump(metadata, f, indent=2)

    log("Saved metadata to", os.path.join(SAVE_DIR, "prune_summary.json"))
    return metadata

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    out = full_pipeline(MODEL_PATH, DATASET_PATH)
    log("Pipeline finished. Keys saved in metadata:", out.keys())


[INFO] Loading model: D:\college\sem-8\models\garbage_cnn_model.h5
[INFO] Loaded model directly.
[INFO] Model loaded. Summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,564,998 (36.49 MB)

 Trainable params: 9,564,998 (36.49 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8_pruned (Conv2D)        │ (None, 148, 148, 22)   │           616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 74, 74, 22)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9_pruned (Conv2D)        │ (None, 72, 72, 44)     │         8,756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 44)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10_pruned (Conv2D)       │ (None, 34, 34, 89)     │        35,333 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 89)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25721)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4_pruned (Dense)          │ (None, 179)            │     4,604,238 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 179)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5_pruned (Dense)          │ (None, 6)              │         1,080 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,650,023 (17.74 MB)

 Trainable params: 4,650,023 (17.74 MB)

 Non-trainable params: 0 (0.00 B)

[INFO] Evaluating Original model:
[INFO] Original acc: 0.7947019934654236
[INFO] Evaluating Masked model (after FT):
[INFO] Masked acc: 0.7880794405937195
[INFO] Evaluating Pruned model:
[INFO] Pruned acc: 0.13907285034656525
[INFO] Pruned FLOPS: 207,042,362 (0.207042 GFLOPS), pruned time: 0.1718s
[INFO] ======================================================================
[INFO] FINAL SUMMARY
[INFO] ======================================================================
[INFO] Baseline GFLOPS: 0.418355 GFLOPS
[INFO] Pruned GFLOPS:   0.207042 GFLOPS
[INFO] FLOPS reduction: 50.51%
[INFO] Baseline time (avg batch): 0.2519s
[INFO] Pruned time (avg batch):   0.1718s
[INFO] Speed-up ratio (baseline/pruned): 1.466x
[INFO] Original acc: 0.7947, Pruned acc: 0.1391
[INFO] Accuracy drop: 0.6556 (82.50%)
[INFO] Original params: 9,564,998, Pruned params: 4,650,023
[INFO] Parameter reduction: 51.39%
[INFO] Per-layer remaining (kept/total):
[INFO]   conv2d_8: 22/32 (68.75%)
[INFO]   conv2d_9: 44/64 

# **CNN Structure Pruning with 90% keep ratio**


In [3]:
"""
sequential_cnn_pruning_full.py
Full Sequential CNN pruning pipeline (single-file)

- Loads user Sequential CNN (.h5 / .keras / SavedModel)
- Sanitizes layer names in .h5 if they contain '/'
- Loads folder dataset (image_dataset_from_directory)
- Computes activation/gradient/variance importance
- Creates masks (keep_ratio)
- Builds masked model (gating layer)
- Fine-tunes masked model
- Structurally prunes model (conv filter removal, prune Dense outputs only)
- Computes FLOPS & timings, evaluates models
- Saves models and masks

Note: This is intended for Sequential CNN models (Conv2D -> ... -> Flatten -> Dense -> ... -> Dense output).
"""

import os
import json
import math
import time
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# ---------------------------
# USER CONFIG
# ---------------------------
MODEL_PATH = r"D:\college\sem-8\models\flower_cnn_model.h5"   # path to model (.h5/.keras or SavedModel dir)
DATASET_PATH = r"D:\college\sem-8\dataset\flower_photos"      # folder with subfolders per class
SAVE_DIR = r"pruning_output"                                 # where outputs are saved

KEEP_RATIO = 0.9            # fraction of channels/units to keep
ALPHA, BETA, GAMMA = 0.4, 0.3, 0.3  # importance weights
BATCH_SIZE = 64
CALIB_BATCHES = 30
FT_EPOCHS = 3
FT_BATCHES_TO_USE = 150
PLOT_RESULTS = True
VERBOSE = True

os.makedirs(SAVE_DIR, exist_ok=True)
tf.random.set_seed(42)
np.random.seed(42)

def log(*args):
    if VERBOSE:
        print("[INFO]", *args)

# ---------------------------
# Safe load model (handles '/' in h5 layer names)
# ---------------------------
def safe_load_model(model_path):
    """
    Try normal load_model; if fails (e.g., '/' in layer names in H5), sanitize model_config JSON and rebuild.
    Returns a Keras model (compiled=False).
    """
    log("Loading model:", model_path)
    # Try direct load first
    try:
        m = tf.keras.models.load_model(model_path, compile=False)
        log("Loaded model directly.")
        return m
    except Exception as e:
        log("Direct load failed:", e)

    # If HDF5, attempt to sanitize layer names in model_config
    try:
        with h5py.File(model_path, "r") as f:
            if "model_config" in f:
                raw = f["model_config"][()]
                if isinstance(raw, bytes):
                    raw = raw.decode("utf-8")
                cfg_json = json.loads(raw)
                changed = False
                for layer in cfg_json.get("config", {}).get("layers", []):
                    cfg = layer.get("config", {})
                    name = cfg.get("name")
                    if isinstance(name, str) and "/" in name:
                        new_name = name.replace("/", "_")
                        cfg["name"] = new_name
                        changed = True
                        log(f"[FIX] layer name: {name} -> {new_name}")
                if changed:
                    fixed_json = json.dumps(cfg_json)
                    model = tf.keras.models.model_from_json(fixed_json)
                    model.load_weights(model_path)
                    log("Loaded model from sanitized JSON + weights.")
                    return model
    except Exception as e2:
        log("H5 sanitization attempt failed:", e2)

    # final fallback: try load_model with safe_mode=False (older TF)
    try:
        m = tf.keras.models.load_model(model_path, compile=False, safe_mode=False)
        log("Loaded model with safe_mode=False.")
        return m
    except Exception as e3:
        log("All load attempts failed:", e3)
        raise RuntimeError("Failed to load model. Ensure path and format are correct.")

# ---------------------------
# Dataset loader (folder-based)
# ---------------------------
def load_image_folder_dataset(path, image_size, batch_size=BATCH_SIZE):
    log("Loading dataset folder:", path, "image_size:", image_size)
    train_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    rescaler = layers.Rescaling(1.0 / 255)
    train_ds = train_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds

# ---------------------------
# Activation & gradient stats
# ---------------------------
def compute_activation_grad_stats(model, layer_names, dataset, max_batches=CALIB_BATCHES):
    """
    For each layer in layer_names, compute:
      - A: mean(abs(activation)) per filter/unit
      - G: mean(abs(grad wrt activation)) per filter/unit
      - V: variance(activation) per filter/unit
    Returns: dict name -> (A, G, V)
    """
    log("Computing activation & gradient stats...")
    results = {n: [] for n in layer_names}
    grad_results = {n: [] for n in layer_names}
    var_results = {n: [] for n in layer_names}

    batch_count = 0
    for x_batch, y_batch in dataset:
        if batch_count >= max_batches:
            break
        batch_count += 1
        layer_acts = {}
        with tf.GradientTape(persistent=True) as tape:
            a = x_batch
            for layer in model.layers:
                a = layer(a)
                if layer.name in layer_names:
                    tape.watch(a)
                    layer_acts[layer.name] = a
            preds = a
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, preds)
            loss = tf.reduce_mean(loss)

        for name in layer_names:
            a = layer_acts[name]
            if len(a.shape) == 4:
                A = tf.reduce_mean(tf.abs(a), axis=(0,1,2)).numpy()
                V = tf.math.reduce_variance(a, axis=(0,1,2)).numpy()
            else:
                A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
                V = tf.math.reduce_variance(a, axis=0).numpy()

            grad = tape.gradient(loss, a)
            if grad is None:
                G = np.zeros_like(A)
            else:
                if len(grad.shape) == 4:
                    G = tf.reduce_mean(tf.abs(grad), axis=(0,1,2)).numpy()
                else:
                    G = tf.reduce_mean(tf.abs(grad), axis=0).numpy()

            results[name].append(A)
            var_results[name].append(V)
            grad_results[name].append(G)
        del tape

    stats = {}
    for name in layer_names:
        A = np.mean(results[name], axis=0)
        V = np.mean(var_results[name], axis=0)
        G = np.mean(grad_results[name], axis=0)
        stats[name] = (A, G, V)
        log(f"{name}: len={len(A)} meanA={A.mean():.6e} meanG={G.mean():.6e} meanV={V.mean():.6e}")
    return stats

# ---------------------------
# Importance scores & masks
# ---------------------------
def compute_importance_scores(stats, alpha=ALPHA, beta=BETA, gamma=GAMMA):
    def normalize(x):
        x = x - x.min()
        if x.max() > 0:
            x = x / x.max()
        return x
    scores = {}
    for name, (A, G, V) in stats.items():
        scores[name] = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)
    return scores

def make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO):
    masks = {}
    for name, score in score_map.items():
        k = max(1, int(len(score) * keep_ratio))
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)
        masks[name] = mask
        log(f"{name}: keep {int(mask.sum())}/{len(mask)}")
    return masks

# ---------------------------
# Mask gate layer & masked model
# ---------------------------
class CNNGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        self.mask = tf.constant(mask, dtype=tf.float32) if mask is not None else None
    def call(self, x):
        if self.mask is None:
            return x
        if len(x.shape) == 4:
            return x * tf.reshape(self.mask, (1,1,1,-1))
        else:
            return x * self.mask

def build_masked_model(orig_model, masks):
    log("Building masked model with gates...")
    inp = tf.keras.Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = CNNGate(mask=masks[layer.name], name=layer.name + "_gate")(x)
    masked = tf.keras.Model(inputs=inp, outputs=x, name=orig_model.name + "_masked")
    # copy weights (best-effort)
    for layer in orig_model.layers:
        try:
            masked_layer = masked.get_layer(layer.name)
            masked_layer.set_weights(layer.get_weights())
        except Exception:
            # layer might not exist by same name (e.g., InputLayer) -> ignore
            pass
    return masked

# ---------------------------
# Structural pruning (safe)
# ---------------------------
def prune_structural_sequential(orig_model, masks, input_shape):
    """
    Structural pruning for Sequential models.
    - Prune Conv2D output filters using masks[layer.name]
    - Prune Dense output units only (do not slice Dense input rows that come from Flatten/Conv)
    - Attempt to slice BatchNorm params to match conv outputs
    """
    log("Structural pruning (safe) start...")
    new_layers = []
    prev_was_conv_like = False  # indicates that Flatten/Conv preceded Dense inputs

    for layer in orig_model.layers:
        # Conv2D: slice output channels
        if isinstance(layer, layers.Conv2D):
            W, b = layer.get_weights()
            orig_out = W.shape[-1]
            mask = masks.get(layer.name, np.ones(orig_out, dtype=np.float32))
            keep_idx = np.where(mask == 1)[0]
            if keep_idx.size == 0:
                keep_idx = np.array([int(np.argmax(mask))], dtype=np.int32)
            W_new = W[:, :, :, keep_idx]
            b_new = b[keep_idx]
            new_conv = layers.Conv2D(
                filters=len(keep_idx),
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=layer.activation,
                use_bias=layer.use_bias,
                name=layer.name + "_pruned"
            )
            new_layers.append((new_conv, [W_new, b_new]))
            prev_was_conv_like = True
            continue

        # BatchNorm: slice params if prev was conv-like
        if isinstance(layer, layers.BatchNormalization):
            try:
                weights = layer.get_weights()
                if prev_was_conv_like and new_layers:
                    prev_layer_obj, prev_w = new_layers[-1]
                    if prev_w is not None:
                        out_ch = prev_w[0].shape[-1]  # kernel last dim
                        gamma, beta, mean, var = weights
                        gamma = gamma[:out_ch]
                        beta = beta[:out_ch]
                        mean = mean[:out_ch]
                        var = var[:out_ch]
                        new_bn = layers.BatchNormalization.from_config(layer.get_config())
                        new_layers.append((new_bn, [gamma, beta, mean, var]))
                        continue
            except Exception:
                pass
            # fallback keep BN as-is
            try:
                new_bn = layers.BatchNormalization.from_config(layer.get_config())
                new_layers.append((new_bn, layer.get_weights()))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            # prev_was_conv_like unchanged
            continue

        # MaxPool/Activation/Dropout/Flatten/GlobalAvgPool: clone or reuse
        if isinstance(layer, (layers.MaxPooling2D, layers.Activation, layers.ReLU, layers.Dropout, layers.Flatten, layers.GlobalAveragePooling2D)):
            try:
                cloned = layer.__class__.from_config(layer.get_config())
                w = layer.get_weights() if hasattr(layer, "get_weights") else None
                new_layers.append((cloned, w if w else None))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            if isinstance(layer, layers.Flatten) or isinstance(layer, layers.GlobalAveragePooling2D):
                prev_was_conv_like = True
            continue

        # Dense: prune outputs only (safe)
        if isinstance(layer, layers.Dense):
            W, b = layer.get_weights()  # shape (in_dim, out_dim)
            out_mask = masks.get(layer.name, np.ones(W.shape[1], dtype=np.float32))
            out_idx = np.where(out_mask == 1)[0]
            if out_idx.size == 0:
                out_idx = np.array([int(np.argmax(out_mask))], dtype=np.int32)
            W_new = W[:, out_idx]   # keep all input rows (safe)
            b_new = b[out_idx]
            new_dense = layers.Dense(units=W_new.shape[1], activation=layer.activation, name=layer.name + "_pruned")
            new_layers.append((new_dense, [W_new, b_new]))
            prev_was_conv_like = False
            continue

        # Fallback for other layers
        try:
            cloned = layer.__class__.from_config(layer.get_config())
            w = layer.get_weights() if hasattr(layer, "get_weights") else None
            new_layers.append((cloned, w if w else None))
        except Exception:
            new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
        prev_was_conv_like = False

    # Build new Sequential model with InputLayer
    seq = models.Sequential(name=orig_model.name + "_struct_pruned")
    seq.add(layers.InputLayer(input_shape=tuple(input_shape)))
    for lyr_obj, w in new_layers:
        seq.add(lyr_obj)
        if w is not None:
            try:
                seq.layers[-1].set_weights(w)
            except Exception as e:
                log("Warning: couldn't set weights for", seq.layers[-1].name, ":", e)
    log("Structural pruning complete. New model summary:")
    seq.summary()
    return seq

# ---------------------------
# FLOPS and timing helpers
# ---------------------------
def calculate_conv_flops(input_shape, kernel_shape, strides=(1,1), padding='same'):
    h_in, w_in, c_in = input_shape
    kh, kw, _, c_out = kernel_shape
    if padding == 'same':
        h_out = math.ceil(h_in / strides[0])
        w_out = math.ceil(w_in / strides[1])
    else:
        h_out = math.ceil((h_in - kh + 1) / strides[0])
        w_out = math.ceil((w_in - kw + 1) / strides[1])
    flops = h_out * w_out * (kh * kw * c_in) * c_out * 2
    return flops, (h_out, w_out, c_out)

def calculate_dense_flops(in_size, out_size):
    return in_size * out_size * 2

def calculate_model_flops(model, input_shape):
    total = 0
    current_shape = tuple(input_shape)
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            weights = layer.get_weights()
            if not weights:
                continue
            kernel_shape = weights[0].shape  # (kh, kw, in_c, out_c)
            layer_flops, current_shape = calculate_conv_flops(current_shape, kernel_shape, strides=layer.strides, padding=layer.padding)
            total += layer_flops
        elif isinstance(layer, layers.Flatten):
            current_shape = (int(np.prod(current_shape)),)
        elif isinstance(layer, layers.Dense):
            in_size = current_shape[0] if isinstance(current_shape, tuple) and len(current_shape)>0 else int(current_shape)
            layer_flops = calculate_dense_flops(in_size, layer.units)
            total += layer_flops
            current_shape = (layer.units,)
        elif isinstance(layer, layers.MaxPooling2D):
            h,w,c = current_shape
            pool = layer.pool_size[0] if hasattr(layer.pool_size, "__getitem__") else layer.pool_size
            current_shape = (h//pool, w//pool, c)
        else:
            # ignore other layers for shape changes
            pass
    return total

def measure_inference_time(model, sample_batch, steps=20):
    model.predict(sample_batch, verbose=0)  # warmup
    t0 = time.time()
    for _ in range(steps):
        model.predict(sample_batch, verbose=0)
    t1 = time.time()
    return (t1 - t0) / steps

# ---------------------------
# Save masks helper
# ---------------------------
def save_masks(masks, path):
    serial = {k: v.tolist() for k,v in masks.items()}
    with open(path, "w") as f:
        json.dump(serial, f, indent=2)
    log("Saved masks to", path)

# ---------------------------
# Plot helper
# ---------------------------
def plot_mask_histograms(masks, outdir=SAVE_DIR):
    if not PLOT_RESULTS:
        return
    for name, mask in masks.items():
        plt.figure(figsize=(5,2))
        plt.title(name)
        plt.hist(mask, bins=2)
        plt.xlabel("0=pruned, 1=kept")
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"mask_{name}.png"))
        plt.close()

# ---------------------------
# MAIN pipeline
# ---------------------------
def full_pipeline(model_path, dataset_path):
    # 1) load model (safe)
    model = safe_load_model(model_path)
    log("Model loaded. Summary:")
    model.summary()

    # compile original so evaluate works (use small lr default)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # 2) infer input shape
    input_shape = model.input_shape[1:]
    log("Inferred input shape:", input_shape)

    # 3) load dataset
    train_ds, val_ds = load_image_folder_dataset(dataset_path, image_size=input_shape[:2], batch_size=BATCH_SIZE)
    # calibration subset
    calib_ds = train_ds.take(CALIB_BATCHES)

    # sample for timing
    try:
        sample_x, _ = next(iter(val_ds))
    except Exception:
        sample_x, _ = next(iter(train_ds))
    sample_x_small = sample_x[:min(16, sample_x.shape[0])]

    # 4) choose layers to prune (conv + hidden dense only)
    dense_layers = [lyr for lyr in model.layers if isinstance(lyr, layers.Dense)]
    last_dense = dense_layers[-1] if dense_layers else None

    prune_layer_names = []
    for lyr in model.layers:
        if isinstance(lyr, layers.Conv2D):
            prune_layer_names.append(lyr.name)
        elif isinstance(lyr, layers.Dense) and lyr is not last_dense:
            prune_layer_names.append(lyr.name)
    log("Layers considered for pruning:", prune_layer_names)

    # 5) compute stats
    stats = compute_activation_grad_stats(model, prune_layer_names, calib_ds, max_batches=CALIB_BATCHES)

    # 6) importance & masks
    score_map = compute_importance_scores(stats)
    masks = make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO)
    save_masks(masks, os.path.join(SAVE_DIR, "masks.json"))
    plot_mask_histograms(masks)

    # 7) build masked model and compile
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    log("Masked model built.")

    # quick eval before FT
    try:
        loss0, acc0 = masked_model.evaluate(val_ds.take(5), verbose=0)
        log("Masked model pre-FT acc:", acc0)
    except Exception as e:
        log("Masked model pre-eval failed:", e)

    # 8) measure baseline flops & time
    baseline_flops = calculate_model_flops(model, input_shape)
    baseline_time = measure_inference_time(model, sample_x_small, steps=10)
    log(f"Baseline FLOPS: {baseline_flops:,}, baseline time (avg batch): {baseline_time:.4f}s")

    # 9) fine-tune masked model
    try:
        masked_model.fit(train_ds.take(FT_BATCHES_TO_USE), validation_data=val_ds.take(5), epochs=FT_EPOCHS, verbose=2)
    except Exception as e:
        log("Masked fine-tune failed/partial:", e)

    # 10) structural prune
    pruned_model = prune_structural_sequential(model, masks, input_shape)
    # compile pruned model for evaluation
    pruned_model.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # 11) evaluate models
    log("Evaluating Original model:")
    orig_loss, orig_acc = model.evaluate(val_ds, verbose=0)
    log("Original acc:", orig_acc)

    log("Evaluating Masked model (after FT):")
    try:
        mask_loss, mask_acc = masked_model.evaluate(val_ds, verbose=0)
        log("Masked acc:", mask_acc)
    except Exception as e:
        log("Masked evaluate failed:", e)
        mask_acc = None

    log("Evaluating Pruned model:")
    try:
        pruned_loss, pruned_acc = pruned_model.evaluate(val_ds, verbose=0)
        log("Pruned acc:", pruned_acc)
    except Exception as e:
        log("Pruned evaluate failed:", e)
        pruned_acc = None

    # 12) FLOPS & timing after prune
    pruned_flops = calculate_model_flops(pruned_model, input_shape)
    pruned_time = measure_inference_time(pruned_model, sample_x_small, steps=10)
    log(f"Pruned FLOPS: {pruned_flops:,}, pruned time: {pruned_time:.4f}s")

    # 13) summary & save
    reduction = 1.0 - (pruned_flops / baseline_flops) if baseline_flops > 0 else 0.0
    log("="*60)
    log("SUMMARY:")
    log(f"Baseline FLOPS: {baseline_flops:,}")
    log(f"Pruned FLOPS:   {pruned_flops:,}")
    log(f"FLOPS reduction: {reduction:.2%}")
    log(f"Original acc: {orig_acc}, Masked acc: {mask_acc}, Pruned acc: {pruned_acc}")
    log("="*60)

    # ---- Extra Metrics: GFLOPS + Accuracy Reduction ----
    baseline_gflops = baseline_flops / 1e9
    pruned_gflops = pruned_flops / 1e9
    gflops_reduction = 1.0 - (pruned_gflops / baseline_gflops) if baseline_gflops > 0 else 0.0

    acc_reduction = (orig_acc - pruned_acc) if (orig_acc is not None and pruned_acc is not None) else None

    log(f"Baseline GFLOPS: {baseline_gflops:.4f}")
    log(f"Pruned GFLOPS:   {pruned_gflops:.4f}")
    log(f"GFLOPS reduction: {gflops_reduction:.2%}")

    if acc_reduction is not None:
        log(f"Accuracy reduction: {acc_reduction:.4f}")
    else:
        log("Accuracy reduction: N/A")


    # save artifacts
    try:
        model.save(os.path.join(SAVE_DIR, "model_baseline"))
        masked_model.save(os.path.join(SAVE_DIR, "model_masked"))
        pruned_model.save(os.path.join(SAVE_DIR, "model_pruned"))
        log("Saved models to", SAVE_DIR)
    except Exception as e:
        log("Save models failed:", e)

    return {
        "model": model,
        "masked_model": masked_model,
        "pruned_model": pruned_model,
        "masks": masks,
        "baseline_flops": baseline_flops,
        "pruned_flops": pruned_flops,
        "baseline_time": baseline_time,
        "pruned_time": pruned_time,
        "orig_acc": orig_acc,
        "mask_acc": mask_acc,
        "pruned_acc": pruned_acc
    }

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    out = full_pipeline(MODEL_PATH, DATASET_PATH)
    log("Pipeline finished. Outputs:", out.keys())


[INFO] Loading model: D:\college\sem-8\models\flower_cnn_model.h5
[INFO] Loaded model directly.
[INFO] Model loaded. Summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 18, 18, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 7, 7, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536,645 (2.05 MB)

 Trainable params: 536,645 (2.05 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5_pruned (Conv2D)        │ (None, 178, 178, 28)   │           784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 89, 89, 28)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6_pruned (Conv2D)        │ (None, 87, 87, 57)     │        14,421 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 43, 43, 57)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7_pruned (Conv2D)        │ (None, 41, 41, 115)    │        59,110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 20, 20, 115)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8_pruned (Conv2D)        │ (None, 18, 18, 115)    │       119,140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 9, 9, 115)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9_pruned (Conv2D)        │ (None, 7, 7, 115)      │       119,140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 3, 3, 115)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1035)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8_pruned (Dense)          │ (None, 115)            │       119,140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 115)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9_pruned (Dense)          │ (None, 5)              │           580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 432,315 (1.65 MB)

 Trainable params: 432,315 (1.65 MB)

 Non-trainable params: 0 (0.00 B)

[INFO] Evaluating Original model:
[INFO] Original acc: 0.5435967445373535
[INFO] Evaluating Masked model (after FT):
[INFO] Masked acc: 0.5299727320671082
[INFO] Evaluating Pruned model:
[INFO] Pruned acc: 0.20572206377983093
[INFO] Pruned FLOPS: 552,721,480, pruned time: 0.2008s
[INFO] ============================================================
[INFO] SUMMARY:
[INFO] Baseline FLOPS: 691,945,472
[INFO] Pruned FLOPS:   552,721,480
[INFO] FLOPS reduction: 20.12%
[INFO] Original acc: 0.5435967445373535, Masked acc: 0.5299727320671082, Pruned acc: 0.20572206377983093
[INFO] ============================================================
[INFO] Baseline GFLOPS: 0.6919
[INFO] Pruned GFLOPS:   0.5527
[INFO] GFLOPS reduction: 20.12%
[INFO] Accuracy reduction: 0.3379
[INFO] Save models failed: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel 

# **CNN structure pruning with early call back with 100 epochs**

In [50]:
"""
sequential_cnn_pruning_full.py
Full Sequential CNN pruning pipeline (single-file)

- Loads user Sequential CNN (.h5 / .keras / SavedModel)
- Sanitizes layer names in .h5 if they contain '/'
- Loads folder dataset (image_dataset_from_directory)
- Computes activation/gradient/variance importance
- Creates masks (keep_ratio)
- Builds masked model (gating layer)
- Fine-tunes masked model
- Structurally prunes model (conv filter removal, prune Dense outputs only)
- Computes FLOPS & timings, evaluates models
- Saves models and masks

Note: This is intended for Sequential CNN models (Conv2D -> ... -> Flatten -> Dense -> ... -> Dense output).
"""

import os
import json
import math
import time
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# ---------------------------
# USER CONFIG
# ---------------------------
MODEL_PATH = r"D:\college\sem-8\models\cat_dog_cnn.h5"   # path to model (.h5/.keras or SavedModel dir)
DATASET_PATH = r"D:\college\sem-8\dataset\reduced_pet_dataset"      # folder with subfolders per class
SAVE_DIR = r"pruning_output"                                 # where outputs are saved

KEEP_RATIO = 0.9            # fraction of channels/units to keep
ALPHA, BETA, GAMMA = 0.4, 0.3, 0.3  # importance weights
BATCH_SIZE = 64
CALIB_BATCHES = 30
FT_EPOCHS = 3
FT_BATCHES_TO_USE = 150
PLOT_RESULTS = True
VERBOSE = True

os.makedirs(SAVE_DIR, exist_ok=True)
tf.random.set_seed(42)
np.random.seed(42)

def log(*args):
    if VERBOSE:
        print("[INFO]", *args)

# ---------------------------
# Safe load model (handles '/' in h5 layer names)
# ---------------------------
def safe_load_model(model_path):
    """
    Try normal load_model; if fails (e.g., '/' in layer names in H5), sanitize model_config JSON and rebuild.
    Returns a Keras model (compiled=False).
    """
    log("Loading model:", model_path)
    # Try direct load first
    try:
        m = tf.keras.models.load_model(model_path, compile=False)
        log("Loaded model directly.")
        return m
    except Exception as e:
        log("Direct load failed:", e)

    # If HDF5, attempt to sanitize layer names in model_config
    try:
        with h5py.File(model_path, "r") as f:
            if "model_config" in f:
                raw = f["model_config"][()]
                if isinstance(raw, bytes):
                    raw = raw.decode("utf-8")
                cfg_json = json.loads(raw)
                changed = False
                for layer in cfg_json.get("config", {}).get("layers", []):
                    cfg = layer.get("config", {})
                    name = cfg.get("name")
                    if isinstance(name, str) and "/" in name:
                        new_name = name.replace("/", "_")
                        cfg["name"] = new_name
                        changed = True
                        log(f"[FIX] layer name: {name} -> {new_name}")
                if changed:
                    fixed_json = json.dumps(cfg_json)
                    model = tf.keras.models.model_from_json(fixed_json)
                    model.load_weights(model_path)
                    log("Loaded model from sanitized JSON + weights.")
                    return model
    except Exception as e2:
        log("H5 sanitization attempt failed:", e2)

    # final fallback: try load_model with safe_mode=False (older TF)
    try:
        m = tf.keras.models.load_model(model_path, compile=False, safe_mode=False)
        log("Loaded model with safe_mode=False.")
        return m
    except Exception as e3:
        log("All load attempts failed:", e3)
        raise RuntimeError("Failed to load model. Ensure path and format are correct.")

# ---------------------------
# Dataset loader (folder-based)
# ---------------------------
def load_image_folder_dataset(path, image_size, batch_size=BATCH_SIZE):
    log("Loading dataset folder:", path, "image_size:", image_size)
    train_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    rescaler = layers.Rescaling(1.0 / 255)
    train_ds = train_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds

# ---------------------------
# Activation & gradient stats
# ---------------------------
def compute_activation_grad_stats(model, layer_names, dataset, max_batches=CALIB_BATCHES):
    """
    For each layer in layer_names, compute:
      - A: mean(abs(activation)) per filter/unit
      - G: mean(abs(grad wrt activation)) per filter/unit
      - V: variance(activation) per filter/unit
    Returns: dict name -> (A, G, V)
    """
    log("Computing activation & gradient stats...")
    results = {n: [] for n in layer_names}
    grad_results = {n: [] for n in layer_names}
    var_results = {n: [] for n in layer_names}

    batch_count = 0
    for x_batch, y_batch in dataset:
        if batch_count >= max_batches:
            break
        batch_count += 1
        layer_acts = {}
        with tf.GradientTape(persistent=True) as tape:
            a = x_batch
            for layer in model.layers:
                a = layer(a)
                if layer.name in layer_names:
                    tape.watch(a)
                    layer_acts[layer.name] = a
            preds = a
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, preds)
            loss = tf.reduce_mean(loss)

        for name in layer_names:
            a = layer_acts[name]
            if len(a.shape) == 4:
                A = tf.reduce_mean(tf.abs(a), axis=(0,1,2)).numpy()
                V = tf.math.reduce_variance(a, axis=(0,1,2)).numpy()
            else:
                A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
                V = tf.math.reduce_variance(a, axis=0).numpy()

            grad = tape.gradient(loss, a)
            if grad is None:
                G = np.zeros_like(A)
            else:
                if len(grad.shape) == 4:
                    G = tf.reduce_mean(tf.abs(grad), axis=(0,1,2)).numpy()
                else:
                    G = tf.reduce_mean(tf.abs(grad), axis=0).numpy()

            results[name].append(A)
            var_results[name].append(V)
            grad_results[name].append(G)
        del tape

    stats = {}
    for name in layer_names:
        A = np.mean(results[name], axis=0)
        V = np.mean(var_results[name], axis=0)
        G = np.mean(grad_results[name], axis=0)
        stats[name] = (A, G, V)
        log(f"{name}: len={len(A)} meanA={A.mean():.6e} meanG={G.mean():.6e} meanV={V.mean():.6e}")
    return stats

# ---------------------------
# Importance scores & masks
# ---------------------------
def compute_importance_scores(stats, alpha=ALPHA, beta=BETA, gamma=GAMMA):
    def normalize(x):
        x = x - x.min()
        if x.max() > 0:
            x = x / x.max()
        return x
    scores = {}
    for name, (A, G, V) in stats.items():
        scores[name] = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)
    return scores

def make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO):
    masks = {}
    for name, score in score_map.items():
        k = max(1, int(len(score) * keep_ratio))
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)
        masks[name] = mask
        log(f"{name}: keep {int(mask.sum())}/{len(mask)}")
    return masks

# ---------------------------
# Mask gate layer & masked model
# ---------------------------
class CNNGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        self.mask = tf.constant(mask, dtype=tf.float32) if mask is not None else None
    def call(self, x):
        if self.mask is None:
            return x
        if len(x.shape) == 4:
            return x * tf.reshape(self.mask, (1,1,1,-1))
        else:
            return x * self.mask

def build_masked_model(orig_model, masks):
    log("Building masked model with gates...")
    inp = tf.keras.Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = CNNGate(mask=masks[layer.name], name=layer.name + "_gate")(x)
    masked = tf.keras.Model(inputs=inp, outputs=x, name=orig_model.name + "_masked")
    # copy weights (best-effort)
    for layer in orig_model.layers:
        try:
            masked_layer = masked.get_layer(layer.name)
            masked_layer.set_weights(layer.get_weights())
        except Exception:
            # layer might not exist by same name (e.g., InputLayer) -> ignore
            pass
    return masked

# ---------------------------
# Structural pruning (safe)
# ---------------------------
def prune_structural_sequential(orig_model, masks, input_shape):
    """
    Structural pruning for Sequential models.
    - Prune Conv2D output filters using masks[layer.name]
    - Prune Dense output units only (do not slice Dense input rows that come from Flatten/Conv)
    - Attempt to slice BatchNorm params to match conv outputs
    """
    log("Structural pruning (safe) start...")
    new_layers = []
    prev_was_conv_like = False  # indicates that Flatten/Conv preceded Dense inputs

    for layer in orig_model.layers:
        # Conv2D: slice output channels
        if isinstance(layer, layers.Conv2D):
            W, b = layer.get_weights()
            orig_out = W.shape[-1]
            mask = masks.get(layer.name, np.ones(orig_out, dtype=np.float32))
            keep_idx = np.where(mask == 1)[0]
            if keep_idx.size == 0:
                keep_idx = np.array([int(np.argmax(mask))], dtype=np.int32)
            W_new = W[:, :, :, keep_idx]
            b_new = b[keep_idx]
            new_conv = layers.Conv2D(
                filters=len(keep_idx),
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=layer.activation,
                use_bias=layer.use_bias,
                name=layer.name + "_pruned"
            )
            new_layers.append((new_conv, [W_new, b_new]))
            prev_was_conv_like = True
            continue

        # BatchNorm: slice params if prev was conv-like
        if isinstance(layer, layers.BatchNormalization):
            try:
                weights = layer.get_weights()
                if prev_was_conv_like and new_layers:
                    prev_layer_obj, prev_w = new_layers[-1]
                    if prev_w is not None:
                        out_ch = prev_w[0].shape[-1]  # kernel last dim
                        gamma, beta, mean, var = weights
                        gamma = gamma[:out_ch]
                        beta = beta[:out_ch]
                        mean = mean[:out_ch]
                        var = var[:out_ch]
                        new_bn = layers.BatchNormalization.from_config(layer.get_config())
                        new_layers.append((new_bn, [gamma, beta, mean, var]))
                        continue
            except Exception:
                pass
            # fallback keep BN as-is
            try:
                new_bn = layers.BatchNormalization.from_config(layer.get_config())
                new_layers.append((new_bn, layer.get_weights()))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            # prev_was_conv_like unchanged
            continue

        # MaxPool/Activation/Dropout/Flatten/GlobalAvgPool: clone or reuse
        if isinstance(layer, (layers.MaxPooling2D, layers.Activation, layers.ReLU, layers.Dropout, layers.Flatten, layers.GlobalAveragePooling2D)):
            try:
                cloned = layer.__class__.from_config(layer.get_config())
                w = layer.get_weights() if hasattr(layer, "get_weights") else None
                new_layers.append((cloned, w if w else None))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            if isinstance(layer, layers.Flatten) or isinstance(layer, layers.GlobalAveragePooling2D):
                prev_was_conv_like = True
            continue

        # Dense: prune outputs only (safe)
        if isinstance(layer, layers.Dense):
            W, b = layer.get_weights()  # shape (in_dim, out_dim)
            out_mask = masks.get(layer.name, np.ones(W.shape[1], dtype=np.float32))
            out_idx = np.where(out_mask == 1)[0]
            if out_idx.size == 0:
                out_idx = np.array([int(np.argmax(out_mask))], dtype=np.int32)
            W_new = W[:, out_idx]   # keep all input rows (safe)
            b_new = b[out_idx]
            new_dense = layers.Dense(units=W_new.shape[1], activation=layer.activation, name=layer.name + "_pruned")
            new_layers.append((new_dense, [W_new, b_new]))
            prev_was_conv_like = False
            continue

        # Fallback for other layers
        try:
            cloned = layer.__class__.from_config(layer.get_config())
            w = layer.get_weights() if hasattr(layer, "get_weights") else None
            new_layers.append((cloned, w if w else None))
        except Exception:
            new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
        prev_was_conv_like = False

    # Build new Sequential model with InputLayer
    seq = models.Sequential(name=orig_model.name + "_struct_pruned")
    seq.add(layers.InputLayer(input_shape=tuple(input_shape)))
    for lyr_obj, w in new_layers:
        seq.add(lyr_obj)
        if w is not None:
            try:
                seq.layers[-1].set_weights(w)
            except Exception as e:
                log("Warning: couldn't set weights for", seq.layers[-1].name, ":", e)
    log("Structural pruning complete. New model summary:")
    seq.summary()
    return seq

# ---------------------------
# FLOPS and timing helpers
# ---------------------------
def calculate_conv_flops(input_shape, kernel_shape, strides=(1,1), padding='same'):
    h_in, w_in, c_in = input_shape
    kh, kw, _, c_out = kernel_shape
    if padding == 'same':
        h_out = math.ceil(h_in / strides[0])
        w_out = math.ceil(w_in / strides[1])
    else:
        h_out = math.ceil((h_in - kh + 1) / strides[0])
        w_out = math.ceil((w_in - kw + 1) / strides[1])
    flops = h_out * w_out * (kh * kw * c_in) * c_out * 2
    return flops, (h_out, w_out, c_out)

def calculate_dense_flops(in_size, out_size):
    return in_size * out_size * 2

def calculate_model_flops(model, input_shape):
    total = 0
    current_shape = tuple(input_shape)
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            weights = layer.get_weights()
            if not weights:
                continue
            kernel_shape = weights[0].shape  # (kh, kw, in_c, out_c)
            layer_flops, current_shape = calculate_conv_flops(current_shape, kernel_shape, strides=layer.strides, padding=layer.padding)
            total += layer_flops
        elif isinstance(layer, layers.Flatten):
            current_shape = (int(np.prod(current_shape)),)
        elif isinstance(layer, layers.Dense):
            in_size = current_shape[0] if isinstance(current_shape, tuple) and len(current_shape)>0 else int(current_shape)
            layer_flops = calculate_dense_flops(in_size, layer.units)
            total += layer_flops
            current_shape = (layer.units,)
        elif isinstance(layer, layers.MaxPooling2D):
            h,w,c = current_shape
            pool = layer.pool_size[0] if hasattr(layer.pool_size, "__getitem__") else layer.pool_size
            current_shape = (h//pool, w//pool, c)
        else:
            # ignore other layers for shape changes
            pass
    return total

def measure_inference_time(model, sample_batch, steps=20):
    model.predict(sample_batch, verbose=0)  # warmup
    t0 = time.time()
    for _ in range(steps):
        model.predict(sample_batch, verbose=0)
    t1 = time.time()
    return (t1 - t0) / steps

# ---------------------------
# Save masks helper
# ---------------------------
def save_masks(masks, path):
    serial = {k: v.tolist() for k,v in masks.items()}
    with open(path, "w") as f:
        json.dump(serial, f, indent=2)
    log("Saved masks to", path)

# ---------------------------
# Plot helper
# ---------------------------
def plot_mask_histograms(masks, outdir=SAVE_DIR):
    if not PLOT_RESULTS:
        return
    for name, mask in masks.items():
        plt.figure(figsize=(5,2))
        plt.title(name)
        plt.hist(mask, bins=2)
        plt.xlabel("0=pruned, 1=kept")
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"mask_{name}.png"))
        plt.close()

# ---------------------------
# MAIN pipeline
# ---------------------------
def full_pipeline(model_path, dataset_path):
    # 1) load model (safe)
    model = safe_load_model(model_path)
    log("Model loaded. Summary:")
    model.summary()

    # compile original so evaluate works (use small lr default)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # 2) infer input shape
    input_shape = model.input_shape[1:]
    log("Inferred input shape:", input_shape)

    # 3) load dataset
    train_ds, val_ds = load_image_folder_dataset(dataset_path, image_size=input_shape[:2], batch_size=BATCH_SIZE)
    # calibration subset
    calib_ds = train_ds.take(CALIB_BATCHES)

    # sample for timing
    try:
        sample_x, _ = next(iter(val_ds))
    except Exception:
        sample_x, _ = next(iter(train_ds))
    sample_x_small = sample_x[:min(16, sample_x.shape[0])]

    # 4) choose layers to prune (conv + hidden dense only)
    dense_layers = [lyr for lyr in model.layers if isinstance(lyr, layers.Dense)]
    last_dense = dense_layers[-1] if dense_layers else None

    prune_layer_names = []
    for lyr in model.layers:
        if isinstance(lyr, layers.Conv2D):
            prune_layer_names.append(lyr.name)
        elif isinstance(lyr, layers.Dense) and lyr is not last_dense:
            prune_layer_names.append(lyr.name)
    log("Layers considered for pruning:", prune_layer_names)

    # 5) compute stats
    stats = compute_activation_grad_stats(model, prune_layer_names, calib_ds, max_batches=CALIB_BATCHES)

    # 6) importance & masks
    score_map = compute_importance_scores(stats)
    masks = make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO)
    save_masks(masks, os.path.join(SAVE_DIR, "masks.json"))
    plot_mask_histograms(masks)

    # 7) build masked model and compile
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    log("Masked model built.")

    # quick eval before FT
    try:
        loss0, acc0 = masked_model.evaluate(val_ds.take(5), verbose=0)
        log("Masked model pre-FT acc:", acc0)
    except Exception as e:
        log("Masked model pre-eval failed:", e)

    # 8) measure baseline flops & time
    baseline_flops = calculate_model_flops(model, input_shape)
    baseline_time = measure_inference_time(model, sample_x_small, steps=10)
    log(f"Baseline FLOPS: {baseline_flops:,}, baseline time (avg batch): {baseline_time:.4f}s")

    # 9) fine-tune masked model
    try:
        masked_model.fit(train_ds.take(FT_BATCHES_TO_USE), validation_data=val_ds.take(5), epochs=FT_EPOCHS, verbose=2)
    except Exception as e:
        log("Masked fine-tune failed/partial:", e)

    # 10) structural prune
    pruned_model = prune_structural_sequential(model, masks, input_shape)
    pruned_model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    log("Starting fine-tuning of PRUNED model (100 epochs + EarlyStopping)...")

    early_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        restore_best_weights=True
    )

    try:
        pruned_model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100,
        batch_size=BATCH_SIZE,
        callbacks=[early_cb],
        verbose=2
        )
    except Exception as e:
        log("Pruned model fine-tune failed/partial:", e)


    # 11) evaluate models
    log("Evaluating Original model:")
    orig_loss, orig_acc = model.evaluate(val_ds, verbose=0)
    log("Original acc:", orig_acc)

    log("Evaluating Masked model (after FT):")
    try:
        mask_loss, mask_acc = masked_model.evaluate(val_ds, verbose=0)
        log("Masked acc:", mask_acc)
    except Exception as e:
        log("Masked evaluate failed:", e)
        mask_acc = None

    log("Evaluating Pruned model:")
    try:
        pruned_loss, pruned_acc = pruned_model.evaluate(val_ds, verbose=0)
        log("Pruned acc:", pruned_acc)
    except Exception as e:
        log("Pruned evaluate failed:", e)
        pruned_acc = None

    # 12) FLOPS & timing after prune
    pruned_flops = calculate_model_flops(pruned_model, input_shape)
    pruned_time = measure_inference_time(pruned_model, sample_x_small, steps=10)
    log(f"Pruned FLOPS: {pruned_flops:,}, pruned time: {pruned_time:.4f}s")

    # 13) summary & save
    reduction = 1.0 - (pruned_flops / baseline_flops) if baseline_flops > 0 else 0.0
    log("="*60)
    log("SUMMARY:")
    log(f"Baseline FLOPS: {baseline_flops:,}")
    log(f"Pruned FLOPS:   {pruned_flops:,}")
    log(f"FLOPS reduction: {reduction:.2%}")
    log(f"Original acc: {orig_acc}, Masked acc: {mask_acc}, Pruned acc: {pruned_acc}")
    log("="*60)

    # ---- Extra Metrics: GFLOPS + Accuracy Reduction ----
    baseline_gflops = baseline_flops / 1e9
    pruned_gflops = pruned_flops / 1e9
    gflops_reduction = 1.0 - (pruned_gflops / baseline_gflops) if baseline_gflops > 0 else 0.0

    acc_reduction = (orig_acc - pruned_acc) if (orig_acc is not None and pruned_acc is not None) else None

    log(f"Baseline GFLOPS: {baseline_gflops:.4f}")
    log(f"Pruned GFLOPS:   {pruned_gflops:.4f}")
    log(f"GFLOPS reduction: {gflops_reduction:.2%}")

    if acc_reduction is not None:
        log(f"Accuracy reduction: {acc_reduction:.4f}")
    else:
        log("Accuracy reduction: N/A")


    # save artifacts
    try:
        model.save(os.path.join(SAVE_DIR, "model_baseline"))
        masked_model.save(os.path.join(SAVE_DIR, "model_masked"))
        pruned_model.save(os.path.join(SAVE_DIR, "model_pruned"))
        log("Saved models to", SAVE_DIR)
    except Exception as e:
        log("Save models failed:", e)

    return {
        "model": model,
        "masked_model": masked_model,
        "pruned_model": pruned_model,
        "masks": masks,
        "baseline_flops": baseline_flops,
        "pruned_flops": pruned_flops,
        "baseline_time": baseline_time,
        "pruned_time": pruned_time,
        "orig_acc": orig_acc,
        "mask_acc": mask_acc,
        "pruned_acc": pruned_acc
    }

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    out = full_pipeline(MODEL_PATH, DATASET_PATH)
    log("Pipeline finished. Outputs:", out.keys())


[INFO] Loading model: D:\college\sem-8\models\cat_dog_cnn.h5
[INFO] Loaded model directly.
[INFO] Model loaded. Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 5, 5, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 409,345 (1.56 MB)

 Trainable params: 409,345 (1.56 MB)

 Non-trainable params: 0 (0.00 B)

[INFO] Inferred input shape: (150, 150, 3)
[INFO] Loading dataset folder: D:\college\sem-8\dataset\reduced_pet_dataset image_size: (150, 150)
Found 7498 files belonging to 2 classes.
Using 5999 files for training.
Found 7498 files belonging to 2 classes.
Using 1499 files for validation.
[INFO] Layers considered for pruning: ['conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'conv2d_4', 'dense']
[INFO] Computing activation & gradient stats...


InvalidArgumentError: {{function_node __wrapped__SparseSoftmaxCrossEntropyWithLogits_device_/job:localhost/replica:0/task:0/device:CPU:0}} Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 0 0 [Op:SparseSoftmaxCrossEntropyWithLogits] name: 

# **Main CNN with early call back for both multi as well as binary classification**

In [9]:
"""
sequential_cnn_pruning_full_fixed.py
Full Sequential CNN pruning pipeline (single-file)

- Loads user Sequential CNN (.h5/.keras / SavedModel)
- Sanitizes layer names in .h5 if they contain '/'
- Loads folder dataset (image_dataset_from_directory)
- Computes activation/gradient/variance importance
- Creates masks (keep_ratio)
- Builds masked model (gating layer)
- Fine-tunes masked model
- Structurally prunes model (conv filter removal, prune Dense outputs only)
- Computes FLOPS & timings, evaluates models
- Saves models and masks

Fixes:
- Ensures final layer matches dataset classes (automatically rebuilds final layer if needed)
- Uses appropriate loss function (binary vs sparse categorical) during stat collection and training
- Adds GFLOPS, accuracy reduction, inference timing
"""

import os
import json
import math
import time
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# ---------------------------
# USER CONFIG (edit paths)
# ---------------------------
MODEL_PATH = r"D:\college\sem-8\models\garbage_cnn_model.h5"   # path to model (.h5/.keras or SavedModel dir)
DATASET_PATH = r"D:\college\sem-8\dataset\Garbage classification\Garbage classification"      # folder with subfolders per class
SAVE_DIR = r"pruning_output"                                 # where outputs are saved
base_name = os.path.splitext(os.path.basename(MODEL_PATH))[0]


KEEP_RATIO = 0.83            # fraction of channels/units to keep
ALPHA, BETA, GAMMA = 0.4, 0.3, 0.3  # importance weights
BATCH_SIZE = 64
CALIB_BATCHES = 30
FT_EPOCHS = 3
FT_BATCHES_TO_USE = 150
PLOT_RESULTS = True
VERBOSE = True

os.makedirs(SAVE_DIR, exist_ok=True)
tf.random.set_seed(42)
np.random.seed(42)

def log(*args):
    if VERBOSE:
        print("[INFO]", *args)

# ---------------------------
# Safe load model (handles '/' in h5 layer names)
# ---------------------------
def safe_load_model(model_path):
    """
    Try normal load_model; if fails (e.g., '/' in layer names in H5), sanitize model_config JSON and rebuild.
    Returns a Keras model (compiled=False).
    """
    log("Loading model:", model_path)
    # Try direct load first
    try:
        m = tf.keras.models.load_model(model_path, compile=False)
        log("Loaded model directly.")
        return m
    except Exception as e:
        log("Direct load failed:", e)

    # If HDF5, attempt to sanitize layer names in model_config
    try:
        with h5py.File(model_path, "r") as f:
            if "model_config" in f:
                raw = f["model_config"][()]
                if isinstance(raw, bytes):
                    raw = raw.decode("utf-8")
                cfg_json = json.loads(raw)
                changed = False
                for layer in cfg_json.get("config", {}).get("layers", []):
                    cfg = layer.get("config", {})
                    name = cfg.get("name")
                    if isinstance(name, str) and "/" in name:
                        new_name = name.replace("/", "_")
                        cfg["name"] = new_name
                        changed = True
                        log(f"[FIX] layer name: {name} -> {new_name}")
                if changed:
                    fixed_json = json.dumps(cfg_json)
                    model = tf.keras.models.model_from_json(fixed_json)
                    model.load_weights(model_path)
                    log("Loaded model from sanitized JSON + weights.")
                    return model
    except Exception as e2:
        log("H5 sanitization attempt failed:", e2)

    # final fallback: try load_model with safe_mode=False (older TF)
    try:
        m = tf.keras.models.load_model(model_path, compile=False, safe_mode=False)
        log("Loaded model with safe_mode=False.")
        return m
    except Exception as e3:
        log("All load attempts failed:", e3)
        raise RuntimeError("Failed to load model. Ensure path and format are correct.")

# ---------------------------
# Rebuild final layer to match num_classes (safe, best-effort)
# ---------------------------
def ensure_output_matches_dataset(orig_model, num_classes):
    """
    If the model's current output shape doesn't match num_classes, rebuild final output
    to match. Works for Sequential-style networks. Returns new_model, loss_fn.
    """
    # Determine model's output dim
    out_shape = tuple(orig_model.output_shape) if orig_model.output_shape is not None else None
    # If already matches (and for multiclass softmax case), return original and appropriate loss.
    if out_shape is not None:
        if num_classes == 2 and (out_shape[-1] == 1 or out_shape[-1] == 2):
            # binary case: allow Dense(1) or Dense(2) (Dense(2) could be softmax but labels are 0/1)
            log("Model output seems compatible with binary classification.")
            loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False) if out_shape[-1] == 1 else tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
            return orig_model, loss_fn
        if num_classes > 2 and out_shape[-1] == num_classes:
            log("Model output matches dataset classes.")
            loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
            return orig_model, loss_fn

    # Need to rebuild final layer
    log("Rebuilding model final layer to match num_classes:", num_classes)
    # We'll clone all layers except the last one and then append a new final dense.
    # Use layer.from_config to avoid reusing layer objects in two models.
    new_seq = models.Sequential(name=(orig_model.name or "rebuilt_model") + "_rebuilt")
    # Add InputLayer
    input_shape = orig_model.input_shape[1:]
    new_seq.add(layers.InputLayer(input_shape=tuple(input_shape)))

    # Clone all layers except the last (we will replace last)
    # We'll attempt to copy weights for layers that remain identical
    for layer in orig_model.layers[:-1]:
        try:
            cfg = layer.get_config()
            Cls = layer.__class__
            cloned = Cls.from_config(cfg)
            new_seq.add(cloned)
            # set weights if possible and shapes match
            try:
                w = layer.get_weights()
                if w:
                    new_seq.layers[-1].set_weights(w)
            except Exception:
                # ignore weight copy failures
                pass
        except Exception:
            # fallback: try to append original layer (may cause errors but best-effort)
            try:
                new_seq.add(layer)
            except Exception:
                log("Warning: couldn't clone layer", layer.name, "- skipping weights copy.")

    # Add new final layer depending on num_classes
    if num_classes == 2:
        # binary: Dense(1, activation='sigmoid')
        new_seq.add(layers.Dense(1, activation="sigmoid", name="output_rebuilt"))
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    else:
        new_seq.add(layers.Dense(num_classes, activation="softmax", name="output_rebuilt"))
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

    # Try to compile minimal to ensure shape correctness (we'll let caller compile fully later)
    return new_seq, loss_fn

# ---------------------------
# Dataset loader (folder-based)
# ---------------------------
def load_image_folder_dataset(path, image_size, batch_size=BATCH_SIZE):
    log("Loading dataset folder:", path, "image_size:", image_size)
    train_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    rescaler = layers.Rescaling(1.0 / 255)
    train_ds = train_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds

# ---------------------------
# Activation & gradient stats (uses provided loss function)
# ---------------------------
def compute_activation_grad_stats(model, layer_names, dataset, loss_fn, max_batches=CALIB_BATCHES):
    """
    For each layer in layer_names, compute:
      - A: mean(abs(activation)) per filter/unit
      - G: mean(abs(grad wrt activation)) per filter/unit
      - V: variance(activation) per filter/unit
    Returns: dict name -> (A, G, V)
    """
    log("Computing activation & gradient stats...")
    results = {n: [] for n in layer_names}
    grad_results = {n: [] for n in layer_names}
    var_results = {n: [] for n in layer_names}

    batch_count = 0
    for x_batch, y_batch in dataset:
        if batch_count >= max_batches:
            break
        batch_count += 1
        layer_acts = {}
        with tf.GradientTape(persistent=True) as tape:
            a = x_batch
            for layer in model.layers:
                a = layer(a)
                if layer.name in layer_names:
                    tape.watch(a)
                    layer_acts[layer.name] = a
            preds = a
            # compute loss using provided loss_fn (works for both binary & sparse categorical)
            # When loss_fn is a Keras loss instance, call like a function
            # Ensure y_batch dtype is int for sparse CE, float for binary BCE
            try:
                loss_vals = loss_fn(y_batch, preds)
            except Exception:
                # fallback attempt: convert types for binary
                if isinstance(loss_fn, tf.keras.losses.BinaryCrossentropy):
                    loss_vals = loss_fn(tf.cast(y_batch, tf.float32), preds)
                else:
                    loss_vals = loss_fn(y_batch, preds)
            loss = tf.reduce_mean(loss_vals)

        for name in layer_names:
            a = layer_acts[name]
            if len(a.shape) == 4:
                A = tf.reduce_mean(tf.abs(a), axis=(0,1,2)).numpy()
                V = tf.math.reduce_variance(a, axis=(0,1,2)).numpy()
            else:
                A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
                V = tf.math.reduce_variance(a, axis=0).numpy()

            grad = tape.gradient(loss, a)
            if grad is None:
                G = np.zeros_like(A)
            else:
                if len(grad.shape) == 4:
                    G = tf.reduce_mean(tf.abs(grad), axis=(0,1,2)).numpy()
                else:
                    G = tf.reduce_mean(tf.abs(grad), axis=0).numpy()

            results[name].append(A)
            var_results[name].append(V)
            grad_results[name].append(G)
        del tape

    stats = {}
    for name in layer_names:
        A = np.mean(results[name], axis=0)
        V = np.mean(var_results[name], axis=0)
        G = np.mean(grad_results[name], axis=0)
        stats[name] = (A, G, V)
        log(f"{name}: len={len(A)} meanA={A.mean():.6e} meanG={G.mean():.6e} meanV={V.mean():.6e}")
    return stats

# ---------------------------
# Importance scores & masks
# ---------------------------
def compute_importance_scores(stats, alpha=ALPHA, beta=BETA, gamma=GAMMA):
    def normalize(x):
        x = x - x.min()
        if x.max() > 0:
            x = x / x.max()
        return x
    scores = {}
    for name, (A, G, V) in stats.items():
        scores[name] = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)
    return scores

def make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO):
    masks = {}
    for name, score in score_map.items():
        k = max(1, int(len(score) * keep_ratio))
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)
        masks[name] = mask
        log(f"{name}: keep {int(mask.sum())}/{len(mask)}")
    return masks

# ---------------------------
# Mask gate layer & masked model
# ---------------------------
class CNNGate(tf.keras.layers.Layer):
    def __init__(self, channels, mask=None, **kwargs):
        """
        channels: int number of channels this gate controls
        mask: 1D array-like of length 'channels' with 0/1 values (or floats in [0,1]).
              If provided, gate variable is initialized to these values; otherwise ones.
        """
        super().__init__(**kwargs)
        self.channels = int(channels)
        # store mask as numpy array if provided (for serialization convenience)
        self._init_mask = None if mask is None else np.array(mask, dtype=np.float32)
        init_val = self._init_mask if self._init_mask is not None else np.ones((self.channels,), dtype=np.float32)
        # gate is non-trainable scalar per-channel multiplier
        self.gate = self.add_weight(
            name="gate",
            shape=(self.channels,),
            initializer=tf.keras.initializers.Constant(init_val),
            trainable=False,
            dtype=tf.float32,
        )

    def call(self, inputs):
        # supports inputs with shape [batch, H, W, C] or [batch, C] (works broadcasting)
        g = self.gate
        # expand dims to match channels in conv output
        if len(inputs.shape) == 4:
            return inputs * g[None, None, None, :]
        elif len(inputs.shape) == 2:
            return inputs * g[None, :]
        else:
            # fallback broadcasting
            return inputs * g

    def get_config(self):
        cfg = super().get_config()
        # do NOT embed gate values directly here (weights are saved by Keras),
        # but we include channels for reconstruction convenience.
        cfg.update({
            "channels": self.channels,
            # don't include mask/gate here to avoid duplicating weight data;
            # the gate weight will be saved/loaded normally by Keras.
        })
        return cfg

    @classmethod
    def from_config(cls, config):
        # config may only contain "channels" — weight values will be restored by Keras load.
        channels = config.pop("channels")
        return cls(channels=channels, **config)


def build_masked_model(orig_model, masks):
    """
    Build a new functional model with cloned layers from orig_model and insert a CNNGate
    after each layer named in `masks`. The gate is initialized from masks[layer_name].
    """
    log("Building masked model with gates (cloning layers)...")
    inp = tf.keras.Input(shape=orig_model.input_shape[1:])
    x = inp

    # keep mapping from original layer name -> new layer object (for weight copying)
    for layer in orig_model.layers:
        # clone layer if possible to avoid reusing original layer objects
        try:
            cfg = layer.get_config()
            Cls = layer.__class__
            new_layer = Cls.from_config(cfg)
        except Exception:
            # fallback: try to reuse the layer (less safe)
            new_layer = layer

        # apply the new layer to current tensor
        x = new_layer(x)

        # copy weights if layer had weights and we cloned it
        try:
            w = layer.get_weights()
            if w:
                try:
                    new_layer.set_weights(w)
                except Exception:
                    # some layers (e.g., fused ops) may not accept direct set_weights — ignore
                    pass
        except Exception:
            pass

        # insert gate if this original layer is in masks
        if layer.name in masks:
            mask = np.array(masks[layer.name], dtype=np.float32)
            channels = int(mask.shape[0])
            gate_layer = CNNGate(channels=channels, mask=mask, name=layer.name + "_gate")
            x = gate_layer(x)
            # gate weight is already initialized from mask in CNNGate.__init__, so nothing else to do

    masked = tf.keras.Model(inputs=inp, outputs=x, name=(orig_model.name or "model") + "_masked")
    log("Masked model created:", masked.name)
    return masked

# ---------------------------
# Structural pruning (safe)
# ---------------------------
def prune_structural_sequential(orig_model, masks, input_shape):
    """
    Structural pruning for Sequential models.
    - Prune Conv2D output filters using masks[layer.name]
    - Prune Dense output units only (do not slice Dense input rows that come from Flatten/Conv)
    - Attempt to slice BatchNorm params to match conv outputs
    """
    log("Structural pruning (safe) start...")
    new_layers = []
    prev_was_conv_like = False  # indicates that Flatten/Conv preceded Dense inputs

    for layer in orig_model.layers:
        # Conv2D: slice output channels
        if isinstance(layer, layers.Conv2D):
            W, b = layer.get_weights()
            orig_out = W.shape[-1]
            mask = masks.get(layer.name, np.ones(orig_out, dtype=np.float32))
            keep_idx = np.where(mask == 1)[0]
            if keep_idx.size == 0:
                keep_idx = np.array([int(np.argmax(mask))], dtype=np.int32)
            W_new = W[:, :, :, keep_idx]
            b_new = b[keep_idx]
            new_conv = layers.Conv2D(
                filters=len(keep_idx),
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=layer.activation,
                use_bias=layer.use_bias,
                name=layer.name + "_pruned"
            )
            new_layers.append((new_conv, [W_new, b_new]))
            prev_was_conv_like = True
            continue

        # BatchNorm: slice params if prev was conv-like
        if isinstance(layer, layers.BatchNormalization):
            try:
                weights = layer.get_weights()
                if prev_was_conv_like and new_layers:
                    prev_layer_obj, prev_w = new_layers[-1]
                    if prev_w is not None:
                        out_ch = prev_w[0].shape[-1]  # kernel last dim
                        gamma, beta, mean, var = weights
                        gamma = gamma[:out_ch]
                        beta = beta[:out_ch]
                        mean = mean[:out_ch]
                        var = var[:out_ch]
                        new_bn = layers.BatchNormalization.from_config(layer.get_config())
                        new_layers.append((new_bn, [gamma, beta, mean, var]))
                        continue
            except Exception:
                pass
            # fallback keep BN as-is
            try:
                new_bn = layers.BatchNormalization.from_config(layer.get_config())
                new_layers.append((new_bn, layer.get_weights()))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            # prev_was_conv_like unchanged
            continue

        # MaxPool/Activation/Dropout/Flatten/GlobalAvgPool: clone or reuse
        if isinstance(layer, (layers.MaxPooling2D, layers.Activation, layers.ReLU, layers.Dropout, layers.Flatten, layers.GlobalAveragePooling2D)):
            try:
                cloned = layer.__class__.from_config(layer.get_config())
                w = layer.get_weights() if hasattr(layer, "get_weights") else None
                new_layers.append((cloned, w if w else None))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            if isinstance(layer, layers.Flatten) or isinstance(layer, layers.GlobalAveragePooling2D):
                prev_was_conv_like = True
            continue

        # Dense: prune outputs only (safe)
        if isinstance(layer, layers.Dense):
            W, b = layer.get_weights()  # shape (in_dim, out_dim)
            out_mask = masks.get(layer.name, np.ones(W.shape[1], dtype=np.float32))
            out_idx = np.where(out_mask == 1)[0]
            if out_idx.size == 0:
                out_idx = np.array([int(np.argmax(out_mask))], dtype=np.int32)
            W_new = W[:, out_idx]   # keep all input rows (safe)
            b_new = b[out_idx]
            new_dense = layers.Dense(units=W_new.shape[1], activation=layer.activation, name=layer.name + "_pruned")
            new_layers.append((new_dense, [W_new, b_new]))
            prev_was_conv_like = False
            continue

        # Fallback for other layers
        try:
            cloned = layer.__class__.from_config(layer.get_config())
            w = layer.get_weights() if hasattr(layer, "get_weights") else None
            new_layers.append((cloned, w if w else None))
        except Exception:
            new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
        prev_was_conv_like = False

    # Build new Sequential model with InputLayer
    seq = models.Sequential(name=orig_model.name + "_struct_pruned")
    seq.add(layers.InputLayer(input_shape=tuple(input_shape)))
    for lyr_obj, w in new_layers:
        seq.add(lyr_obj)
        if w is not None:
            try:
                seq.layers[-1].set_weights(w)
            except Exception as e:
                log("Warning: couldn't set weights for", seq.layers[-1].name, ":", e)
    log("Structural pruning complete. New model summary:")
    seq.summary()
    return seq

# ---------------------------
# FLOPS and timing helpers
# ---------------------------
def calculate_conv_flops(input_shape, kernel_shape, strides=(1,1), padding='same'):
    h_in, w_in, c_in = input_shape
    kh, kw, _, c_out = kernel_shape
    if padding == 'same':
        h_out = math.ceil(h_in / strides[0])
        w_out = math.ceil(w_in / strides[1])
    else:
        h_out = math.ceil((h_in - kh + 1) / strides[0])
        w_out = math.ceil((w_in - kw + 1) / strides[1])
    flops = h_out * w_out * (kh * kw * c_in) * c_out * 2
    return flops, (h_out, w_out, c_out)

def calculate_dense_flops(in_size, out_size):
    return in_size * out_size * 2

def calculate_model_flops(model, input_shape):
    total = 0
    current_shape = tuple(input_shape)
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            weights = layer.get_weights()
            if not weights:
                continue
            kernel_shape = weights[0].shape  # (kh, kw, in_c, out_c)
            layer_flops, current_shape = calculate_conv_flops(current_shape, kernel_shape, strides=layer.strides, padding=layer.padding)
            total += layer_flops
        elif isinstance(layer, layers.Flatten):
            current_shape = (int(np.prod(current_shape)),)
        elif isinstance(layer, layers.Dense):
            in_size = current_shape[0] if isinstance(current_shape, tuple) and len(current_shape)>0 else int(current_shape)
            layer_flops = calculate_dense_flops(in_size, layer.units)
            total += layer_flops
            current_shape = (layer.units,)
        elif isinstance(layer, layers.MaxPooling2D):
            h,w,c = current_shape
            pool = layer.pool_size[0] if hasattr(layer.pool_size, "__getitem__") else layer.pool_size
            current_shape = (h//pool, w//pool, c)
        else:
            # ignore other layers for shape changes
            pass
    return total

def measure_inference_time(model, sample_batch, steps=20):
    model.predict(sample_batch, verbose=0)  # warmup
    t0 = time.time()
    for _ in range(steps):
        model.predict(sample_batch, verbose=0)
    t1 = time.time()
    return (t1 - t0) / steps

# ---------------------------
# Save masks helper
# ---------------------------
def save_masks(masks, path):
    serial = {k: v.tolist() for k,v in masks.items()}
    with open(path, "w") as f:
        json.dump(serial, f, indent=2)
    log("Saved masks to", path)

# ---------------------------
# Plot helper
# ---------------------------
def plot_mask_histograms(masks, outdir=SAVE_DIR):
    if not PLOT_RESULTS:
        return
    for name, mask in masks.items():
        plt.figure(figsize=(5,2))
        plt.title(name)
        plt.hist(mask, bins=2)
        plt.xlabel("0=pruned, 1=kept")
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"mask_{name}.png"))
        plt.close()

# ---------------------------
# MAIN pipeline
# ---------------------------
def full_pipeline(model_path, dataset_path):
    # 1) load model (safe)
    model = safe_load_model(model_path)
    log("Model loaded. Summary:")
    model.summary()

    # 2) load dataset and infer num_classes
    # We need input size so infer from model
    input_shape = model.input_shape[1:]
    log("Inferred input shape:", input_shape)

    train_ds, val_ds = load_image_folder_dataset(dataset_path, image_size=input_shape[:2], batch_size=BATCH_SIZE)
    # Determine number of classes from dataset
    try:
        # tf.data.Dataset from image_dataset_from_directory has .class_names on the original Dataset returned object,
        # but not on the batched dataset — so inspect via a fresh loader:
        tmp = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=input_shape[:2], batch_size=1)
        num_classes = len(tmp.class_names)
        del tmp
    except Exception:
        # fallback: infer from labels in train_ds
        classes = set()
        for _, y in train_ds.take(10):
            classes.update(y.numpy().tolist())
        num_classes = max(classes) + 1 if classes else 2

    log("Detected dataset classes (num_classes):", num_classes)

    # 3) ensure model output matches dataset classes
    model, loss_fn = ensure_output_matches_dataset(model, num_classes)
    # compile original so evaluate works (use small lr default) with detected loss
    if isinstance(loss_fn, tf.keras.losses.BinaryCrossentropy):
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_fn, metrics=["accuracy"])
    else:
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_fn, metrics=["accuracy"])

    # reprint summary
    log("Final model used (after potential rebuild). Summary:")
    model.summary()

    # calibration subset for stats
    calib_ds = train_ds.take(CALIB_BATCHES)

    # sample for timing
    try:
        sample_x, _ = next(iter(val_ds))
    except Exception:
        sample_x, _ = next(iter(train_ds))
    sample_x_small = sample_x[:min(16, sample_x.shape[0])]

    # 4) choose layers to prune (conv + hidden dense only, not final output)
    dense_layers = [lyr for lyr in model.layers if isinstance(lyr, layers.Dense)]
    last_dense = dense_layers[-1] if dense_layers else None

    prune_layer_names = []
    for lyr in model.layers:
        if isinstance(lyr, layers.Conv2D):
            prune_layer_names.append(lyr.name)
        elif isinstance(lyr, layers.Dense) and lyr is not last_dense:
            prune_layer_names.append(lyr.name)
    log("Layers considered for pruning:", prune_layer_names)

    # 5) compute stats (pass loss_fn)
    stats = compute_activation_grad_stats(model, prune_layer_names, calib_ds, loss_fn=loss_fn, max_batches=CALIB_BATCHES)

    # 6) importance & masks
    score_map = compute_importance_scores(stats)
    masks = make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO)
    save_masks(masks, os.path.join(SAVE_DIR, "masks.json"))
    plot_mask_histograms(masks)

    # 7) build masked model and compile (use same loss)
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=loss_fn, metrics=["accuracy"])
    log("Masked model built.")

    # quick eval before FT (small subset)
        # quick eval before FT (small subset)
    try:
        loss0, acc0 = masked_model.evaluate(val_ds.take(5), verbose=0)
        log("Masked model pre-FT acc:", acc0)
    except Exception as e:
        log("Masked model pre-eval failed:", e)
        acc0 = None

    # 8) measure baseline flops & time
    baseline_flops = calculate_model_flops(model, input_shape)
    baseline_time = measure_inference_time(model, sample_x_small, steps=10)
    log(f"Baseline FLOPS: {baseline_flops:,}, baseline time (avg batch): {baseline_time:.4f}s")

    # 9) fine-tune masked model
    try:
        masked_model.fit(
            train_ds.take(FT_BATCHES_TO_USE),
            validation_data=val_ds.take(5),
            epochs=FT_EPOCHS,
            verbose=2
        )
    except Exception as e:
        log("Masked fine-tune failed/partial:", e)

    # 10) structural prune
    # NOTE: you can prune the original model or the masked_model.
    #       pruning masked_model preserves gating decisions — often desired.
    try:
        pruned_model = prune_structural_sequential(masked_model, masks, input_shape)
    except Exception:
        # fallback to pruning original model if pruning masked_model fails
        pruned_model = prune_structural_sequential(model, masks, input_shape)

    # compile pruned model for evaluation / training with same loss
    pruned_model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss=loss_fn,
        metrics=["accuracy"]
    )

    log("Starting fine-tuning of PRUNED model (EarlyStopping)...")
    early_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
        restore_best_weights=True
    )
    try:
        pruned_model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=20,
            batch_size=BATCH_SIZE,
            callbacks=[early_cb],
            verbose=2
        )
    except Exception as e:
        log("Pruned model fine-tune failed/partial:", e)

    # 11) evaluate models
    log("Evaluating Original model:")
    orig_loss, orig_acc = model.evaluate(val_ds, verbose=0)
    log("Original acc:", orig_acc)

    log("Evaluating Masked model (after FT):")
    try:
        mask_loss, mask_acc = masked_model.evaluate(val_ds, verbose=0)
        log("Masked acc:", mask_acc)
    except Exception as e:
        log("Masked evaluate failed:", e)
        mask_acc = None

    log("Evaluating Pruned model:")
    try:
        pruned_loss, pruned_acc = pruned_model.evaluate(val_ds, verbose=0)
        log("Pruned acc:", pruned_acc)
    except Exception as e:
        log("Pruned evaluate failed:", e)
        pruned_acc = None

    # 12) FLOPS & timing after prune
    pruned_flops = calculate_model_flops(pruned_model, input_shape)
    pruned_time = measure_inference_time(pruned_model, sample_x_small, steps=10)
    log(f"Pruned FLOPS: {pruned_flops:,}, pruned time: {pruned_time:.4f}s")

    # 13) summary & save
    reduction = 1.0 - (pruned_flops / baseline_flops) if baseline_flops > 0 else 0.0
    log("="*60)
    log("SUMMARY:")
    log(f"Baseline FLOPS: {baseline_flops:,}")
    log(f"Pruned FLOPS: {pruned_flops:,}")
    log(f"FLOPS reduction: {reduction:.2%}")
    log(f"Original acc: {orig_acc}, Masked acc: {mask_acc}, Pruned acc: {pruned_acc}")
    log("="*60)

    # ---- Extra Metrics: GFLOPS + Accuracy Reduction ----
    baseline_gflops = baseline_flops / 1e9
    pruned_gflops = pruned_flops / 1e9
    gflops_reduction = 1.0 - (pruned_gflops / baseline_gflops) if baseline_gflops > 0 else 0.0
    acc_reduction = (orig_acc - pruned_acc) if (orig_acc is not None and pruned_acc is not None) else None

    log(f"Baseline GFLOPS: {baseline_gflops:.4f}")
    log(f"Pruned GFLOPS: {pruned_gflops:.4f}")
    log(f"GFLOPS reduction: {gflops_reduction:.2%}")
    if acc_reduction is not None:
        log(f"Accuracy reduction: {acc_reduction:.4f}")
    else:
        log("Accuracy reduction: N/A")

    # save artifacts
    try:
        # use proper file extensions to avoid Keras save errors
        baseline_name = base_name + "_baseline.keras"
        masked_name   = base_name + "_masked.keras"
        pruned_name   = base_name + "_pruned.keras"

        model.save(os.path.join(SAVE_DIR, baseline_name))
        masked_model.save(os.path.join(SAVE_DIR, masked_name))
        pruned_model.save(os.path.join(SAVE_DIR, pruned_name))

        log(f"Saved models under names: {baseline_name}, {masked_name}, {pruned_name}")

        log("Saved models to", SAVE_DIR)
    except Exception as e:
        log("Save models failed:", e)

    return {
        "model": model,
        "masked_model": masked_model,
        "pruned_model": pruned_model,
        "masks": masks,
        "baseline_flops": baseline_flops,
        "pruned_flops": pruned_flops,
        "baseline_time": baseline_time,
        "pruned_time": pruned_time,
        "orig_acc": orig_acc,
        "mask_acc": mask_acc,
        "pruned_acc": pruned_acc
    }

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    out = full_pipeline(MODEL_PATH, DATASET_PATH)
    log("Pipeline finished. Outputs:", out.keys())


[INFO] Loading model: D:\college\sem-8\models\garbage_cnn_model.h5
[INFO] Loaded model directly.
[INFO] Model loaded. Summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,564,998 (36.49 MB)

 Trainable params: 9,564,998 (36.49 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8_pruned (Conv2D)        │ (None, 148, 148, 26)   │           728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 74, 74, 26)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9_pruned (Conv2D)        │ (None, 72, 72, 53)     │        12,455 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 53)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10_pruned (Conv2D)       │ (None, 34, 34, 106)    │        50,668 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 106)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 30634)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4_pruned (Dense)          │ (None, 212)            │     6,494,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 212)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5_pruned (Dense)          │ (None, 6)              │         1,278 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,559,749 (25.02 MB)

 Trainable params: 6,559,749 (25.02 MB)

 Non-trainable params: 0 (0.00 B)

[INFO] Starting fine-tuning of PRUNED model (EarlyStopping)...
Epoch 1/20
32/32 - 23s - 724ms/step - accuracy: 0.3398 - loss: 1.5953 - val_accuracy: 0.3842 - val_loss: 1.4580
Epoch 2/20
32/32 - 19s - 597ms/step - accuracy: 0.4545 - loss: 1.3824 - val_accuracy: 0.4436 - val_loss: 1.3553
Epoch 3/20
32/32 - 21s - 655ms/step - accuracy: 0.4931 - loss: 1.2840 - val_accuracy: 0.4634 - val_loss: 1.3128
Epoch 4/20
32/32 - 17s - 532ms/step - accuracy: 0.5223 - loss: 1.2243 - val_accuracy: 0.4891 - val_loss: 1.2259
Epoch 5/20
32/32 - 17s - 533ms/step - accuracy: 0.5569 - loss: 1.1549 - val_accuracy: 0.5307 - val_loss: 1.2143
Epoch 6/20
32/32 - 17s - 533ms/step - accuracy: 0.5692 - loss: 1.1153 - val_accuracy: 0.5485 - val_loss: 1.1570
Epoch 7/20
32/32 - 18s - 573ms/step - accuracy: 0.5989 - loss: 1.0659 - val_accuracy: 0.5545 - val_loss: 1.1248
Epoch 8/20
32/32 - 17s - 534ms/step - accuracy: 0.6182 - loss: 1.0355 - val_accuracy: 0.5842 - val_loss: 1.0907
Epoch 9/20
32/32 - 18s - 574ms/step - acc

# **CNN user input masking only**

In [55]:
"""
sequential_cnn_mask_only.py

Mask-only pruning pipeline (Sequential CNN)
- No structural pruning (no physical removal of filters)
- Builds masked model with gates, fine-tunes masked model
- Computes baseline FLOPS and effective FLOPS when applying masks (estimation)
- Measures inference time and prints metrics
"""

import os
import json
import math
import time
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# ---------------------------
# USER CONFIG
# ---------------------------
MODEL_PATH = r"D:\college\sem-8\models\pothole_cnn_model.h5"
DATASET_PATH = r"D:\college\sem-8\dataset\pothole"
SAVE_DIR = r"pruning_output"
MODEL_SAVE_EXT = ".keras"   # use ".keras" or ".h5"

KEEP_RATIO = 0.7
ALPHA, BETA, GAMMA = 0.4, 0.3, 0.3
BATCH_SIZE = 64
CALIB_BATCHES = 30
FT_EPOCHS = 5
FT_BATCHES_TO_USE = 150
PLOT_RESULTS = True
VERBOSE = True

os.makedirs(SAVE_DIR, exist_ok=True)
tf.random.set_seed(42)
np.random.seed(42)

def log(*args):
    if VERBOSE:
        print("[INFO]", *args)

# ---------------------------
# Safe load model (handles '/' in h5 layer names)
# ---------------------------
def safe_load_model(model_path):
    log("Loading model:", model_path)
    try:
        m = tf.keras.models.load_model(model_path, compile=False)
        log("Loaded model directly.")
        return m
    except Exception as e:
        log("Direct load failed:", e)
    # try sanitizing H5 model_config
    try:
        with h5py.File(model_path, "r") as f:
            if "model_config" in f:
                raw = f["model_config"][()]
                if isinstance(raw, bytes):
                    raw = raw.decode("utf-8")
                cfg_json = json.loads(raw)
                changed = False
                for layer in cfg_json.get("config", {}).get("layers", []):
                    cfg = layer.get("config", {})
                    name = cfg.get("name")
                    if isinstance(name, str) and "/" in name:
                        new_name = name.replace("/", "_")
                        cfg["name"] = new_name
                        changed = True
                        log(f"[FIX] layer name: {name} -> {new_name}")
                if changed:
                    fixed_json = json.dumps(cfg_json)
                    model = tf.keras.models.model_from_json(fixed_json)
                    model.load_weights(model_path)
                    log("Loaded model from sanitized JSON + weights.")
                    return model
    except Exception as e2:
        log("H5 sanitization attempt failed:", e2)
    # final fallback
    try:
        m = tf.keras.models.load_model(model_path, compile=False, safe_mode=False)
        log("Loaded model with safe_mode=False.")
        return m
    except Exception as e3:
        log("All load attempts failed:", e3)
        raise RuntimeError("Failed to load model. Ensure path and format are correct.")

# ---------------------------
# Dataset loader (folder)
# ---------------------------
def load_image_folder_dataset(path, image_size, batch_size=BATCH_SIZE):
    log("Loading dataset folder:", path, "image_size:", image_size)
    train_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    rescaler = layers.Rescaling(1.0 / 255)
    train_ds = train_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds

# ---------------------------
# Stats (activation, grad, variance)
# ---------------------------
def compute_activation_grad_stats(model, layer_names, dataset, max_batches=CALIB_BATCHES):
    log("Computing activation & gradient stats...")
    results_A, results_V, results_G = {n:[] for n in layer_names}, {n:[] for n in layer_names}, {n:[] for n in layer_names}
    batch_count = 0
    for x_batch, y_batch in dataset:
        if batch_count >= max_batches:
            break
        batch_count += 1
        layer_acts = {}
        with tf.GradientTape(persistent=True) as tape:
            a = x_batch
            for layer in model.layers:
                a = layer(a)
                if layer.name in layer_names:
                    tape.watch(a)
                    layer_acts[layer.name] = a
            preds = a
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, preds)
            loss = tf.reduce_mean(loss)
        for name in layer_names:
            a = layer_acts[name]
            if len(a.shape) == 4:
                A = tf.reduce_mean(tf.abs(a), axis=(0,1,2)).numpy()
                V = tf.math.reduce_variance(a, axis=(0,1,2)).numpy()
            else:
                A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
                V = tf.math.reduce_variance(a, axis=0).numpy()
            grad = tape.gradient(loss, a)
            if grad is None:
                G = np.zeros_like(A)
            else:
                if len(grad.shape) == 4:
                    G = tf.reduce_mean(tf.abs(grad), axis=(0,1,2)).numpy()
                else:
                    G = tf.reduce_mean(tf.abs(grad), axis=0).numpy()
            results_A[name].append(A)
            results_V[name].append(V)
            results_G[name].append(G)
        del tape
    stats = {}
    for name in layer_names:
        A = np.mean(results_A[name], axis=0)
        V = np.mean(results_V[name], axis=0)
        G = np.mean(results_G[name], axis=0)
        stats[name] = (A, G, V)
        log(f"{name}: units={len(A)} meanA={A.mean():.6e} meanG={G.mean():.6e} meanV={V.mean():.6e}")
    return stats

# ---------------------------
# Importance scores & masks
# ---------------------------
def compute_importance_scores(stats, alpha=ALPHA, beta=BETA, gamma=GAMMA):
    def normalize(x):
        x = x - x.min()
        if x.max() > 0:
            x = x / x.max()
        return x
    scores = {}
    for name, (A,G,V) in stats.items():
        scores[name] = alpha*normalize(A) + beta*normalize(G) + gamma*normalize(V)
    return scores

def make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO):
    masks = {}
    for name, score in score_map.items():
        k = max(1, int(len(score) * keep_ratio))
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)
        masks[name] = mask
        log(f"{name}: keep {int(mask.sum())}/{len(mask)}")
    return masks

# ---------------------------
# Mask gate & masked model
# ---------------------------
class CNNGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        self.mask = tf.constant(mask, dtype=tf.float32) if mask is not None else None
    def call(self, x):
        if self.mask is None:
            return x
        if len(x.shape) == 4:
            return x * tf.reshape(self.mask, (1,1,1,-1))
        else:
            return x * self.mask

def build_masked_model(orig_model, masks):
    log("Building masked model with gates (no structural changes)...")
    inp = tf.keras.Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = CNNGate(mask=masks[layer.name], name=layer.name + "_gate")(x)
    masked = tf.keras.Model(inputs=inp, outputs=x, name=orig_model.name + "_masked")
    # best-effort copy weights to identically named layers
    for layer in orig_model.layers:
        try:
            masked_layer = masked.get_layer(layer.name)
            masked_layer.set_weights(layer.get_weights())
        except Exception:
            pass
    return masked

# ---------------------------
# FLOPS helpers: baseline & effective (using masks)
# ---------------------------
def calculate_conv_flops(input_shape, kernel_shape, strides=(1,1), padding='same'):
    h_in, w_in, c_in = input_shape
    kh, kw, _, c_out = kernel_shape
    if padding == 'same':
        h_out = math.ceil(h_in / strides[0])
        w_out = math.ceil(w_in / strides[1])
    else:
        h_out = math.ceil((h_in - kh + 1) / strides[0])
        w_out = math.ceil((w_in - kw + 1) / strides[1])
    flops = h_out * w_out * (kh * kw * c_in) * c_out * 2
    return flops, (h_out, w_out, c_out)

def calculate_dense_flops(in_size, out_size):
    return in_size * out_size * 2

def calculate_model_flops(model, input_shape):
    total = 0
    current_shape = tuple(input_shape)
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            weights = layer.get_weights()
            if not weights:
                continue
            kernel_shape = weights[0].shape
            layer_flops, current_shape = calculate_conv_flops(current_shape, kernel_shape, strides=layer.strides, padding=layer.padding)
            total += layer_flops
        elif isinstance(layer, layers.Flatten):
            current_shape = (int(np.prod(current_shape)),)
        elif isinstance(layer, layers.Dense):
            in_size = current_shape[0] if isinstance(current_shape, tuple) and len(current_shape)>0 else int(current_shape)
            layer_flops = calculate_dense_flops(in_size, layer.units)
            total += layer_flops
            current_shape = (layer.units,)
        elif isinstance(layer, layers.MaxPooling2D):
            h,w,c = current_shape
            pool = layer.pool_size[0] if hasattr(layer.pool_size, "__getitem__") else layer.pool_size
            current_shape = (h//pool, w//pool, c)
        else:
            # ignore other layers
            pass
    return total

def calculate_effective_flops_with_masks(model, masks, input_shape):
    """
    Estimate effective FLOPS when applying masks (gates).
    For Conv layers, reduce out_channels by mask.sum().
    For Dense layers, reduce output units by mask.sum() if masked.
    This is an estimate (masked model still contains same kernels; runtime reduction depends on backend).
    """
    total = 0
    current_shape = tuple(input_shape)
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            weights = layer.get_weights()
            if not weights:
                continue
            kernel_shape = weights[0].shape  # (kh, kw, in_c, out_c)
            orig_out = kernel_shape[3]
            if layer.name in masks:
                effective_out = int(np.sum(masks[layer.name]))
            else:
                effective_out = orig_out
            kh, kw = kernel_shape[0], kernel_shape[1]
            # use current in_channels from current_shape
            h_in, w_in, c_in = current_shape
            # if mask existed on previous conv, c_in should be adjusted in this estimation:
            # We will approximate c_in as kernel_shape[2] scaled by ratio of previous mask if previous named layer was masked.
            # Simpler estimation: use kernel_shape[2] as in_channels
            c_in_effective = kernel_shape[2]
            if layer.name in masks:
                # compute output dims
                strides = layer.strides
                padding = layer.padding
                h_out = math.ceil(h_in / strides[0]) if padding == 'same' else math.ceil((h_in - kh + 1)/strides[0])
                w_out = math.ceil(w_in / strides[1]) if padding == 'same' else math.ceil((w_in - kw + 1)/strides[1])
            else:
                strides = layer.strides
                padding = layer.padding
                kh, kw = kernel_shape[0], kernel_shape[1]
                if padding == 'same':
                    h_out = math.ceil(h_in / strides[0])
                    w_out = math.ceil(w_in / strides[1])
                else:
                    h_out = math.ceil((h_in - kh + 1)/strides[0])
                    w_out = math.ceil((w_in - kw + 1)/strides[1])
            flops = h_out * w_out * (kh * kw * c_in_effective) * effective_out * 2
            total += flops
            current_shape = (h_out, w_out, effective_out)
        elif isinstance(layer, layers.Flatten):
            current_shape = (int(np.prod(current_shape)),)
        elif isinstance(layer, layers.Dense):
            in_size = current_shape[0] if isinstance(current_shape, tuple) and len(current_shape)>0 else int(current_shape)
            if layer.name in masks:
                effective_out = int(np.sum(masks[layer.name]))
            else:
                effective_out = layer.units
            layer_flops = calculate_dense_flops(in_size, effective_out)
            total += layer_flops
            current_shape = (effective_out,)
        elif isinstance(layer, layers.MaxPooling2D):
            h,w,c = current_shape
            pool = layer.pool_size[0] if hasattr(layer.pool_size, "__getitem__") else layer.pool_size
            current_shape = (h//pool, w//pool, c)
        else:
            pass
    return total

# ---------------------------
# inference timing
# ---------------------------
def measure_inference_time(model, sample_batch, steps=20):
    model.predict(sample_batch, verbose=0)
    t0 = time.time()
    for _ in range(steps):
        model.predict(sample_batch, verbose=0)
    t1 = time.time()
    return (t1 - t0) / steps

# ---------------------------
# save helpers
# ---------------------------
def save_masks(masks, path):
    serial = {k: v.tolist() for k, v in masks.items()}
    with open(path, "w") as f:
        json.dump(serial, f, indent=2)
    log("Saved masks to", path)

def save_model_safe(model, path_no_ext):
    ext = MODEL_SAVE_EXT if MODEL_SAVE_EXT.startswith(".") else "." + MODEL_SAVE_EXT
    fullpath = path_no_ext + ext
    try:
        model.save(fullpath)
        log("Saved model to", fullpath)
    except Exception as e:
        log("Failed to save model to", fullpath, ":", e)

def plot_mask_histograms(masks, outdir=SAVE_DIR):
    if not PLOT_RESULTS:
        return
    for name, mask in masks.items():
        plt.figure(figsize=(5,2))
        plt.title(name)
        plt.hist(mask, bins=2)
        plt.xlabel("0=pruned, 1=kept")
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"mask_{name}.png"))
        plt.close()

def per_layer_remaining_stats(masks):
    stats = {}
    for name, mask in masks.items():
        total = len(mask)
        kept = int(mask.sum())
        stats[name] = {"kept": kept, "total": total, "ratio": kept/total}
    return stats

# ---------------------------
# MAIN pipeline (mask-only)
# ---------------------------
def full_pipeline_mask_only(model_path, dataset_path):
    model = safe_load_model(model_path)
    log("Model loaded. Summary:")
    model.summary()

    # compile original for evaluation
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    input_shape = model.input_shape[1:]
    log("Inferred input shape:", input_shape)

    train_ds, val_ds = load_image_folder_dataset(dataset_path, image_size=input_shape[:2], batch_size=BATCH_SIZE)
    calib_ds = train_ds.take(CALIB_BATCHES)

    try:
        sample_x, _ = next(iter(val_ds))
    except Exception:
        sample_x, _ = next(iter(train_ds))
    sample_x_small = sample_x[:min(16, sample_x.shape[0])]

    # choose pruneable layers: conv + hidden dense (exclude final classifier)
    dense_layers = [lyr for lyr in model.layers if isinstance(lyr, layers.Dense)]
    last_dense = dense_layers[-1] if dense_layers else None
    prune_layer_names = []
    for lyr in model.layers:
        if isinstance(lyr, layers.Conv2D):
            prune_layer_names.append(lyr.name)
        elif isinstance(lyr, layers.Dense) and lyr is not last_dense:
            prune_layer_names.append(lyr.name)
    log("Layers considered for masking:", prune_layer_names)

    # compute stats, scores, masks
    stats = compute_activation_grad_stats(model, prune_layer_names, calib_ds, max_batches=CALIB_BATCHES)
    score_map = compute_importance_scores(stats)
    masks = make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO)
    save_masks(masks, os.path.join(SAVE_DIR, "masks.json"))
    plot_mask_histograms(masks)

    # build masked model
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    log("Masked model built.")

    # evaluate masked before FT
    try:
        loss0, acc0 = masked_model.evaluate(val_ds.take(5), verbose=0)
        log("Masked model pre-FT acc:", acc0)
    except Exception as e:
        log("Masked model pre-eval failed:", e)

    # baseline flops/time
    baseline_flops = calculate_model_flops(model, input_shape)
    baseline_gflops = baseline_flops / 1e9
    baseline_time = measure_inference_time(model, sample_x_small, steps=10)
    log(f"Baseline FLOPS: {baseline_flops:,} ({baseline_gflops:.6f} GFLOPS), baseline time (avg batch): {baseline_time:.4f}s")

    # fine-tune masked model
    try:
        masked_model.fit(train_ds.take(FT_BATCHES_TO_USE), validation_data=val_ds.take(5), epochs=FT_EPOCHS, verbose=2)
    except Exception as e:
        log("Masked fine-tune failed/partial:", e)

    # eval
    log("Evaluating Original model:")
    orig_loss, orig_acc = model.evaluate(val_ds, verbose=0)
    log("Original acc:", orig_acc)

    log("Evaluating Masked model (after FT):")
    try:
        mask_loss, mask_acc = masked_model.evaluate(val_ds, verbose=0)
        log("Masked acc:", mask_acc)
    except Exception as e:
        log("Masked evaluate failed:", e)
        mask_acc = None

    # Effective FLOPS after masking (estimate using masks)
    masked_effective_flops = calculate_effective_flops_with_masks(model, masks, input_shape)
    masked_effective_gflops = masked_effective_flops / 1e9

    # measure masked inference time (actual)
    masked_time = measure_inference_time(masked_model, sample_x_small, steps=10)
    log(f"Masked effective FLOPS (estimate): {masked_effective_flops:,} ({masked_effective_gflops:.6f} GFLOPS)")
    log(f"Masked actual inference time (avg batch): {masked_time:.4f}s")

    # metrics
    speedup_time = (baseline_time / masked_time) if masked_time > 0 else float("inf")
    flops_reduction = (1 - (masked_effective_flops / baseline_flops)) if baseline_flops > 0 else 0.0
    acc_drop_abs = (orig_acc - mask_acc) if (orig_acc is not None and mask_acc is not None) else None
    acc_drop_pct = (acc_drop_abs / orig_acc * 100) if (acc_drop_abs is not None and orig_acc != 0) else None

    orig_params = model.count_params()
    masked_params = masked_model.count_params()  # same structure -> same params, gates don't add many
    param_reduction_pct = (1 - masked_params / orig_params) * 100 if orig_params > 0 else 0.0

    layer_stats = per_layer_remaining_stats(masks)

    # summary
    log("="*70)
    log("FINAL SUMMARY (MASK-ONLY)")
    log("="*70)
    log(f"Baseline GFLOPS: {baseline_gflops:.6f} GFLOPS")
    log(f"Masked effective GFLOPS (estimate): {masked_effective_gflops:.6f} GFLOPS")
    log(f"FLOPS reduction (estimate): {flops_reduction*100:.2f}%")
    log(f"Baseline time (avg batch): {baseline_time:.4f}s")
    log(f"Masked time (avg batch):   {masked_time:.4f}s")
    log(f"Speed-up ratio (time): {speedup_time:.3f}x")
    if acc_drop_abs is not None:
        log(f"Original acc: {orig_acc:.4f}, Masked acc: {mask_acc:.4f}")
        log(f"Accuracy drop: {acc_drop_abs:.4f} ({acc_drop_pct:.2f}%)")
    else:
        log("Accuracy numbers unavailable for drop computation.")
    log(f"Original params: {orig_params:,}, Masked params: {masked_params:,}")
    log(f"Parameter reduction (structure-level): {param_reduction_pct:.2f}% (usually 0 for mask-only)")
    log("Per-layer remaining (kept/total):")
    for name, s in layer_stats.items():
        log(f"  {name}: {s['kept']}/{s['total']} ({s['ratio']:.2%})")
    log("="*70)

    # save artifacts
    save_masks(masks, os.path.join(SAVE_DIR, "masks.json"))
    try:
        save_model_safe(model, os.path.join(SAVE_DIR, "model_baseline"))
        save_model_safe(masked_model, os.path.join(SAVE_DIR, "model_masked"))
    except Exception as e:
        log("Save models failed:", e)

    metadata = {
        "baseline_flops": int(baseline_flops),
        "masked_effective_flops": int(masked_effective_flops),
        "baseline_gflops": float(baseline_gflops),
        "masked_effective_gflops": float(masked_effective_gflops),
        "baseline_time": float(baseline_time),
        "masked_time": float(masked_time),
        "orig_acc": float(orig_acc),
        "mask_acc": float(mask_acc) if mask_acc is not None else None,
        "orig_params": int(orig_params),
        "masked_params": int(masked_params),
        "flops_reduction_pct": float(flops_reduction*100),
        "per_layer_stats": layer_stats
    }
    with open(os.path.join(SAVE_DIR, "mask_only_summary.json"), "w") as f:
        json.dump(metadata, f, indent=2)

    log("Saved metadata to", os.path.join(SAVE_DIR, "mask_only_summary.json"))
    return metadata

# ---------------------------
# helper reused at end
# ---------------------------
def per_layer_remaining_stats(masks):
    stats = {}
    for name, mask in masks.items():
        total = len(mask)
        kept = int(mask.sum())
        stats[name] = {"kept": kept, "total": total, "ratio": kept/total}
    return stats

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    out = full_pipeline_mask_only(MODEL_PATH, DATASET_PATH)
    log("Pipeline finished. Metadata keys:", out.keys())


[INFO] Loading model: D:\college\sem-8\models\pothole_cnn_model.h5
[INFO] Loaded model directly.
[INFO] Model loaded. Summary:


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

[INFO] Inferred input shape: (224, 224, 3)
[INFO] Loading dataset folder: D:\college\sem-8\dataset\pothole image_size: (224, 224)
Found 681 files belonging to 2 classes.
Using 545 files for training.
Found 681 files belonging to 2 classes.
Using 136 files for validation.


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 